# Import files from snu-2021-1-ds-project-1
## wine_train_df, wine_test_df

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

wine_train_df_org = pd.read_csv('../input/snu-2021-1-ds-project-1/wine_train.csv')
wine_train_df = wine_train_df_org.copy()

wine_test_df_org = pd.read_csv('../input/snu-2021-1-ds-project-1/wine_test.csv')
wine_test_df = wine_test_df_org.copy()

# Import files from word-frequency
## word_high_df(상위 단어 150), word_low_df(하위 단어 150)
### 단어의 점수 = word_high_df.high/low & word_low_df.low/high

In [ ]:
word_high_df_org = pd.read_csv('../input/word-frequency/word_high.csv')
word_high_df = word_high_df_org.copy()
word_low_df_org = pd.read_csv('../input/word-frequency/word_low.csv')
word_low_df = word_low_df_org.copy()

In [ ]:
# 1. list & high/low만 vs 2. list 만 
word_high_df.drop(["Frequency","Frequency on High-point descriptions","Frequency on Low-point descriptions","total","Word_low"], axis = 1, inplace = True)
word_low_df.drop(["Frequency","Frequency on High-point descriptions","Frequency on Low-point descriptions","total","Word_high"], axis = 1, inplace = True)

In [ ]:
word_high_df.drop(word_high_df.loc[word_high_df['high/low']<10].index, inplace=True)
word_low_df.drop(word_low_df.loc[word_low_df['low/high']<3].index, inplace=True)

print(word_high_df.shape)
print(word_low_df.shape)

In [ ]:
from scipy.stats import skew
print(skew(np.log1p(word_high_df['high/low'])))

plt.title('log word high Histogram')
sns.distplot(np.log1p(word_high_df['high/low']))

In [ ]:
from scipy.stats import skew
print(skew(np.log1p(word_low_df['low/high'])))

plt.title('log word low Histogram')
sns.distplot(np.log1p(word_low_df['low/high']))

In [ ]:
# word_high_df['high/low'] = np.log1p(word_high_df['high/low'])
# word_low_df['low/high'] = np.log1p(word_low_df['low/high'])

In [ ]:
word_high_df

In [ ]:
word_high_list = word_high_df.values.tolist()
word_low_list = word_low_df.values.tolist()


# WINE_TRAIN_DF FEATURE ENGINEERING

# Extract description length from description

In [ ]:
# description length
length_list = []
for description in wine_train_df['description']:
    length_list.append(len(description))
wine_train_df['length'] = length_list

# Region_1 빈자리 Province로 채우기

In [ ]:
# region = region_1 fill with province
wine_train_df['region'] = np.where(pd.notnull(wine_train_df['region_1']) == True, wine_train_df['region_1'], wine_train_df['province'])

In [ ]:
print('데이터 전체의 Shape:', wine_train_df.shape)
print('\n전체 피처의 type\n', wine_train_df.dtypes.value_counts())
isnull_series = wine_train_df.isnull().sum()/116974*100
print('\nNull 칼럼과 그 건수:\n', isnull_series[isnull_series>0].sort_values(ascending=False))

In [ ]:
plt.title('Original Wine Price Histogram')
sns.distplot(wine_train_df['price'])

In [ ]:
plt.title('Log Transformed Wine Price Histogram')
log_price = np.log1p(wine_train_df['price'])
sns.distplot(log_price)

In [ ]:
plt.title('Original Wine Descriptrion Length Histogram')
sns.distplot(wine_train_df['length'])

In [ ]:
plt.title('Log Transformed Wine Descriptrion Length Histogram')
log_length = np.log1p(wine_train_df['length'])
sns.distplot(log_length)

# Price Log 변환

In [ ]:
# Price 로그 변환
original_price = wine_train_df['price']
wine_train_df['price'] = np.log1p(wine_train_df['price'])

# 칼럼 삭제
## 삭제된 칼럼: "designation", "province","region_1","region_2","taster_twitter_handle","title"

In [ ]:
# Null이 너무 많은 칼럼과 불필요한 칼럼 삭제
wine_train_df.drop(["designation", "province","region_1","region_2","taster_twitter_handle","title"], axis = 1, inplace = True)

In [ ]:
wine_train_df

# Grouping
## Grouping 할 칼럼: variety, winery, region
### 미리 만들어둔 list 활용(py file 참고)

In [ ]:
# grouping list 
variety_group = [['Chancellor', 'Airen', 'Aidani', 'Shiraz-Tempranillo', 'Picapoll', 'Gragnano', 'País', 'Forcallà', 'Paralleda', 'Chardonel', 'Premsal', 'Merlot-Petite Verdot', 'Athiri', 'Tempranillo Blanco', 'Brachetto', 'Chambourcin', 'Kinali Yapincak', 'Asprinio', 'Siegerrebe', 'Cabernet Sauvignon-Barbera', 'Pigato', 'Macabeo-Chardonnay', 'Verdosilla', 'Pinotage-Merlot', 'Morio Muskat', 'Espadeiro', 'Viognier-Valdiguié', 'Bobal-Cabernet Sauvignon', 'Norton', 'Seyval Blanc', 'Garnacha-Syrah', 'Viura-Chardonnay', 'Viura-Verdejo', 'Pignoletto', 'Marquette', 'Malvar', 'Cayuga', 'Traminette', 'Portuguese Rosé', 'Lambrusco', 'Rosado', 'Sauvignon Blanc-Verdejo', 'Chenin Blanc-Sauvignon Blanc', 'Merlot-Argaman', 'Blanc du Bois', 'Ekigaïna', 'Colombard-Ugni Blanc', 'Chardonnay-Pinot Gris', 'Pinot Blanc-Pinot Noir', 'Tinto Velasco', 'Verdejo-Sauvignon Blanc', 'Malvasia-Viura', 'Macabeo-Moscatel', 'Sideritis', 'Moschofilero-Chardonnay', 'Biancolella', 'Jacquez', 'Poulsard', 'Misket', 'Folle Blanche', 'Merseguera-Sauvignon Blanc', 'Chelois', 'Biancale', 'St. Vincent', 'Garnacha-Cabernet', 'Dafni', 'Tempranillo-Shiraz', 'Macabeo', 'Viura', 'Tempranillo-Garnacha'], ['Tempranillo-Merlot', 'Moscatel de Alejandría', 'Malbec-Syrah', 'Colombard-Sauvignon Blanc', 'Symphony', 'Vignoles', 'Chardonnay-Viognier', 'Chinuri', 'Rufete', 'Madeleine Angevine', 'Muscadine', 'Bastardo', 'Yapincak', 'Torrontés', 'Passerina', 'Coda di Volpe', 'Semillon-Chardonnay', 'Pinot Noir-Gamay', 'Xarel-lo', 'Pansa Blanca', 'Ugni Blanc-Colombard', 'Verdejo-Viura', 'Bobal', 'Sauvignon Gris', 'Garnacha', 'Inzolia', 'Portuguese Sparkling', 'Viognier-Chardonnay', 'Muscat Blanc à Petits Grains', 'Albanello', 'Loureiro-Arinto', 'Diamond', 'Sauvignon-Sémillon', 'Verdil', 'Sangiovese-Cabernet Sauvignon', 'Viognier-Gewürztraminer', 'Gros Plant', 'Grignolino', 'Rivaner', 'Cococciola', 'Azal', 'Portuguiser', 'Maria Gomes-Bical', 'Zlahtina', 'Malagouzia-Chardonnay', 'Grolleau', 'Papaskarasi', 'Merlot-Shiraz', 'Teroldego Rotaliano', 'Sauvignon Blanc-Sauvignon Gris', 'Cabernet Franc-Carmenère', 'Vranac', 'Silvaner-Traminer', 'Cabernet-Malbec', 'Garnacha-Tempranillo', 'Durif', 'Franconia', 'Colorino', 'Trajadura', 'Cabernet Franc-Lemberger', 'Parraleta', 'Kuntra', 'Johannisberg Riesling', 'Prieto Picudo', 'Baco Noir', 'Carineña', 'Cabernet Sauvignon-Tempranillo', 'Pinot Grigio', 'Garnacha Blanca', 'Verdejo'], ['Hondarrabi Zuri', 'Vinhão', 'Moscato', 'Gewürztraminer-Riesling', 'Marzemino', 'Chardonnay-Sauvignon', "Muscat d'Alexandrie", 'Tinta del Toro', 'Greco Bianco', 'Raboso', 'Muskat Ottonel', 'Mavrodaphne', 'Monastrell-Syrah', 'Aligoté', 'Prosecco', 'Cortese', 'Monica', 'Vilana', 'Roditis', 'Cabernet Moravia', 'Marsanne-Viognier', 'Freisa', 'Sacy', 'Shiraz-Cabernet', 'Blauer Portugieser', 'Alvarelhão', 'Çalkarası', 'Pinot Blanc-Chardonnay', 'Souzao', 'Verdeca', 'Tempranillo-Syrah', 'Ansonica', 'Insolia', 'Gaglioppo', 'Piedirosso', 'Carmenère', 'Bonarda', 'Malvasia', 'Dornfelder', 'Alvarinho-Chardonnay', 'Grechetto', 'Bical', 'Refosco', 'Manzoni', 'Chardonnay-Semillon', 'Cabernet Merlot', 'Aleatico', 'Malvasia Istriana', 'Emir', 'Rkatsiteli', 'Grenache Blend', 'Muskateller', 'Rosato', 'Negroamaro', 'Catarratto', 'Müller-Thurgau', 'Montepulciano', 'Ruché', 'Feteascǎ Regalǎ', 'Godello', 'Rosé', 'Xinomavro', 'Portuguese White', 'Vidal Blanc', 'Tempranillo-Cabernet Sauvignon', 'Grillo', 'Primitivo', 'Garnacha Tintorera', 'Trebbiano', 'Merlot-Malbec'], ['Nosiola', 'Morava', 'Kisi', 'Madeira Blend', 'Kangoun', 'Bombino Bianco', 'Groppello', 'Irsai Oliver', 'Tsolikouri', 'Mavroudi', 'Sauvignon Blanc-Chenin Blanc', 'Casavecchia', 'Xynisteri', 'Sauvignon Musqué', 'Fer Servadou', 'Kekfrankos', 'Malvasia di Candia', 'Molinara', 'Muscat of Alexandria', 'Malbec-Bonarda', 'Garnacha-Monastrell', 'Plyto', 'Bombino Nero', 'Saperavi-Merlot', 'Pinot Blanc-Viognier', 'Syrah-Bonarda', 'Malbec-Tempranillo', 'Pinot Grigio-Sauvignon Blanc', 'Karalahna', 'Valvin Muscat', 'Blauburger', 'Grenache Gris', 'Tintilia', 'Colombard', 'Tocai Friulano', 'Agiorgitiko', 'Narince', 'Carmenère-Syrah', 'Moscato Giallo', 'Pinot Auxerrois', 'Trousseau', 'Malvasia Nera', 'Loureiro', 'Valdiguié', 'Falanghina', 'Monastrell', 'Merlot', 'Arinto', 'Marsanne-Roussanne', 'Arneis', 'Glera', 'Castelão', 'Mansois', 'Maturana', 'Dolcetto', 'Mondeuse', 'Verdelho', 'Siria', 'Chenin Blanc-Viognier', 'Vernaccia', 'Lemberger', 'Lambrusco Grasparossa', 'Cabernet Sauvignon-Carmenère', 'Pecorino', 'Treixadura', 'Muscatel', 'Vranec', 'White Blend', 'Pinotage', 'Fernão Pires'], ['Cabernet', 'Alicante', 'Sauvignon Blanc', 'Syrah-Cabernet', 'Grecanico', "Nero d'Avola", 'Merlot-Cabernet Sauvignon', 'Nascetta', 'Mtsvane', 'Incrocio Manzoni', 'Moscatel Graúdo', 'Garnacha Blend', 'Cabernet Pfeffer', 'Feteasca', 'Shiraz-Cabernet Sauvignon', 'Meritage', 'Cabernet Sauvignon-Malbec', 'Tempranillo', 'Frappato', 'Vermentino', 'Muscat Canelli', 'Uva di Troia', 'Trincadeira', 'Pallagrello Bianco', 'Jacquère', 'Gamay Noir', 'Garganega', 'Cabernet Sauvignon-Merlot', 'Ribolla Gialla', 'Tinta Miúda', 'Trebbiano Spoletino', 'Marawi', 'Ciliegiolo', 'Abouriou', 'Listán Negro', 'Trebbiano di Lugana', 'Verdicchio', 'Aragonez', 'Pallagrello', 'Negrette', 'Posip', 'Mauzac', 'Mantonico', 'Pinot Blanc', 'Viognier', 'Albariño', 'Tinta Barroca', 'Mazuelo', 'Códega do Larinho', 'Elbling', 'Robola', 'Barbera', 'Merlot-Cabernet', 'Avesso', 'Trepat', 'Pallagrello Nero', 'Zinfandel', 'Plavac Mali', 'Nero di Troia', 'Friulano', 'Chardonnay-Sauvignon Blanc', 'Moschofilero', 'Mavrud', 'Antão Vaz', 'Tannat-Merlot', 'Tinta Fina', 'Traminer', 'Assyrtico', 'Fumé Blanc', 'Touriga Nacional-Cabernet Sauvignon'], ['Cannonau', 'Provence white blend', 'Malbec', 'Feteasca Neagra', 'Pugnitello', 'Picpoul', 'Ojaleshi', 'Perricone', 'Syrah-Merlot', 'Carcajolu', 'Cercial', 'Teran', 'Viognier-Roussanne', 'Tempranillo-Tannat', 'Piquepoul Blanc', 'Carmenère-Cabernet Sauvignon', 'Grenache-Shiraz', 'Zelen', 'Kadarka', 'Thrapsathiri', 'Savatiano', 'Rebula', 'Vespolina', 'Rebo', 'Tamjanika', 'Muscat Hamburg', 'Maria Gomes', 'Erbaluce', 'Pinot Noir-Syrah', 'Žilavka', 'Gouveio', 'Frankovka', 'Albarossa', 'Cabernet Sauvignon Grenache', 'Uvalino', 'Vermentino Nero', 'Muscadel', 'Monastrell-Petit Verdot', 'Blatina', 'Malvasia Fina', 'Roditis-Moschofilero', 'Jampal', 'Petit Courbu', 'Roscetto', 'Chenin Blanc-Chardonnay', 'Sparkling Blend', 'Gamay', 'Marsanne', 'Tinta Roriz', 'Tocai', 'Moscatel', 'Sousão', "Cesanese d'Affile", 'Malbec-Cabernet Franc', 'Sylvaner', 'Fiano', 'Cabernet Franc', 'Tinto del Pais', 'Gros Manseng', 'Saperavi', 'Schiava', 'White Riesling', 'Sauvignon', 'Charbono', 'Prié Blanc', 'Grenache-Mourvèdre', 'Sangiovese-Syrah', 'Petite Sirah', 'Malbec-Cabernet Sauvignon', 'Tempranillo Blend'], ['Claret', 'Tannat-Syrah', 'Sauvignon Blanc-Chardonnay', 'Orange Muscat', 'Petit Verdot', 'Zweigelt', 'Corvina, Rondinella, Molinara', 'Sauvignon Blanc-Semillon', 'Sémillon', 'Edelzwicker', 'Argaman', 'Nuragus', 'Mission', 'Rolle', 'Viognier-Marsanne', 'Chardonnay', 'Greco', 'Malbec-Merlot', 'Red Blend', 'Roussanne-Marsanne', 'Viosinho', 'Duras', 'Pinot Bianco', 'Melon', 'Rhône-style White Blend', 'Malagousia', 'Pinot Gris', 'Semillon-Sauvignon Blanc', 'Cabernet Sauvignon-Shiraz', 'Sangiovese Cabernet', 'Sciaccerellu', 'Malagouzia', 'Merlot-Syrah', 'Babić', 'Mandilaria', 'Lagrein', 'Roussanne', 'Sangiovese', 'Malvasia Bianca', 'Cabernet Blend', 'Touriga Franca', 'Chenin Blanc', 'Corvina', 'Gewürztraminer', 'Black Muscat', 'Moscatel Roxo', 'Cabernet Sauvignon', 'Graciano', 'Grenache-Syrah', 'Merlot-Cabernet Franc', 'Silvaner', 'Melnik', 'Lambrusco Salamino', 'Shiraz-Mourvèdre', 'Pinot Nero', 'Mencía', 'Bordeaux-style White Blend', 'Muscat', 'Muscat Blanc', 'Teroldego', 'Cabernet Franc-Merlot', 'Gelber Muskateller', 'Altesse', 'Marselan', 'Chardonnay-Pinot Blanc', 'Grenache Blanc', 'Auxerrois', 'Grauburgunder', 'Tannat', 'Alvarinho'], ['Prugnolo Gentile', 'Portuguese Red', 'Carignane', 'Kerner', 'Counoise', 'Pinot Meunier', 'Touriga Nacional', 'Aragonês', 'Carignan', 'Turbiana', 'G-S-M', 'Syrah-Grenache', 'Cabernet Sauvignon-Cabernet Franc', 'Grenache', 'Cabernet Sauvignon-Syrah', 'Cinsault', 'Cabernet Sauvignon-Sangiovese', 'Welschriesling', 'Magliocco', 'White Port', 'Grenache Noir', 'Karasakiz', 'Clairette', 'Chasselas', 'Susumaniello', 'Debit', 'Verduzzo Friulano', 'Mataro', 'Cabernet Sauvignon-Merlot-Shiraz', 'Graševina', 'Syrah-Malbec', 'Nielluciu', 'Petit Meslier', 'Vidadillo', 'Other', 'Tamianka', 'Canaiolo', 'Rabigato', 'Biancu Gentile', 'Xinisteri', 'Ryzlink Rýnský', 'Tămâioasă Românească', 'Merlot-Grenache', 'Hárslevelü', 'Apple', 'Tokay Pinot Gris', 'Tinta Madeira', 'Catalanesca', 'Gamza', 'Palomino', 'Assyrtiko', 'Cabernet Franc-Cabernet Sauvignon', 'Alfrocheiro', 'Cabernet-Syrah', 'Bordeaux-style Red Blend', 'Shiraz', 'Carignan-Grenache', 'Petite Verdot', 'Touriga Nacional Blend', 'Rhône-style Red Blend', 'Kalecik Karasi', 'Syrah-Tempranillo', 'Syrah', 'Mourvèdre', 'Aglianico', 'Shiraz-Viognier', 'Touriga', 'Vidal', 'Rotgipfler'], ['Roussanne-Viognier', 'Pinot Noir', 'Syrah-Viognier', 'Okuzgozu', 'Riesling', 'Albana', 'Syrah-Cabernet Sauvignon', 'Weissburgunder', 'Pinot-Chardonnay', 'Cariñena-Garnacha', 'Rara Neagra', 'Merlot-Tannat', 'Syrah-Mourvèdre', 'Morillon', 'Pedro Ximénez', 'Encruzado', 'Zibibbo', 'Tinta de Toro', 'Gros and Petit Manseng', 'Petit Manseng', 'Rieslaner', 'Cerceal', 'Muskat', 'Viognier-Grenache Blanc', 'St. Laurent', 'Champagne Blend', 'Lambrusco di Sorbara', 'Port', 'Nerello Cappuccio', 'Boğazkere', 'Furmint', 'Savagnin', 'Alicante Bouschet', 'Nerello Mascalese', 'Sagrantino', 'Tinto Fino', 'Carignano', 'Jaen', 'Austrian Red Blend', 'Shiraz-Grenache', 'Syrah-Petite Sirah', 'Grenache-Carignan', 'Spätburgunder', 'Sherry', 'Provence red blend', 'Carricante', 'Grüner Veltliner', 'Verduzzo', 'Früburgunder', 'Cesanese', 'Syrah-Petit Verdot', 'Trousseau Gris', 'Veltliner', 'Tinta Cao', 'Pinot Gris-Gewürztraminer', 'Shiraz-Malbec', 'Orangetraube', 'Mavrotragano', 'Shiraz-Roussanne', 'Schwartzriesling', 'Sauvignonasse', 'Syrah-Carignan', 'Moscato Rosa', 'Vital', 'Favorita', 'Otskhanuri Sapere', 'Tsapournakos', 'Ondenc', 'Nasco'], ['Roter Veltliner', 'Blaufränkisch', 'Austrian white blend', 'Baga', 'Durella', 'Zierfandler', 'Syrah-Cabernet Franc', 'Roter Traminer', 'Tannat-Cabernet', 'Zierfandler-Rotgipfler', 'Timorasso', 'Nebbiolo', 'Vitovska', 'Bovale', 'Malbec Blend', 'Trollinger', 'Sangiovese Grosso', 'Scheurebe', 'Alsace white blend', 'Tokay', 'Ugni Blanc', 'Romorantin', 'Malbec-Tannat', 'Tokaji', 'Neuburger', 'Braucol', 'Moscadello', "Loin de l'Oeil", 'Tannat-Cabernet Franc', 'Torbato', 'Mourvèdre-Syrah', 'Prunelard', 'Baga-Touriga Nacional', 'Cabernet Franc-Malbec', 'Tempranillo-Malbec', 'Bacchus', 'Malbec-Carménère', 'Tinta Francisca', 'Carignan-Syrah', 'Cabernet-Shiraz', 'Malbec-Petit Verdot', 'Black Monukka', 'Francisa', 'Sämling', 'Picolit', 'Chardonnay-Albariño', 'Malbec-Cabernet', 'Vespaiolo', 'Chardonnay Weissburgunder', 'Sauvignon Blanc-Assyrtiko', 'Centesimino', 'Babosa Negro', 'Kotsifali', 'Moscato di Noto', 'Torontel', 'Pignolo', 'Caprettone', 'Rosenmuskateller', 'Roviello', 'Muscadelle', 'Blauburgunder', 'Ramisco', 'Garnacha-Cariñena', 'Sirica', 'Riesling-Chardonnay', 'Sercial', 'Bual', 'Tinta del Pais', 'Terrantez']]
winery_id_group = [[117027, 120274, 113922, 19564, 100482, 16847, 77028, 116091, 14343, 90905, 19568, 100490, 90934, 30108, 118055, 113925, 114968, 79714, 42910, 117316, 82748, 90931, 90928, 121881, 128261, 97184, 116140, 113924, 125828, 109331, 99999, 79713, 21839, 82745, 33839, 86714, 7333, 125827, 79097, 116139, 60875, 128263, 83846, 14342, 56186, 60877, 90246, 23433, 26889, 118051, 112364, 33380, 116086, 21843, 35514, 109631, 108639, 48889, 39942, 122554, 14738, 91697, 75555, 98687, 97182, 42913, 82743, 121868, 107767, 81386, 98366, 21782, 33720, 91696, 39959, 33726, 31691, 118052, 49633, 92074, 5903, 45569, 113915, 111747, 113918, 33716, 19122, 3635, 117023, 47426, 97171, 90901, 99889, 11370, 39954, 92431, 44990, 82744, 23435, 102863, 33383, 33382, 100471, 100474, 43990, 81388, 109321, 13019, 82740, 33727, 5905, 109633, 117132, 21786, 13020, 66327, 102604, 2623, 14732, 86716, 111457, 9372, 4546, 39285, 43997, 3632, 15607, 95261, 26930, 16846, 7496, 74885, 43992, 61422, 109327, 98675, 75937, 9411, 12066, 106966, 34532, 23936, 102589, 27581, 97175, 28462, 34975, 6493, 11076, 53983, 3623, 70339, 96621, 26934, 105204, 28458, 96622, 15610, 100483, 102754, 92055, 90893, 48356, 39943, 90894, 121929, 116696, 5541, 14727, 9427, 39499, 30894, 39765, 21817, 116133, 39510, 113343, 40765, 109622, 16843, 6510, 49621, 42787, 339, 16844, 45778, 343, 26918, 23912, 97176, 6497, 16838, 127174, 5897, 109626, 112362, 3945, 92064, 13003, 12069, 7078, 79695, 99896, 47211, 83456, 11352, 127163, 107777, 101451, 104167, 40772, 60863, 102609, 91685, 91686, 29577, 52079, 117012, 92059, 61366, 49389, 50865, 122932, 56028, 7613, 45772, 13780, 56184, 92054, 121928, 92052, 89719, 21778, 63799, 106616, 27227, 40943, 113342, 13009, 97123, 23923, 12074, 124044, 102857, 28581, 5530, 98364, 16845, 81382, 94601, 61427, 23908, 78863, 28455, 56949, 5527, 68767, 49397, 7610, 33831, 91684, 123447, 40767, 7080, 53286, 99890, 50310, 6500, 21822, 56030, 92060, 122930, 103324, 35242, 50866, 39948, 108630, 38621, 36810, 5544, 30639, 24535, 4550, 109625, 39497, 49629, 16556, 34324, 2624, 4535, 30878, 5801, 35302, 49394, 57179, 95875, 63800, 48892, 90900, 102854, 118383, 119177, 71631, 106946, 18344, 19125, 82747, 110949, 36526, 57320, 45781, 6406, 4544, 77854, 28632, 79711, 6294, 116090, 53989, 52074, 19531, 7094, 42906, 98679, 4759, 2619, 3932, 40770, 21832, 18339, 1854, 40763, 39952, 92601, 83836, 3642, 40887, 96411, 61590, 11086, 9429, 11369, 49630, 21733, 14720, 21825, 27224, 23438, 49628, 3641, 22237, 50859, 5961, 34789, 127605, 121759, 80131, 105200, 57316, 33836, 71874, 114292, 79971, 28598, 22762, 80266, 108592, 74341, 56507, 96611, 33984, 71877, 61408, 50590, 77710, 97111, 123566, 112936, 45768, 86872, 102753, 23429, 2621, 105919, 18335, 42112, 48880, 54095, 12670, 42169, 96183, 22761, 1137, 119164, 6485, 77708, 63344, 106964, 61355, 13091, 51078, 35067, 11604, 116911, 119477, 71265, 91590, 46388, 93967, 19523, 76502, 42768, 38716, 69821, 71622, 16567, 11592, 60779, 7600, 101439, 82533, 3246, 74340, 79918, 48978, 16874, 2002, 3409, 21040, 21829, 1128, 43561, 106753, 62608, 118140, 17155, 95873, 76698, 57186, 57266, 119463, 5783, 29744, 74347, 96188, 71876, 101286, 68777, 43394, 20151, 21827, 51406, 99932, 31247, 13775, 37691, 42371, 54110, 78090, 58691, 118689, 121980, 64952, 51090, 42904, 54086, 5804, 108260, 111435, 53269, 57172, 69815, 80386, 45764, 18610, 116910, 127595, 60787, 28912, 91078, 40894, 101444, 57319, 95247, 13761, 74332, 19808, 80268, 106942, 102747, 28915, 106756, 102744, 91807, 51479, 119173, 56495, 4540, 101011, 56941, 121978, 64652, 9158, 109529, 124098, 116204, 21030, 5540, 5785, 64677, 100465, 1855, 6513, 1152, 80275, 106750, 60785, 48355, 96618, 61409, 11367, 94267, 101432, 69818, 127960, 74351, 6514, 106610, 94276, 80264, 11612, 68816, 48365, 101437, 43388, 6293, 11603, 2421, 20143, 118687, 61981, 69819, 42131, 64938, 79980, 53262, 77718, 114456, 83895, 53267, 6533, 57318, 88095, 83921, 17164, 118386, 82614, 101441, 69589, 11586, 78699, 58696, 82543, 51515, 28590, 63355, 18622, 48878, 43397, 93969, 5146, 29762, 67810, 104150, 119461, 35491, 5790, 90609, 123567, 119166, 124032, 1845, 19061, 42176, 119170, 121976, 107073, 57315, 5822, 21044, 74803, 49373, 1847, 51478, 104162, 93971, 1538, 21048, 64669, 124033, 56185, 36506, 82616, 56508, 3952, 9417, 119473, 23917, 42171, 86269, 20146, 42113, 127105, 106941, 127591, 42133, 45767, 121858, 35510, 31681, 50854, 13622, 112934, 45758, 80261, 16543, 121861, 60783, 12668, 76696, 124591, 119175, 338, 52066, 30454, 70352, 127958, 121985, 77717, 59508, 13767, 57175, 35253, 9422, 57322, 121149, 106953, 5904, 4529, 3400, 81427, 14730, 11087, 15350, 1817, 11607, 55540, 2013, 13151, 29566, 52736, 55456, 11361, 12077, 19488, 43611, 31852, 97907, 68775, 28995, 20767, 45, 3615, 16865, 28569, 32782, 35267, 2625, 99271, 13011, 31686, 31678, 82629, 11771, 47204, 43396, 3630, 17681, 78884, 25889, 69348, 28557, 43392, 7669, 10675, 101289, 62750, 2774, 27947, 10587, 6382, 40766, 92310, 15983, 70764, 121143, 45760, 63347, 20761, 16842, 74881, 67794, 404, 2141, 9141, 97837, 45847, 57321, 2286, 14736, 63739, 38972, 35495, 38973, 74326, 4941, 15102, 6509, 7596, 37815, 27940, 48867, 88924, 102740, 53260, 70353, 35239, 18211, 49134, 53272, 102755, 45854, 19009, 95240, 23415, 69809, 88052, 86682, 43536, 39983, 4541, 38632, 24246, 111449, 48965, 7909, 30892, 17786, 45770, 50845, 5787, 3355, 11171, 55771, 38974, 5893, 30225, 34329, 36382, 10440, 36725, 70750, 55031, 37058, 1837, 103004, 19803, 6306, 63798, 90614, 76190, 112213, 112371, 79852, 83504, 46217, 5906, 26881, 3943, 4944, 34960, 47207, 2970, 1816, 5529, 5809, 2762, 2786, 1815, 4549, 14075, 1283, 1705, 7045, 16520, 58695, 23934, 5955, 21265, 13604, 20009, 11608, 5538, 4559, 68875, 89798, 37681, 46623, 42903, 13005, 13232, 7899, 21783, 33149, 32530, 31528, 89809, 6511, 2940, 14258, 3673, 1809, 4543, 50849, 9570, 4547, 12461, 39997, 37329, 78712, 6523, 3356, 7982, 24853, 10209, 13218, 18326, 3738, 12459, 19281, 15953, 29976, 12566, 3941, 60, 9195, 2941, 11366, 56506, 18205, 3951, 3398, 9137, 13227, 2958, 1856, 3670, 112644, 61548, 2933, 13558, 118905, 99277, 60496, 32774, 57308, 25666, 39935, 79133, 96599, 35285, 10697, 26916, 55697, 5950, 59161, 86299, 80380, 10666, 11860, 74635, 55684, 2155, 69492, 128409, 69551, 106600, 92278, 36712, 46615, 117954, 28529, 75166, 96585, 31027, 61973, 28838, 69573, 64803, 2143, 86858, 11535, 81099, 44975, 35288, 41318, 75295, 47248, 88844, 409, 34319, 7910, 78687, 93671, 74633, 99924, 18987, 42960, 37685, 25715, 30660, 12180, 17161, 44846, 10669, 104125, 95883, 57177, 6298, 54596, 122613, 39929, 60512, 40210, 107275, 106094, 7904, 64290, 93674, 13823, 43610, 72423, 72421, 9135, 35298, 121803, 121720, 44863, 12977, 120181, 2780, 117946, 7148, 11851, 32523, 65526, 63750, 75143, 78079, 35279, 116817, 1779, 51407, 92289, 15085, 63337, 36543, 95596, 121727, 88917, 5959, 7163, 42684, 2794, 15084, 1350, 39763, 96600, 34336, 33994, 93676, 2618, 94907, 15394, 124573, 96597, 33326, 44094, 2140, 57270, 81754, 17691, 26038, 119196, 44865, 9428, 118895, 65369, 60492, 35278, 12054, 89681, 43621, 28986, 26062, 119210, 96595, 41213, 120167, 1754, 105457, 10528, 44106, 110756, 1780, 13229, 3631, 38703, 68117, 11849, 64293, 99928, 53830, 97274, 55459, 32776, 16089, 33327, 102058, 11609, 95892, 104126, 40928, 59056, 39926, 31264, 126134, 68129, 129184, 10689, 9214, 42554, 52725, 105804, 43549, 21710, 43605, 65850, 125999, 9364, 61013, 43564, 101998, 13567, 108923, 59178, 122612, 61689, 45204, 58728, 63091, 58889, 2782, 92068, 116934, 124572, 63956, 92050, 66314, 6505, 88534, 97818, 19647, 121730, 1838, 19520, 1758, 42100, 1721, 14399, 13888, 39868, 27760, 69802, 59276, 74629, 33124, 1133, 50135, 3118, 34089, 33323, 102850, 97815, 16866, 47928, 728, 28559, 23546, 110759, 10696, 30638, 92295, 11353, 90802, 110990, 122395, 86601, 121807, 74639, 70737, 88845, 28732, 34320, 80249, 10668, 20772, 39874, 28747, 25670, 68388, 1134, 25872, 50163, 81743, 31241, 44092, 86853, 71280, 119192, 59054, 44980, 58885, 20109, 34342, 74693, 47267, 7590, 36719, 12196, 5646, 95231, 96501, 14494, 105461, 126949, 60491, 110239, 5814, 29579, 12690, 100454, 69283, 3681, 39764, 6423, 112718, 97296, 13833, 59666, 50858, 21749, 17210, 88047, 86291, 50118, 10678, 77705, 17698, 95884, 250, 3123, 46511, 115561, 11772, 2154, 121840, 39917, 69565, 2819, 13559, 80371, 122398, 13245, 64648, 19646, 43311, 79578, 111190, 112726, 31529, 129193, 11856, 60373, 17159, 55700, 88212, 1998, 88916, 129187, 66042, 1774, 10205, 51458, 32516, 7157, 120292, 79850, 13569, 6303, 16834, 96606, 21020, 94922, 108905, 6297, 10671, 86300, 42562, 14483, 74876, 34332, 13625, 62859, 5394, 4287, 15598, 31134, 111192, 11594, 70487, 48883, 32783, 76506, 120165, 30899, 43995, 44856, 112719, 2797, 17699, 1279, 28988, 95891, 60168, 14057, 70489, 10709, 43547, 39913, 112646, 33340, 86864, 69899, 10573, 43526, 126263, 41451, 22229, 126668, 110758, 74683, 60509, 95211, 88841, 54608, 32537, 68138, 16976, 110247, 11779, 33319, 44095, 55317, 61033, 99270, 121710, 63733, 27218, 69798, 92306, 110982, 93670, 6524, 61684, 37687, 82537, 11784, 13828, 47203, 89666, 121837, 35280, 116938, 10661, 2937, 15097, 16991, 121120, 5820, 60169, 12671, 29554, 2012, 88483, 61564, 74364, 65381, 111197, 42136, 118377, 13560, 95229, 7612, 33018, 17400, 92300, 95233, 48693, 75533, 39770, 110107, 23549, 18353, 2135, 11597, 10700, 6566, 63102, 39970, 48626, 49238, 44113, 13887, 45850, 2804, 38395, 35249, 6292, 112720, 32518, 32534, 76692, 129627, 103835, 38392, 64637, 112715, 26036, 3416, 19005, 66324, 38405, 16984, 66326, 42779, 45849, 96447, 39884, 97934, 110753, 42568, 2929, 64532, 11168, 118907, 48547, 55782, 92845, 2755, 74701, 16553, 102168, 78073, 80387, 7900, 7059, 28663, 19516, 50202, 37676, 34321, 119207, 116207, 84368, 35284, 108914, 88045, 16077, 10703, 9139, 40669, 28752, 35287, 111185, 129199, 61016, 3672, 40874, 94257, 76325, 48546, 105437, 102843, 88846, 44101, 65435, 97286, 43088, 104136, 11768, 6309, 16080, 76501, 88843, 7895, 120345, 105441, 7605, 48700, 50174, 55127, 126085, 26888, 97821, 53844, 28563, 17811, 40877, 76189, 22066, 7047, 50165, 63338, 37714, 49245, 29541, 99937, 10702, 13832, 75289, 87949, 33338, 7318, 45754, 119208, 86287, 73287, 7055, 102173, 105817, 88219, 43558, 34327, 31917, 15089, 99939, 95581, 4249, 11598, 379, 2776, 11172, 4557, 35064, 10566, 48621, 31696, 50986, 6518, 9276, 7327, 19283, 5076, 29715, 25891, 58394, 13012, 19040, 23884, 3124, 6288, 7974, 29578, 3352, 6501, 3749, 6554, 2736, 92294, 11083, 34203, 16905, 60356, 69417, 6421, 22389, 4265, 29624, 20310, 44841, 2299, 38387, 46651, 33151, 7696, 20758, 26922, 12761, 12980, 1276, 2435, 233, 3166, 1644, 3347, 10558, 6548, 30675, 7725, 5784, 340, 42384, 50592, 28427, 8432, 22744, 1828, 9408, 34157, 59150, 35061, 63056, 31693, 5786, 71889, 89718, 69, 40676, 43109, 35768, 111442, 28596, 63790, 15094, 4354, 59164, 21688, 39873, 18825, 17163, 73774, 20249, 3109, 27891, 66325, 3338, 16095, 47091, 13873, 35259, 1834, 14430, 11645, 7603, 2934, 3418, 40978, 24320, 52269, 34962, 28562, 17906, 13233, 5838, 6530, 5949, 23420, 6553, 1051, 14641, 12368, 10687, 1263, 1204, 4824, 11358, 33021, 22739, 63934, 91680, 42298, 14474, 17543, 65374, 8433, 22240, 27147, 5168, 63527, 13248, 77900, 3879, 10699, 12190, 63090, 38700, 10559, 30875, 195, 17404, 24317, 64530, 14471, 21031, 32771, 63508, 28446, 42377, 47104, 11620], [66537, 92276, 28599, 55059, 75759, 69701, 28655, 18632, 26920, 12209, 83910, 4825, 23427, 6492, 71146, 20150, 39881, 82102, 3309, 86672, 2436, 9365, 5490, 44989, 10660, 36554, 27124, 91050, 90219, 52724, 68392, 24076, 20989, 13177, 3954, 1058, 50124, 17165, 79899, 25735, 6314, 20262, 1806, 17398, 34526, 43539, 83510, 53588, 2751, 5796, 72415, 108569, 54038, 20456, 16084, 1848, 43628, 18603, 14740, 42378, 56042, 10442, 10457, 120313, 15815, 20995, 97058, 27086, 10447, 51520, 71040, 26020, 1988, 35311, 66502, 25095, 3135, 20153, 92312, 21845, 1281, 7155, 36794, 66037, 1551, 94921, 14286, 344, 30321, 59166, 78086, 46645, 4555, 2281, 29517, 37388, 2163, 15817, 1756, 20580, 1012, 4829, 1273, 7903, 73586, 43607, 16876, 14067, 22228, 14498, 27376, 29732, 12469, 77123, 23518, 25657, 13763, 10577, 3627, 9415, 26052, 45835, 20472, 2923, 25122, 3241, 17245, 11952, 36758, 34967, 1556, 32777, 17153, 828, 44109, 3419, 1260, 7062, 61039, 31531, 3117, 24736, 39028, 54699, 6287, 70026, 49393, 28444, 78920, 7718, 17793, 2759, 16465, 73288, 47089, 99279, 57260, 67884, 80370, 61261, 25720, 42777, 18986, 2706, 29622, 31643, 55902, 36860, 21319, 11777, 46386, 23539, 53835, 36483, 87360, 1138, 1846, 47022, 23916, 414, 2945, 24691, 17696, 6787, 37899, 7602, 842, 982, 1833, 1835, 42050, 1546, 9278, 11167, 5803, 26056, 35238, 31030, 17823, 42516, 13618, 49241, 22748, 26090, 30834, 12052, 96187, 5794, 24732, 50188, 2760, 48855, 51797, 10356, 43079, 23837, 90929, 15236, 30655, 27369, 18826, 36741, 35068, 11365, 9197, 18198, 21072, 2767, 1372, 5174, 16892, 39990, 2336, 41579, 40777, 22597, 6528, 5523, 2437, 11759, 14936, 41218, 9864, 27755, 40706, 6969, 1536, 55522, 21756, 3390, 13219, 26826, 2930, 13162, 12061, 1274, 1842, 991, 3415, 8615, 9155, 1829, 3936, 19359, 14381, 1812, 9406, 5952, 9200, 22832, 53, 1043, 2324, 96404, 16864, 39023, 13555, 81431, 23444, 126688, 41092, 87103, 34033, 106562, 66983, 37062, 4819, 77865, 20679, 59449, 23275, 122635, 21315, 62839, 92021, 1055, 34806, 253, 109678, 58822, 57332, 119563, 92026, 100865, 91745, 27080, 86196, 32772, 118552, 64496, 86560, 30704, 7833, 18201, 125025, 2744, 97916, 6541, 49926, 84073, 3134, 122386, 25377, 60680, 66981, 109346, 75424, 121531, 87574, 92018, 127505, 108781, 23425, 33006, 16889, 123258, 100091, 7814, 13830, 86559, 40199, 94690, 50014, 16673, 36750, 51392, 79037, 12755, 75604, 41008, 63017, 42143, 725, 7983, 24680, 4467, 44962, 28555, 1760, 47346, 125805, 88463, 16870, 41872, 22972, 1541, 94325, 12093, 114775, 19002, 60734, 37166, 75059, 50920, 72414, 51517, 12992, 67871, 77870, 87915, 15389, 35074, 5898, 43845, 31128, 6301, 75755, 41885, 24682, 50974, 43258, 47244, 20129, 67274, 34297, 2343, 59221, 12376, 13403, 45167, 71896, 78089, 108774, 59209, 45530, 1663, 44835, 37767, 93986, 95872, 7664, 25593, 114149, 1545, 32767, 20489, 40731, 71536, 24733, 62843, 40929, 58873, 81094, 106081, 87104, 7447, 11948, 19918, 98734, 89799, 91301, 37164, 107542, 75426, 81009, 43266, 43832, 109671, 30754, 128128, 121841, 9355, 19684, 20976, 40733, 97909, 63596, 39521, 89805, 94586, 45413, 58147, 40461, 6656, 16677, 40236, 71891, 29068, 22205, 100855, 80309, 33229, 30228, 43272, 29443, 55133, 63039, 25583, 61483, 40886, 75777, 100107, 20148, 1811, 93982, 17409, 1489, 48317, 20134, 44923, 39982, 24115, 71163, 1624, 70772, 127046, 30664, 6603, 86563, 128116, 11184, 62235, 86215, 90267, 62613, 112210, 61484, 88800, 114502, 116467, 93987, 66189, 40451, 41084, 64213, 41869, 71263, 45514, 10819, 10808, 80982, 43257, 25530, 4579, 107248, 34912, 18693, 7644, 86212, 18708, 62025, 48872, 15816, 72489, 59497, 13350, 30750, 25061, 127497, 31110, 84846, 69901, 86674, 62679, 41866, 18, 94958, 109359, 43108, 27690, 29190, 11524, 1919, 61515, 87118, 65422, 82911, 11850, 32526, 11951, 13349, 31116, 125036, 732, 9571, 55457, 110782, 821, 16671, 95874, 1810, 97903, 11868, 46952, 20436, 97696, 93998, 37053, 61489, 59180, 20463, 23268, 96688, 94955, 19450, 114139, 44956, 28229, 70639, 13845, 64979, 50822, 92020, 86580, 11634, 14539, 27398, 91178, 64984, 54835, 81620, 38494, 65156, 93914, 51508, 91161, 12614, 75431, 12762, 54042, 33749, 93905, 28208, 117389, 1531, 110238, 96833, 7577, 23458, 15406, 88553, 7889, 108771, 102199, 94570, 232, 12766, 27372, 80976, 86689, 35322, 108971, 96203, 1657, 64505, 24647, 12501, 92036, 3934, 34317, 70767, 37050, 106586, 19670, 104596, 71560, 64510, 94577, 72402, 109369, 83533, 114499, 5402, 23545, 77866, 41516, 7705, 985, 81735, 62690, 88546, 75196, 69187, 18600, 7583, 72397, 38479, 70049, 12178, 98001, 80224, 40726, 65709, 128809, 14530, 7584, 10191, 18698, 80979, 31848, 39941, 99490, 114755, 42109, 83686, 24249, 94189, 17852, 13498, 18830, 64965, 109365, 58553, 54075, 51214, 49931, 88163, 68406, 122399, 106568, 71159, 964, 31118, 28649, 19906, 98738, 61924, 40234, 12114, 51400, 113372, 21700, 100595, 46633, 20985, 79345, 14397, 32604, 46636, 51500, 1724, 17799, 3406, 78938, 87975, 120121, 64494, 17059, 87914, 75081, 7486, 82099, 106223, 116264, 19448, 24114, 6413, 87924, 42705, 81167, 29979, 112284, 13943, 37211, 33003, 34190, 48314, 46396, 17908, 55047, 80975, 113311, 1715, 30335, 29994, 127060, 66550, 50832, 118554, 97369, 26019, 36748, 128890, 2704, 30315, 87934, 30736, 70779, 61496, 5403, 2348, 124838, 39014, 39771, 1488, 35317, 24122, 54093, 55124, 109564, 94691, 43577, 39013, 67682, 73952, 35320, 1989, 20576, 61691, 96485, 88539, 45149, 25537, 90406, 9359, 119011, 47880, 7089, 22218, 42569, 17912, 16666, 5466, 33004, 22017, 51132, 88902, 22220, 111875, 93201, 743, 32638, 25671, 115919, 45882, 12682, 50404, 117641, 4283, 58268, 114757, 111188, 32755, 104732, 1658, 32219, 91675, 34788, 116249, 31127, 17788, 55116, 6305, 58153, 64604, 37697, 34180, 1851, 15222, 100980, 31081, 91875, 74769, 55770, 13882, 127054, 93910, 54837, 18701, 51408, 41208, 41474, 57331, 7666, 20975, 18209, 1804, 11995, 20999, 16463, 106080, 78082, 11929, 48862, 59498, 50867, 106815, 42053, 27937, 126653, 73199, 125035, 94416, 1049, 45162, 100594, 20969, 97820, 59216, 39018, 13835, 76707, 104614, 108783, 28832, 70777, 118892, 97695, 88403, 52597, 75532, 123919, 18591, 4790, 16911, 36458, 49938, 39025, 106595, 19913, 94580, 33137, 52272, 9642, 8423, 116003, 17064, 48301, 63012, 30743, 2257, 114765, 1667, 5962, 4281, 4561, 66982, 16994, 87232, 80980, 29062, 28669, 12673, 108772, 15967, 42518, 98940, 88196, 40453, 43093, 108782, 89821, 110768, 87555, 92839, 86567, 96539, 66978, 36708, 102998, 6614, 28505, 68626, 44968, 75791, 112631, 7807, 39007, 47333, 89812, 46962, 79357, 116013, 63974, 59435, 25080, 3993, 78980, 75782, 51403, 71561, 114766, 4828, 20977, 47098, 70055, 23851, 91998, 37143, 109361, 34196, 13871, 116005, 128821, 88788, 29935, 2267, 69972, 89804, 17056, 125215, 39353, 33230, 90404, 5954, 67685, 12078, 108964, 114521, 113370, 33231, 243, 7156, 88734, 73364, 40994, 10820, 14512, 69294, 81446, 61592, 108777, 32629, 41878, 44924, 2975, 21707, 51137, 58555, 30316, 84074, 100587, 111306, 88795, 50778, 24077, 61808, 61817, 11947, 61972, 19495, 57180, 94969, 28744, 44191, 39357, 20464, 64976, 5907, 29981, 9233, 3172, 15829, 88901, 12750, 20469, 94700, 94574, 36467, 87476, 78942, 29455, 123912, 5336, 17851, 107250, 2971, 54839, 42179, 62848, 118936, 75817, 20452, 40674, 75444, 49181, 96234, 108776, 16915, 18586, 52595, 106227, 1755, 24057, 17690, 17863, 51820, 37803, 24314, 40723, 41876, 78933, 3665, 44190, 128606, 17862, 75156, 17072, 61577, 40444, 4270, 2323, 127063, 33155, 23267, 95879, 37061, 13563, 86195, 15332, 91236, 57314, 109351, 104601, 69306, 6202, 103011, 63034, 94311, 63989, 69542, 46961, 1643, 46392, 16249, 2908, 3996, 15345, 5112, 1777, 12757, 4294, 11848, 2775, 12063, 10584, 12559, 4950, 8805, 3981, 15964, 2757, 53915, 6308, 11983, 6381, 915, 971, 742, 14338, 55, 2338, 9208, 28065, 4553, 52300, 79348, 39286, 31706, 10662, 22182, 18398, 26326, 19211, 40212, 2156, 5795, 9018, 30837, 1163, 19482, 1661, 2006, 2292, 20943, 3395, 20868, 16338, 2278, 20903, 10591, 37900, 6487, 50987, 3667, 4935, 1702, 13556, 6515, 41147, 55308, 27101, 59, 29929, 2770, 55095, 20604, 36795, 7459, 84082, 9291, 63328, 39264, 11079, 54645, 3779, 24693, 19287, 72, 3877, 17812, 4801, 5096, 60298, 1826, 3609, 21681, 21351, 19656, 26486, 3269, 16462, 14482, 10045, 1633, 2414, 35286, 17200, 30467, 11166, 1638, 959, 10674, 9633, 1277, 2902, 16555, 2627, 12174, 5404, 580, 4446, 11477, 1135, 3359, 10107, 6419, 33047, 26304, 7958, 23423, 3114, 3713, 7967, 16614, 25673, 14426, 11360, 36808, 15161, 88714, 63533, 11533, 5739, 20137, 69566, 3340, 604, 63, 49258, 30064, 16549, 42043, 10207, 15397, 69681, 9361, 14937, 19292, 89591, 33304, 43077, 1995, 18557, 41472, 1555, 74101, 2287, 6567, 13758, 4832, 246, 38389, 7702, 22693, 3935, 2341, 24392, 1046, 12478, 27934, 2822, 3783, 30942, 1712, 1165, 67861, 60420, 40393, 18374, 1704, 92048, 10665, 45529, 71556, 11386, 55099, 87094, 263, 17936, 81450, 49092, 7673, 1059, 7449, 10698, 15351, 2337, 11661, 27708, 43386, 10695, 29764, 19410, 4266, 15005, 46612, 30762, 28004, 31237, 26576, 8728, 21847, 14538, 15984, 18227, 7352, 6210, 27106, 40455, 2790, 801, 11993, 9409, 5603, 4209, 28659, 13950, 19461, 2801, 38391, 96148, 43548, 1723, 1275, 15824, 42295, 36511, 31685, 11380, 12631, 29617, 5528, 60430, 28547, 15243, 5611, 2737, 9978, 49909, 19688, 20475, 5479, 21056, 1373, 1353, 18703, 9854, 9420, 24031, 12451, 1347, 5908, 1060, 8269, 8414, 9143, 17382, 3333, 1480, 1645, 2534, 19669, 29560, 58287, 59647, 47000, 92827, 14216, 28002, 57465, 5079, 731, 96412, 64296, 34761, 22473, 9288, 2356, 17891, 63756, 17864, 57421, 25282, 50200, 24982, 28337, 60376, 64668, 21803, 10585, 56938, 61797, 46235, 51409, 64723, 23905, 50953, 25724, 24241, 97484, 57132, 77934, 69667, 50915, 11626, 73797, 29433, 4798, 41550, 20011, 42517, 11359, 51507, 63059, 54821, 23456, 76911, 4, 9098, 65049, 7484, 66553, 12374, 7905, 62583, 34084, 9366, 4102, 13968, 54642, 17415, 15996, 32183, 46090, 54073, 15995, 936, 96221, 6906, 28667, 72106, 61209, 15354, 5958, 23260, 832, 45879, 100985, 1191, 60158, 26320, 8493, 79855, 13678, 10286, 60664, 34823, 26061, 21659, 10960, 48362, 24360, 91918, 24532, 29918, 2111, 34920, 61569, 22834, 81218, 30765, 17217, 20018, 2423, 75750, 15401, 46053, 91289, 61579, 6304, 87982, 7490, 77861, 27883, 2963, 14484, 1757, 29730, 6045, 87499, 29461, 51531, 17833, 80313, 49157, 4285, 29720, 30283, 16878, 78919, 35876, 15969, 30753, 1239, 30634, 11695, 5288, 66762, 3727, 3933, 7701, 33752, 43087, 12359, 12083, 88208, 14489, 67689, 64830, 8050, 4286, 19127, 41498, 27465, 1129, 31256, 49378, 1840, 27686, 406, 8429, 21049, 2917, 29581, 419, 9101, 65411, 12097, 14645, 20573, 42702, 77910, 3679, 69864, 19985, 2151, 52064, 86414, 110939, 14928, 4181, 20467, 76849, 1990, 20141, 69946, 4943, 43521, 39029, 47326, 2364, 24040, 34456, 17807, 9764, 7663, 4103, 15989, 35107, 1285, 38, 3391, 3401, 1497, 6560, 7836, 1132, 39860, 3099, 25888, 22405, 5389, 9737, 2149, 2953, 6299, 7482, 911, 1262, 51798, 21752, 21856, 26063, 57488, 30109, 25070, 18479, 7573, 9503, 2932, 21708, 5977, 9867, 29193, 12391, 12081, 81080, 1491, 22414, 6486, 3869, 3053, 20960, 29170, 26336, 34333, 15380, 3629, 4012, 18552], [23562, 15, 2646, 29115, 2630, 16551, 4312, 6402, 47618, 6223, 55619, 28850, 44115, 28032, 55698, 57849, 17831, 32094, 11481, 58582, 51817, 20799, 45396, 3424, 102164, 12988, 20448, 33146, 20302, 2344, 20, 55128, 19476, 54044, 36711, 12086, 17900, 96594, 65368, 19800, 41582, 82971, 1831, 14488, 21303, 40077, 16882, 47598, 12381, 41108, 41449, 47095, 31260, 57967, 68857, 41067, 4173, 9496, 42272, 16282, 6598, 15125, 18473, 62044, 6988, 17937, 247, 7402, 15605, 29993, 30145, 70655, 6174, 54817, 19491, 498, 33029, 87105, 491, 19227, 31459, 25571, 6519, 99716, 11272, 5462, 6970, 65284, 16041, 17846, 5526, 47338, 24283, 37098, 22060, 18207, 21302, 80253, 33361, 28020, 12194, 7069, 1493, 1127, 3625, 4192, 3746, 499, 11614, 10663, 25943, 2434, 3744, 41468, 4542, 80, 9837, 28001, 10968, 1482, 12217, 979, 20507, 5604, 16804, 1950, 91181, 30356, 44885, 50833, 14147, 21779, 62448, 20987, 47971, 30733, 11040, 66203, 618, 7494, 48882, 1695, 13612, 20002, 401, 28618, 7906, 3168, 2359, 21272, 51415, 18568, 17, 11824, 22905, 36894, 15930, 35251, 2100, 54644, 17122, 2832, 177, 39501, 71988, 15364, 34296, 5938, 44861, 8521, 186, 9235, 7717, 60740, 9847, 6488, 1168, 18992, 326, 46, 2136, 1628, 38502, 2814, 28419, 9882, 13074, 12487, 1767, 4566, 6209, 3171, 6404, 3748, 2616, 32067, 19007, 10259, 15367, 2032, 11992, 5460, 3756, 8547, 32857, 1623, 831, 6181, 10680, 15811, 11617, 8274, 4548, 31092, 28860, 14847, 16263, 34910, 1097, 11936, 10444, 12108, 1397, 17571, 43379, 14646, 4793, 6522, 28410, 73809, 31227, 5390, 27303, 13361, 30697, 13880, 27750, 2732, 328, 9138, 1268, 14490, 61563, 4033, 6491, 1939, 37800, 9212, 10132, 33482, 26960, 14141, 5789, 1354, 6310, 8175, 14417, 5831, 4804, 5308, 17587, 26307, 2645, 2332, 3736, 7419, 18166, 13193, 6562, 1474, 1052, 10120, 13593, 8709, 13771, 7404, 7543, 81, 3907, 603, 10514, 11276, 818, 23674, 12219, 5840, 38003, 14577, 1377, 5967, 4817, 184, 13208, 14163, 20319, 34884, 1539, 7699, 9647, 4558, 1053, 14464, 6973, 970, 1626, 1761, 3931, 1146, 6820, 3252, 2698, 70662, 38341, 125694, 3881, 102030, 20304, 73527, 80404, 88019, 66721, 31043, 41494, 50603, 96472, 123330, 99002, 99145, 3349, 35053, 22471, 105886, 31082, 42650, 13672, 46662, 21054, 25522, 68797, 27190, 30309, 20877, 87400, 34783, 108133, 69952, 41581, 1709, 51226, 86904, 84857, 104724, 14249, 4003, 8617, 29977, 102921, 49119, 56357, 63588, 71505, 74277, 28727, 44827, 45528, 1358, 90247, 19409, 21359, 14278, 108102, 84106, 28965, 71030, 82, 35054, 49107, 122153, 102928, 62142, 65068, 60303, 78915, 41366, 12849, 2834, 62833, 11279, 84390, 72203, 44085, 110991, 91817, 99699, 20994, 103941, 80889, 125612, 13076, 15994, 21016, 13850, 91909, 99867, 50472, 10510, 118611, 51291, 16924, 107522, 96512, 7970, 115375, 52999, 18490, 79479, 41357, 69898, 52259, 78940, 69699, 7431, 108118, 11525, 82995, 5397, 32687, 23689, 9700, 42688, 68793, 34914, 8618, 44904, 98510, 90397, 14431, 7712, 11029, 41094, 33920, 91922, 2713, 3603, 90795, 125131, 12228, 7435, 18477, 120654, 13079, 4956, 24363, 107961, 89080, 76381, 9868, 36178, 22470, 68411, 20779, 65833, 52757, 51150, 6396, 1859, 106444, 70614, 68543, 42701, 5857, 30274, 23533, 125152, 7882, 96929, 99379, 34924, 127783, 72554, 96917, 10815, 99640, 93202, 42289, 33311, 41584, 49850, 66056, 82076, 18619, 101255, 12473, 56877, 34777, 95407, 78475, 966, 51692, 36055, 9974, 46238, 37310, 63598, 2950, 71504, 10946, 50846, 4953, 72666, 84860, 3889, 22952, 79472, 124759, 11688, 14434, 118621, 13409, 115503, 51292, 73255, 19845, 98000, 111856, 3178, 11808, 36034, 96520, 107533, 66495, 27677, 20136, 71115, 84904, 2838, 1921, 30778, 90391, 126497, 56359, 40956, 99709, 1147, 71135, 19113, 103203, 35139, 8497, 17523, 20502, 2827, 17839, 11990, 1933, 106435, 33508, 42281, 4939, 80439, 103223, 98993, 48033, 52012, 48477, 109300, 98511, 83393, 66220, 106452, 32179, 27522, 72836, 93296, 113631, 30779, 91908, 112973, 99378, 65035, 46829, 46505, 51809, 65075, 39056, 36040, 78329, 32169, 99625, 49678, 118352, 117378, 3604, 30229, 126905, 102719, 127792, 1382, 84852, 13342, 33931, 126395, 20862, 6613, 110561, 18576, 58348, 45392, 73426, 78704, 33754, 46234, 62, 49521, 82965, 38618, 109242, 2015, 68504, 91732, 98602, 3093, 28682, 375, 33738, 71902, 67384, 103515, 17212, 88783, 114083, 99719, 23455, 106156, 53158, 41497, 73582, 23698, 10952, 1836, 50783, 47015, 68496, 95386, 7654, 10950, 118597, 16278, 94634, 40220, 82081, 121781, 13951, 6607, 41501, 7479, 88289, 6414, 36852, 72214, 2831, 104685, 30625, 14293, 102776, 105884, 43545, 56388, 97057, 11269, 7430, 79256, 14649, 8619, 31797, 24635, 12517, 44901, 32764, 16271, 108190, 15358, 99485, 1665, 51710, 4107, 52744, 2333, 6508, 71065, 103511, 22564, 41022, 83000, 2335, 91091, 32773, 10829, 9695, 122142, 19899, 99714, 4928, 24054, 40717, 73856, 111544, 119380, 70056, 45889, 111281, 14299, 3593, 3729, 13200, 46826, 37469, 27987, 116996, 88303, 51416, 20734, 2959, 59247, 15107, 16603, 4586, 67567, 126478, 9702, 91411, 40721, 36759, 76363, 1668, 16275, 17211, 44074, 17227, 22940, 106434, 12521, 108020, 16626, 82939, 15238, 49837, 70622, 111805, 84951, 98624, 117379, 28263, 23760, 91823, 51304, 69361, 87999, 41109, 8003, 41313, 9623, 82959, 108187, 78477, 19060, 40221, 28313, 71523, 15371, 10210, 12075, 109296, 6397, 51298, 22031, 57855, 30320, 53851, 77749, 3786, 2783, 123204, 51691, 66221, 41484, 106174, 72124, 101819, 28037, 30770, 63532, 60323, 57486, 58401, 19233, 33729, 99161, 64939, 10115, 5692, 24020, 50798, 50219, 1819, 70051, 56705, 87716, 93218, 37470, 85953, 69866, 13665, 87354, 89394, 84383, 60967, 78490, 125682, 3766, 64992, 9862, 10129, 33213, 59271, 74445, 23338, 32158, 12472, 11461, 98604, 49116, 45400, 21258, 44206, 25661, 34911, 49173, 74442, 90380, 115817, 64210, 76172, 4090, 123756, 14470, 124787, 16544, 102464, 30230, 104021, 67265, 11119, 120664, 36455, 35815, 37468, 27674, 108130, 322, 125671, 124789, 14926, 84332, 8663, 111810, 18995, 37709, 12589, 391, 100106, 78154, 32171, 108115, 17818, 35252, 118598, 95562, 27822, 83505, 95405, 32173, 102458, 19457, 77925, 72517, 73208, 106257, 16945, 25516, 102576, 112709, 81177, 1669, 13607, 32187, 115053, 6376, 119406, 7997, 59153, 54710, 38386, 74108, 89114, 45654, 31083, 119379, 70052, 27818, 27409, 87714, 46445, 95216, 98516, 6983, 124777, 45519, 91729, 26348, 51807, 50440, 84526, 69661, 18476, 41093, 2961, 11379, 2747, 83499, 65840, 70840, 97979, 126413, 4450, 23142, 35041, 106183, 50927, 6182, 80970, 51528, 34759, 87214, 2954, 49854, 73214, 77438, 10667, 42325, 19990, 67501, 8616, 22964, 56358, 3721, 63595, 48619, 73078, 14242, 103942, 56863, 69856, 32645, 13025, 39334, 51800, 119417, 11753, 50481, 1355, 122203, 37768, 5083, 69801, 50448, 88290, 90793, 1771, 5269, 17828, 34903, 3334, 36850, 28014, 1653, 8138, 91412, 78767, 87211, 108374, 37710, 37491, 107712, 58815, 49129, 86901, 1117, 31129, 48681, 69775, 79480, 40218, 9693, 49155, 119368, 70660, 85958, 99157, 37525, 15365, 48779, 20320, 126399, 125170, 63928, 127776, 61587, 25532, 49183, 19134, 65856, 28841, 98866, 16276, 41496, 82437, 83969, 93299, 28981, 33878, 32375, 52815, 101279, 118038, 120647, 60139, 1047, 93193, 88011, 91902, 93151, 104951, 15386, 50475, 2150, 14042, 14910, 91744, 40870, 5539, 7451, 41493, 3521, 1361, 125072, 113624, 68417, 21968, 108376, 97197, 26330, 99718, 71669, 35049, 57510, 7987, 28654, 50458, 13192, 11385, 13674, 417, 3896, 15821, 54550, 3638, 55105, 118599, 21360, 49126, 125168, 70037, 1352, 7427, 3891, 27382, 85969, 69069, 54603, 110155, 51696, 73063, 63531, 27840, 4988, 20577, 103226, 2283, 104626, 1708, 35764, 123521, 49182, 108372, 5991, 20786, 118982, 83288, 6395, 3157, 14933, 91270, 7458, 24339, 42651, 125164, 99717, 49401, 22015, 66224, 57330, 24358, 50433, 104945, 102727, 7154, 91829, 15408, 64991, 8520, 12602, 3294, 48062, 12210, 87463, 125079, 125663, 88143, 10358, 104728, 85968, 32186, 93904, 33728, 14099, 66429, 62596, 44369, 51293, 108135, 119388, 10713, 16231, 8705, 56889, 26582, 18213, 51793, 13570, 7979, 95387, 9471, 51286, 16873, 2705, 117895, 13401, 25467, 46497, 46813, 68851, 87703, 21361, 107538, 20955, 26587, 47269, 118614, 6967, 27821, 34768, 19229, 51695, 5386, 46518, 27403, 78333, 84345, 104709, 1642, 44075, 41444, 1476, 10944, 72342, 86902, 82983, 72680, 87713, 115507, 25113, 108132, 19994, 12098, 70012, 27217, 86204, 28749, 53843, 12570, 82970, 84327, 54062, 41379, 81601, 8058, 11809, 27378, 127778, 16023, 12504, 51290, 41486, 46509, 35056, 59769, 33976, 73422, 55746, 20951, 25514, 74097, 16005, 8062, 108134, 98495, 1481, 1179, 3461, 9209, 13050, 1770, 3683, 29555, 16616, 980, 30782, 12641, 3770, 4802, 29931, 26965, 1900, 1616, 16881, 2474, 2862, 7053, 298, 973, 9635, 1660, 3345, 968, 13760, 32172, 13512, 1966, 5917, 17709, 19602, 7623, 11267, 8049, 4063, 10284, 16291, 2898, 6891, 24865, 34, 5393, 9144, 11348, 6389, 173, 28869, 242, 11858, 24328, 1142, 2830, 5586, 12348, 9237, 19338, 17609, 7364, 2620, 13149, 13214, 9870, 27741, 22710, 15769, 23910, 19096, 5960, 20902, 24297, 13658, 974, 2224, 11932, 12579, 2772, 2447, 5153, 17682, 30409, 12422, 23463, 1666, 7682, 3968, 15234, 20307, 16808, 66604, 43275, 5617, 17955, 20738, 1254, 41297, 8502, 8252, 1348, 58655, 13826, 10208, 17820, 7403, 41556, 16415, 2005, 1360, 568, 1322, 21000, 189, 2264, 1166, 15720, 29060, 1928, 3595, 4795, 21979, 2127, 16604, 7150, 234, 1936, 9216, 14811, 11, 2768, 11107, 66777, 28895, 20657, 992, 49924, 39953, 101, 3167, 58402, 38879, 31889, 18530, 22226, 36476, 10221, 1636, 10067, 3612, 70394, 2293, 5380, 21368, 18277, 7100, 10685, 32648, 44076, 14253, 23279, 8007, 26272, 42344, 32924, 21742, 11305, 4002, 87917, 28635, 18206, 4617, 12982, 10181, 9475, 13575, 43047, 3325, 16558, 3254, 41527, 42089, 15818, 605, 495, 5325, 4154, 8016, 1167, 8166, 2726, 228, 3636, 20766, 1987, 12211, 15954, 2754, 4615, 26150, 5535, 71, 15241, 13117, 3501, 229, 20473, 2799, 7650, 2258, 1813, 1711, 3620, 307, 21926, 42659, 37112, 768, 47551, 9102, 18565, 52813, 29920, 67635, 14739, 52426, 16303, 21153, 11802, 24053, 18327, 61116, 15413, 19814, 24262, 5345, 28976, 8354, 3591, 22904, 10578, 8256, 1040, 28271, 1267, 10822, 8532, 1652, 10803, 63793, 42297, 27707, 11368, 52309, 1143, 33721, 36743, 6900, 19522, 24094, 10214, 2329, 61493, 33744, 88109, 20598, 3668, 63053, 4188, 16601, 26022, 87599, 20099, 16277, 4712, 1064, 26589, 11022, 33520, 22879, 38833, 41211, 33488, 38876, 9696, 16409, 56937, 22826, 50304, 35817, 31124, 46524, 30850, 2897, 15709, 1370, 93038, 16345, 44871, 11623, 20712, 7011, 2277, 23428, 91513, 2270, 47353, 14887, 41018, 12616, 20470, 40054, 24918, 85207, 34897, 40390, 750, 28979, 8065, 3773, 24991, 9492, 325, 5458, 81748, 22399, 16559, 52044, 29002, 25791, 13934, 71274, 50793, 30959, 32679, 12132, 30349, 57489, 44867, 50416, 57140, 56512, 27829, 36457, 54443, 19902, 7436, 383, 10953, 19, 2353, 4271, 2926, 3499, 1116, 1065, 5533, 3994, 9354, 24348, 19226, 9976, 3886, 6633, 844, 10375, 24549, 20087, 28288, 64704, 16055, 3813, 25127, 59445, 10642, 14550, 27753, 2216, 4024, 36825, 15868, 9151, 44916, 3411, 30967, 17457, 39187, 14260, 978, 30628, 19255, 4448, 3314], [12212, 1537, 9410, 24028, 26046, 42447, 19222, 47796, 28720, 4883, 14391, 43540, 23647, 671, 13518, 2722, 2144, 3469, 9750, 906, 4455, 17810, 1170, 4386, 6781, 2753, 57655, 12513, 31388, 16682, 40934, 5537, 2340, 166, 1627, 7133, 302, 6974, 3764, 15313, 31041, 20442, 27467, 2653, 8144, 9125, 316, 18849, 24592, 576, 14894, 4842, 26491, 22212, 599, 5602, 1266, 5599, 387, 3094, 2711, 295, 1823, 6811, 2253, 28806, 69071, 34384, 3169, 7953, 6624, 100833, 21341, 25242, 40713, 4870, 2263, 55518, 26024, 19121, 49217, 4278, 9962, 12507, 10460, 32960, 71580, 10074, 4208, 106439, 41015, 19490, 71537, 9961, 22747, 104597, 82303, 11307, 121751, 66760, 47063, 104181, 51336, 3170, 36255, 23703, 91324, 17468, 1766, 32388, 26025, 40398, 95393, 60808, 16995, 8741, 7366, 3256, 8368, 72689, 17635, 1696, 165, 17066, 33216, 80771, 3396, 13552, 110120, 109353, 46281, 66079, 14362, 11049, 94672, 17115, 23528, 83258, 8624, 26137, 67526, 20396, 55926, 69776, 23895, 22244, 35043, 31807, 27549, 2833, 22533, 23757, 17634, 497, 40724, 12591, 17865, 65316, 8632, 98499, 77929, 3249, 8060, 127461, 22404, 22028, 1169, 67068, 5622, 36851, 56022, 3901, 37792, 57005, 5319, 8054, 18562, 24542, 41301, 58158, 11275, 3412, 75133, 82246, 9094, 25263, 75183, 39890, 96215, 19909, 53828, 40180, 106274, 54661, 52768, 72964, 52374, 6225, 84519, 35651, 28687, 14514, 31375, 45599, 68564, 26926, 2816, 68799, 9597, 15828, 93469, 90790, 67561, 55138, 46496, 3219, 30269, 6596, 8732, 15349, 7826, 48479, 11129, 25526, 51432, 26142, 33874, 106146, 98003, 22022, 3519, 37692, 62470, 8013, 14892, 942, 21021, 66299, 42103, 11933, 51202, 96839, 25310, 1326, 33620, 65176, 2899, 7437, 7990, 66714, 22947, 44124, 11310, 420, 191, 35202, 14900, 67572, 15110, 61904, 46021, 32977, 23392, 6615, 37754, 11697, 31545, 60462, 6215, 58290, 43227, 9769, 41665, 9221, 9095, 80423, 8373, 14166, 12560, 1387, 15396, 3780, 96938, 7412, 50633, 33823, 114610, 54920, 42309, 6911, 34311, 507, 68817, 7063, 28289, 12199, 97587, 60246, 21489, 2697, 922, 6048, 3159, 664, 207, 4013, 3288, 3488, 9192, 1035, 10682, 1264, 52, 493, 17201, 7989, 8696, 5312, 3497, 42, 11115, 4331, 4178, 3795, 4135, 21314, 7893, 3438, 6987, 3258, 6822, 504, 10615, 57, 1016, 3096, 6377, 42654, 6971, 77940, 36870, 23662, 22579, 35072, 7622, 21967, 15379, 397, 3301, 8956, 7321, 16398, 44103, 30780, 53913, 4027, 22884, 58421, 11026, 52816, 85864, 3897, 13671, 2160, 5491, 16928, 20139, 25946, 9105, 16403, 3218, 54060, 16685, 29856, 22472, 1778, 39716, 1407, 18426, 14812, 5098, 3684, 20616, 14170, 22876, 24996, 4242, 3179, 2969, 19027, 2735, 393, 3429, 33742, 2094, 20103, 6188, 3312, 30, 9829, 1640, 943, 3597, 602, 11685, 18971, 19429, 2282, 1947, 5732, 20010, 21336, 34286, 2980, 7068, 1654, 4268, 30323, 49105, 11530, 13363, 34314, 10285, 1351, 13415, 22265, 73071, 68113, 49559, 50105, 82239, 29343, 67040, 46229, 18650, 28041, 16983, 80886, 19345, 56366, 1821, 7532, 31033, 317, 14651, 17395, 16402, 27928, 50495, 47012, 44479, 1252, 627, 9229, 17550, 7698, 614, 42042, 22008, 10351, 4590, 12449, 7168, 17821, 28753, 59043, 2724, 62873, 16829, 47594, 32690, 7171, 19435, 42381, 26986, 5318, 12375, 25899, 3753, 3580, 96116, 66184, 3483, 34035, 955, 55670, 52818, 3257, 87564, 96528, 3035, 10355, 480, 1832, 8257, 28527, 18226, 12725, 45523, 45750, 7618, 4789, 81729, 16359, 8045, 17643, 25, 29438, 14400, 93589, 31042, 10034, 2273, 3399, 27627, 23053, 11382, 51806, 720, 41553, 7745, 19425, 26309, 60390, 70057, 66674, 41315, 29427, 12071, 79754, 22053, 4890, 50652, 84739, 12366, 47455, 20788, 5061, 1124, 1099, 1500, 16562, 2029, 14801, 1271, 6393, 1395, 9168, 43075, 25178, 722, 4233, 3281, 11600, 3909, 5888, 5829, 3180, 421, 9470, 3601, 4698, 58263, 30682, 27631, 7131, 13165, 17916, 2946, 13381, 4839, 4440, 21435, 8159, 2920, 5745, 12072, 3782, 1542, 5159, 49123, 30756, 14475, 7119, 3250, 11378, 114, 11667, 22478, 5392, 24613, 25264, 12216, 64398, 3330, 41374, 4796, 36401, 908, 6027, 4947, 35038, 36479, 16618, 26967, 81795, 5982, 31179, 5989, 73344, 733, 11998, 39806, 10563, 3982, 13088, 24916, 38783, 21350, 4570, 29769, 59431, 17238, 5940, 60160, 3286, 33043, 2818, 47001, 29938, 25480, 10169, 26827, 26717, 67070, 12426, 55135, 25921, 113, 9844, 93145, 16625, 23335, 6502, 31021, 39206, 31651, 6399, 258, 7938, 1787, 40, 14259, 4981, 8326, 10265, 12315, 14363, 17167, 7204, 2381, 7117, 3594, 26228, 9220, 2422, 6611, 2974, 190, 179, 3133, 914, 21260, 28521, 17136, 11690, 3626, 252, 27704, 1686, 34487, 1402, 3848, 5525, 13741, 20610, 25365, 10260, 25682, 19552, 18153, 69774, 1164, 44026, 29027, 18869, 12746, 1186, 6807, 17209, 7457, 3121, 56057, 34293, 9485, 51, 4214, 15730, 16253, 67560, 19679, 4724, 39345, 4204, 5935, 2955, 13660, 17658, 921, 3150, 1123, 3149, 4397, 1003, 400, 43034, 3397, 3997, 8604, 76859, 11243, 5861, 19998, 20012, 9836, 8640, 13513, 60307, 749, 9244, 7308, 50935, 7464, 7137, 6984, 3524, 388, 1485, 578, 9373, 5643, 3804, 49, 28220, 3032, 6887, 22993, 62469, 27, 11179, 962, 15228, 16621, 24117, 903, 10824, 40012, 20783, 18594, 8263, 4055, 11930, 2825, 48778, 20605, 11357, 7401, 4065, 2031, 3485, 6907, 11621, 2703, 23560, 18285, 3238, 12470, 487, 16050, 9477, 7991, 16668, 11108, 7834, 3637, 7984, 12317, 4193, 1175, 14486, 3564, 9089, 8749, 20880, 9421, 1687, 13259, 4221, 16782, 12267, 2392, 4969, 1063, 11618, 98, 5332, 10647, 48, 7060, 28990, 380, 3161, 14533, 7231, 13927, 7128, 6107, 1587, 2371, 2016, 4379, 3944, 15101, 33002, 3248, 331, 1938, 2634, 1718, 110205, 38503, 10451, 25375, 105055, 65243, 64575, 8526, 65245, 49524, 45856, 88593, 84136, 104109, 65710, 63202, 76163, 39051, 69430, 7823, 9169, 62931, 62940, 57670, 25256, 38610, 9813, 79967, 88721, 71983, 72007, 19459, 13182, 24278, 36873, 81814, 28269, 7740, 53927, 2396, 77698, 25001, 17555, 52411, 17444, 72293, 107316, 92558, 107507, 77939, 11675, 87512, 99348, 19003, 32778, 30069, 68107, 118452, 11817, 62712, 37580, 17931, 2455, 49526, 49761, 71543, 100348, 42955, 40052, 7829, 5000, 8735, 484, 73859, 30621, 75398, 70010, 108129, 2295, 2262, 127873, 59585, 5331, 123161, 18697, 89120, 40055, 4203, 74535, 32185, 13280, 2389, 12420, 120704, 52544, 56702, 67331, 40122, 77111, 1762, 92782, 55327, 83599, 1158, 60856, 89605, 17416, 24985, 22174, 9955, 30156, 119362, 23402, 27130, 33030, 91425, 25103, 51782, 109021, 74671, 623, 7838, 3712, 28594, 105937, 92911, 115491, 92810, 93827, 36258, 104778, 21128, 111651, 193, 29729, 82234, 4589, 10635, 78151, 30277, 54448, 98904, 5633, 13965, 40720, 126339, 23692, 10359, 9484, 116336, 15707, 76814, 126912, 8340, 2345, 8333, 33943, 763, 3201, 119341, 41680, 95901, 47283, 80291, 108833, 21493, 21581, 128217, 121478, 44390, 93711, 18556, 961, 25473, 43236, 14265, 112790, 7950, 73853, 85000, 129762, 117271, 106292, 36093, 76841, 32177, 111, 61102, 76821, 47638, 119288, 119335, 8609, 45083, 11462, 1909, 21956, 945, 99369, 91970, 48030, 39702, 10594, 24903, 4207, 21577, 19892, 80393, 111014, 16297, 94819, 28022, 94927, 17193, 43072, 54015, 28519, 208, 15168, 345, 11751, 110426, 2584, 33063, 7743, 71953, 7076, 101420, 25918, 51773, 93867, 112870, 8488, 71551, 39546, 4606, 17599, 13211, 62098, 4227, 38173, 109080, 109019, 65688, 7632, 82509, 65325, 11796, 91079, 75222, 71539, 5, 58413, 39536, 18549, 93374, 20318, 49050, 75902, 92557, 28833, 94638, 76792, 41479, 18533, 118516, 79965, 28991, 20091, 104798, 4705, 34510, 75221, 17606, 8785, 11482, 42634, 72453, 37473, 21058, 11654, 3624, 11306, 119613, 25783, 65711, 33026, 9704, 44942, 101195, 26585, 63887, 15895, 128201, 11653, 106303, 23852, 80769, 50218, 3473, 112140, 16053, 11466, 10512, 38002, 5278, 113990, 9225, 127846, 10170, 2130, 85355, 32368, 33283, 55160, 63300, 35547, 7694, 21570, 76236, 11304, 27135, 82206, 2716, 14006, 41926, 104184, 11387, 126345, 626, 28257, 80776, 3902, 125632, 48412, 22738, 64566, 101731, 15875, 101430, 27126, 121301, 25474, 55899, 33493, 3240, 49174, 84460, 66836, 110420, 64695, 90007, 60068, 101424, 62239, 16814, 952, 93031, 29714, 75472, 30087, 59556, 89121, 11460, 127247, 11113, 30348, 57548, 3465, 28280, 24198, 1307, 5965, 23332, 25848, 113985, 2692, 33279, 38010, 41942, 97146, 106120, 28334, 101976, 16226, 33295, 52760, 77687, 40007, 104117, 43782, 2861, 8377, 27628, 21492, 33374, 19905, 22688, 98777, 10161, 2053, 39332, 1768, 62936, 99227, 19844, 57389, 10199, 65134, 27152, 123172, 85862, 23706, 3760, 44547, 80766, 97537, 54801, 20753, 92212, 47558, 75369, 78166, 56791, 3445, 24855, 50213, 108841, 4220, 27634, 119305, 20749, 16067, 65157, 904, 49234, 3329, 78861, 128236, 3328, 82502, 43436, 7244, 86156, 93590, 32381, 22971, 39545, 1366, 3511, 25795, 24626, 96045, 2098, 6813, 11669, 5996, 127682, 61675, 66185, 30769, 52517, 41064, 62927, 111423, 43265, 28495, 35536, 115824, 6184, 115055, 36887, 1101, 80894, 1378, 62827, 37801, 93341, 20599, 20730, 32837, 44012, 16058, 100911, 49227, 84999, 5970, 47771, 24352, 16796, 59586, 22961, 98808, 37089, 65680, 12107, 1639, 32608, 92221, 22978, 109809, 114179, 57669, 116338, 72047, 33996, 4562, 14196, 82513, 40447, 37863, 5171, 46331, 5156, 1932, 9223, 113171, 122300, 57955, 68505, 127876, 6889, 41025, 9483, 15106, 52546, 14746, 104405, 21481, 36254, 103358, 494, 50231, 103104, 102364, 30267, 912, 17438, 110555, 47736, 121361, 129447, 37789, 9199, 46261, 9107, 28494, 27386, 5071, 18871, 93581, 2321, 11124, 30092, 96985, 92786, 103112, 96131, 47735, 18379, 10168, 10593, 101425, 29984, 78636, 7144, 4176, 33415, 78242, 2709, 25780, 65346, 43879, 14644, 118429, 6089, 121426, 13590, 64777, 20655, 12203, 37782, 27155, 18152, 19117, 50684, 95677, 25308, 11841, 41587, 26915, 32394, 51448, 50034, 90791, 51158, 11679, 11220, 96039, 94884, 16885, 58538, 79055, 23032, 127844, 96136, 741, 70060, 2093, 80749, 67503, 3787, 6989, 53998, 59631, 8375, 43727, 11390, 124013, 35096, 22186, 83598, 36052, 76123, 67081, 23404, 3455, 86894, 111002, 21566, 127853, 97002, 13620, 48413, 60827, 57218, 93580, 79956, 36955, 53906, 2096, 13975, 14864, 67066, 22188, 15181, 58423, 16361, 110427, 65708, 112770, 32832, 30862, 24119, 16619, 16267, 5086, 80883, 50214, 45600, 126972, 3808, 67042, 56707, 18918, 43865, 8353, 97691, 100809, 2330, 10281, 40840, 96988, 76263, 110994, 71253, 129754, 77372, 100613, 4810, 24662, 29641, 24167, 44487, 6901, 30398, 127874, 103781, 93863, 24646, 11823, 40004, 52396, 66630, 41345, 24859, 36872, 86881, 37621, 112796, 44015, 40190, 1934, 16912, 73648, 80779, 104120, 61118, 98048, 17539, 22701, 55920, 85006, 64495, 74381, 39839, 14300, 118523, 71250, 27735, 59749, 54450, 92961, 128209, 8155, 57101, 4018, 14544, 105133, 28501, 12319, 10670, 62095, 92943, 6026, 31321, 67255, 62451, 1941, 80589, 3266, 50205, 34901, 59721, 41074, 44612, 55069, 39699, 119265, 7424, 59446, 4843, 51769, 1286, 3129, 57962, 21738, 3296, 41726, 77118, 108401, 4219, 42946, 5275, 59724, 33363, 25091, 70223, 24976, 7620, 98067, 66382, 34243, 66828, 31052, 8425, 98044, 107896, 58537, 9593, 114194, 883, 7121, 19856, 9977, 32835, 78171, 109023, 16234, 83397, 2110, 105071], [74293, 1937, 105, 80983, 4605, 101698, 48012, 39729, 87516, 23389, 28970, 79951, 24992, 30947, 24871, 11389, 66118, 41009, 111076, 93523, 116516, 5286, 572, 20079, 93951, 70403, 96085, 103350, 489, 4815, 9279, 22986, 8176, 25240, 111419, 14913, 59563, 21563, 53387, 24644, 18160, 51776, 88594, 111010, 35731, 116329, 49026, 764, 81501, 9995, 107894, 121670, 108996, 74677, 10449, 65663, 64765, 24401, 38006, 50207, 52959, 29366, 77695, 5821, 37779, 105365, 7243, 109817, 122290, 48048, 69167, 1155, 82951, 71232, 8011, 43232, 5080, 8536, 43028, 3454, 342, 6899, 10508, 23333, 13267, 57534, 33917, 8820, 59950, 30836, 26080, 79250, 30983, 7809, 16228, 23632, 58569, 127978, 44010, 104931, 101196, 126934, 53487, 8145, 41954, 38025, 43342, 33419, 8574, 46379, 44391, 8361, 41521, 13670, 119276, 95850, 127837, 72041, 41945, 67067, 47619, 92767, 49067, 86926, 70011, 30827, 104792, 110298, 103105, 83953, 129709, 10296, 29355, 44207, 60501, 96974, 10365, 93852, 104894, 65129, 61864, 31801, 10123, 77699, 26493, 61666, 42638, 52283, 28194, 125070, 4611, 19904, 12446, 114839, 98654, 53046, 60127, 84358, 71544, 2254, 17244, 9852, 28512, 35537, 126322, 21675, 5160, 46751, 41995, 80412, 4007, 98811, 25261, 6608, 31017, 54484, 90005, 10505, 36360, 39526, 25903, 93347, 28516, 77692, 109041, 96996, 76510, 58336, 6896, 27842, 23685, 61679, 22020, 98057, 71541, 106042, 103369, 7835, 55714, 77275, 25528, 52532, 2036, 10176, 117277, 4877, 21502, 10276, 13600, 107354, 44029, 21867, 28508, 12443, 5979, 39711, 46930, 9331, 96552, 46212, 44212, 128204, 69658, 115033, 18217, 21491, 3509, 59058, 11309, 20306, 105848, 30974, 64565, 81123, 86928, 55415, 48929, 9951, 56706, 83283, 2976, 37086, 102065, 84983, 39042, 12222, 50687, 54486, 17922, 35070, 37758, 87450, 21660, 74686, 53463, 8160, 89226, 3271, 80415, 86002, 56534, 2807, 85561, 121342, 126888, 49114, 4256, 11479, 40383, 39707, 120717, 62723, 30979, 6666, 33490, 75815, 24602, 70389, 57780, 5164, 62570, 16830, 7232, 112134, 38118, 19643, 67328, 73305, 96976, 56, 39342, 25496, 1940, 204, 50, 747, 299, 215, 1903, 501, 3049, 1722, 3552, 1533, 1858, 4669, 1759, 7626, 4803, 245, 9586, 12862, 8941, 9494, 9014, 15048, 14888, 10118, 2331, 2784, 2403, 39, 24275, 235, 44, 240, 16266, 30835, 21373, 24079, 65, 6785, 3154, 6815, 10555, 1384, 1525, 31251, 28709, 14096, 7348, 25590, 9504, 305, 1238, 17630, 5435, 19934, 410, 5167, 1773, 2014, 10273, 16088, 29654, 3113, 11937, 17077, 11228, 16254, 21984, 2449, 9969, 276, 395, 2157, 15183, 42362, 1946, 19865, 4460, 5660, 17576, 36878, 32594, 248, 19256, 53402, 3459, 1062, 22687, 14161, 42874, 30232, 11854, 47641, 3304, 4594, 14857, 620, 7173, 1171, 8474, 5849, 11845, 12994, 9280, 17470, 1992, 1554, 5853, 16357, 15992, 23758, 22636, 14307, 42926, 24519, 45364, 15471, 82119, 13595, 11116, 6599, 54845, 6129, 29425, 18990, 11855, 21199, 4284, 1614, 13168, 21202, 4222, 26809, 40191, 1411, 24260, 9605, 24285, 4433, 41220, 1776, 6248, 371, 7151, 13592, 14872, 1282, 2549, 25849, 3733, 10976, 7791, 23, 188, 24864, 8736, 10173, 946, 5788, 16611, 33214, 12181, 20797, 14026, 36981, 41985, 4932, 24235, 51094, 100, 2309, 104, 10102, 46527, 16979, 10058, 9219, 28704, 1632, 20581, 6768, 5441, 24272, 269, 5900, 12765, 25225, 1969, 2746, 18509, 11807, 15368, 38939, 8068, 9984, 8946, 19916, 3777, 7228, 4536, 1225, 16863, 39567, 4711, 8726, 4105, 39273, 14168, 12277, 12265, 1802, 68, 12488, 5106, 5598, 274, 2222, 40786, 4304, 30492, 75876, 32366, 9122, 28023, 30724, 28705, 22260, 35152, 41583, 41615, 11273, 56875, 33885, 7649, 58529, 6398, 42321, 1195, 44290, 11693, 275, 45180, 53056, 27769, 2742, 4042, 8378, 85582, 19923, 4459, 21921, 7056, 12105, 2855, 3920, 11799, 26121, 14968, 1830, 14025, 4984, 28077, 9508, 23820, 18690, 2412, 25189, 802, 29452, 17583, 45862, 4409, 2030, 384, 2349, 2383, 297, 1601, 7892, 241, 3503, 8712, 11293, 2470, 3916, 4115, 2935, 8149, 17433, 1548, 2745, 43804, 8807, 488, 6960, 31, 4977, 24228, 7233, 7929, 21037, 3222, 8017, 12739, 451, 26259, 15929, 17194, 2101, 77, 14380, 1359, 14341, 817, 7540, 11703, 1265, 4530, 92, 2823, 76169, 29093, 31661, 7249, 41640, 72605, 6715, 8522, 2560, 10193, 1670, 9007, 50385, 49047, 60657, 8506, 30865, 30444, 17720, 12392, 2723, 579, 80202, 26583, 14365, 74963, 38189, 44312, 64019, 41627, 26069, 1389, 18481, 17380, 72593, 66812, 33428, 63719, 3814, 3765, 12938, 28846, 34782, 1030, 2638, 8044, 22128, 5043, 14, 26211, 20796, 30246, 102026, 3806, 72646, 13488, 21909, 44899, 34454, 6968, 1156, 43833, 6966, 16460, 28178, 85027, 5003, 5128, 81684, 17141, 4989, 39547, 15119, 8413, 32371, 18542, 26501, 94820, 3566, 57365, 13669, 43038, 25172, 86392, 61208, 1983, 993, 3508, 16057, 3268, 11659, 4979, 12160, 33355, 25317, 25025, 30143, 30351, 12399, 7200, 21800, 16034, 10683, 19161, 50839, 30703, 4560, 4468, 14906, 42458, 25104, 41928, 12225, 73206, 70233, 41604, 17549, 12456, 11019, 18829, 42854, 59475, 11299, 39938, 44292, 123153, 2280, 6032, 81224, 8200, 37399, 55749, 88128, 3307, 36302, 14893, 102, 1033, 856, 27358, 4215, 37, 7684, 1045, 4205, 836, 1479, 8810, 21595, 4683, 4257, 22956, 21852, 1688, 26285, 3739, 88, 21073, 5813, 2346, 7367, 19539, 4948, 7672, 12330, 16296, 19977, 1701, 8199, 23318, 3092, 10268, 2376, 428, 8739, 17930, 13851, 2188, 32686, 22166, 3245, 30403, 21920, 58996, 14442, 18384, 7964, 39205, 7086, 1957, 5304, 44302, 40409, 47647, 8606, 45515, 9890, 4872, 4481, 28889, 13973, 16301, 29519, 8955, 25255, 15878, 11677, 18182, 24213, 26070, 2497, 7101, 52555, 3086, 35069, 8262, 2000, 621, 2190, 1161, 5447, 7202, 7638, 619, 9134, 5365, 5429, 997, 8622, 1648, 1010, 4982, 40531, 2199, 7111, 21299, 6893, 6594, 6551, 5266, 20744, 29112, 1349, 6628, 4445, 9, 9730, 2378, 17224, 3710, 19462, 7819, 4778, 10880, 5316, 1258, 1967, 11290, 7563, 426, 2684, 806, 5483, 4263, 10412, 8409, 4865, 22040, 1912, 6857, 4952, 4462, 2385, 4401, 2384, 8170, 2189, 8034, 11417, 1061, 2910, 3880, 230, 1364, 9835, 42473, 2840, 116515, 92666, 62838, 1956, 30975, 2788, 20750, 77054, 36378, 6944, 23761, 55062, 1309, 52573, 42933, 58824, 6006, 98046, 26950, 21323, 17564, 51394, 51788, 80972, 22202, 22051, 249, 55077, 47556, 6033, 3992, 8153, 11230, 60996, 92030, 4728, 11655, 63047, 85371, 3512, 121908, 3435, 20665, 9127, 20752, 15911, 62523, 6212, 17146, 15758, 76910, 31113, 16436, 62955, 38518, 77768, 7680, 61085, 5987, 30824, 14199, 5933, 18342, 22877, 75890, 5683, 17429, 24366, 8131, 10187, 114835, 30287, 32609, 29736, 51431, 78632, 15764, 84222, 12736, 44801, 15164, 13645, 50183, 5992, 4915, 17506, 8580, 26755, 3033, 14423, 63080, 44045, 10478, 73736, 16931, 8133, 37197, 10178, 6547, 9287, 6959, 51447, 61097, 61442, 30213, 115, 75460, 38845, 14053, 25841, 95593, 62388, 26201, 44562, 17053, 16341, 25902, 104400, 23331, 93351, 85128, 62066, 93915, 12360, 31626, 52629, 47023, 99252, 5200, 66425, 38922, 15033, 17872, 57759, 67521, 29648, 32958, 42637, 3239, 42672, 60679, 66301, 10657, 11315, 12680, 114070, 55072, 38800, 254, 2196, 37178, 18850, 44889, 23127, 22034, 27149, 14136, 47646, 62751, 9340, 20433, 33096, 66754, 38281, 10292, 11844, 29354, 5556, 34036, 41455, 18866, 26234, 5084, 93878, 102333, 3530, 5845, 72574, 97951, 53475, 61279, 830, 17621, 80441, 29881, 45074, 39597, 104438, 51436, 3616, 15042, 26747, 3830, 67836, 98537, 77662, 9895, 37393, 5984, 23329, 95098, 5193, 21251, 28874, 31731, 11529, 8171, 18176, 6407, 49177, 48923, 9072, 2758, 17267, 10396, 40137, 32642, 79597, 76058, 14807, 26502, 17373, 3769, 6175, 37060, 86209, 16896, 376, 4259, 16248, 34358, 18239, 42345, 21541, 19290, 61682, 107980, 92384, 29294, 60238, 12779, 34361, 31094, 39026, 65342, 12584, 27308, 86008, 32688, 76957, 1250, 2699, 4951, 1597, 25498, 32387, 79, 41192, 49058, 18218, 1021, 32932, 24928, 5729, 30138, 11787, 49175, 5303, 42653, 94282, 6979, 12202, 81526, 68908, 33198, 6986, 4091, 583, 4296, 3807, 837, 7074, 5657, 5593, 4185, 15404, 4381, 4334, 7635, 10905, 1172, 486, 21998, 12379, 2021, 25153, 330, 8356, 2033, 35304, 321, 3484, 1547, 17463, 928, 7677, 181, 7012, 929, 3010, 15831, 12001, 12417, 36969, 6415, 6338, 14192, 23320, 8673, 22966, 43125, 9747, 4237, 6453, 13520, 23492, 13839, 1651, 4774, 3471, 1293, 1827, 2771, 496, 4076, 18596, 5276, 369, 19671, 19625, 5986, 5472, 51380, 38866, 23878, 47969, 29989, 20913, 33281, 16837, 14197, 4306, 10981, 787, 3754, 13082, 13507, 33840, 3029, 6895, 21973, 334, 17316, 10941, 2401, 33343, 3440, 31619, 14424, 75877, 12575, 11682, 2556, 738, 917, 21735, 2019, 15602, 35807, 5355, 21559, 3611, 16252, 20867, 4224, 16576, 18115, 2, 23524, 55682, 2595, 17459, 5258, 25032, 7219, 9897, 1414, 10445, 6030, 33623, 327, 737, 5606, 2640, 43, 11495, 3617, 85, 2320, 757, 2334, 949, 17062, 17700, 1991, 1136, 1141, 31317, 867, 151, 4359, 10579, 24171, 385, 49072, 10956, 14829, 22702, 10776, 60397, 5859, 1475, 38171, 312, 73302, 30546, 16054, 8625, 14557, 767, 35535, 44200, 1187, 20986, 10629, 11137, 17890, 36090, 8157, 62183, 18651, 44211, 55175, 23357, 13852, 1675, 727, 41012, 33164, 7731, 104787, 2660, 16225, 29589, 1017, 39157, 41542, 5103, 9880, 3212, 61109, 4049, 79732, 16483, 54057, 13722, 53569, 7641, 58037, 2535, 6663, 20733, 6591, 2322, 22169, 31628, 73067, 3265, 28180, 12439, 11739, 6173, 8962, 4258, 3758, 30519, 27979, 22183, 27681, 8763, 10211, 34410, 42628, 1256, 3890, 21213, 38551, 23136, 7553, 3175, 52529, 32836, 68588, 22214, 46465, 5085, 3725, 9674, 30547, 98888, 53891, 25194, 52281, 60834, 45268, 119, 50748, 46444, 97170, 306, 24681, 89, 4019, 8803, 19219, 7189, 25086, 8276, 35769, 10935, 43123, 23751, 3730, 17885, 13754, 25228, 6177, 73845, 37771, 2039, 27293, 11381, 2203, 19597, 11749, 17399, 87776, 21438, 8026, 109, 2103, 719, 729, 9149, 13541, 2826, 2339, 14652, 7607, 22009, 810, 4596, 924, 1978, 396, 2276, 7240, 1, 2846, 4311, 15452, 3323, 26370, 816, 19408, 45933, 4191, 5638, 31716, 12612, 7994, 20115, 39188, 1248, 2856, 13369, 39695, 15360, 647, 13093, 6910, 965, 1216, 6470, 27133, 5052, 6626, 8669, 1318, 774, 19268, 6483, 4863, 6860, 32, 2179, 3160, 7241, 6164, 120, 756, 986, 83, 61352, 5087, 11391, 6455, 2765, 24019, 49691, 5342, 7985, 12774, 25074, 19726, 12429, 8949, 26676, 37581, 23768, 1799, 24711, 72389, 39221, 14269, 17114, 16352, 20814, 267, 38940, 5645, 62226, 64699, 5863, 11825, 8352, 69673, 9973, 278, 33167, 50441, 9222, 6908, 28515, 2370, 12155, 5866, 7811, 26591, 81531, 15763, 23056, 3467, 1381, 6890, 12000, 2112, 19837, 19928, 25207, 17710, 25249, 46454, 2689, 758, 33199, 277, 2559, 231, 5676, 1019, 10517, 2223, 4904, 18688, 8794, 93, 10053, 4876, 7, 14142, 2907, 944, 7625, 2180, 1943, 23825, 127, 22410, 7002, 2459, 2563, 5891, 17207, 8620, 5333, 99, 1335, 35872, 14370, 2781, 1613, 6187, 2137, 44811, 35554, 5524, 16036, 31931, 12214, 2654], [76730, 45485, 52345, 68404, 8178, 10912, 1496, 4328, 7850, 12458, 33272, 3038, 8525, 5334, 49199, 14841, 1612, 15113, 64950, 28884, 4859, 38925, 7411, 20125, 22388, 11975, 3775, 15385, 3900, 3486, 1041, 721, 18389, 3781, 1296, 3510, 3, 10316, 36, 5869, 7707, 41078, 20725, 10223, 50933, 1023, 24590, 9228, 9893, 21444, 17641, 31768, 2038, 62633, 34049, 19541, 3106, 34480, 8635, 28206, 32338, 744, 6790, 1719, 23285, 29058, 8970, 7335, 8053, 508, 3143, 25094, 2192, 4267, 9085, 4707, 16072, 4527, 7837, 19707, 23103, 10632, 20636, 23001, 26196, 10302, 2612, 41, 29285, 4081, 3522, 4938, 4088, 7169, 4764, 4048, 21912, 3289, 1396, 662, 211, 5855, 35641, 6161, 23659, 5064, 4040, 6561, 2547, 3344, 1246, 14301, 5007, 3904, 47, 3573, 11134, 11470, 736, 5971, 6812, 2117, 5277, 16241, 45829, 7120, 5609, 16272, 8246, 34435, 5578, 9491, 19332, 17652, 25898, 31065, 416, 878, 4838, 1270, 2984, 8484, 10274, 3039, 3490, 5285, 22475, 2655, 1330, 1954, 4896, 16424, 4198, 1226, 5412, 975, 2981, 746, 587, 6894, 8260, 1140, 8321, 3153, 4254, 4588, 1605, 8627, 6029, 51451, 2725, 164, 118592, 13275, 10073, 56567, 1243, 123095, 49858, 84680, 84538, 112150, 4957, 43768, 90305, 106782, 4057, 107171, 83071, 29353, 13163, 64896, 46780, 2812, 29852, 7181, 127744, 56091, 4253, 23113, 56559, 42440, 67470, 2122, 9869, 1713, 87808, 66354, 81544, 100834, 6046, 62049, 14525, 115425, 52341, 82583, 19417, 28086, 5375, 55657, 38889, 15434, 4874, 67125, 2357, 7730, 52511, 126223, 113191, 18288, 65021, 15531, 84715, 48075, 30609, 12624, 2570, 107988, 24872, 41536, 97090, 64456, 64553, 20437, 38837, 111927, 101390, 14516, 49406, 40506, 84651, 49869, 89219, 116962, 84176, 3457, 8376, 122241, 87000, 14206, 27935, 7847, 45431, 75436, 9878, 68100, 7733, 9327, 67142, 28412, 10262, 106542, 10246, 43752, 55819, 98241, 37548, 32310, 17264, 30217, 103389, 10068, 43728, 17014, 80164, 31743, 3221, 126173, 83259, 125524, 51619, 40687, 9883, 28876, 122492, 11818, 117689, 85603, 1241, 55658, 32126, 94945, 37074, 33964, 66348, 55823, 10106, 72793, 117412, 35132, 4390, 44067, 106775, 8719, 32603, 3041, 3231, 22172, 931, 26729, 6871, 45662, 58135, 2366, 114916, 84300, 31458, 37075, 126426, 116976, 31444, 77551, 53938, 8538, 105280, 53303, 6726, 4171, 125748, 16365, 6874, 4885, 60560, 87984, 68479, 260, 27041, 38504, 77549, 10659, 124816, 5457, 11231, 11017, 67585, 81533, 314, 33086, 97102, 76345, 90, 85227, 594, 6961, 86913, 27283, 3527, 98745, 82016, 63315, 86027, 19708, 66269, 120572, 9716, 79309, 30819, 27615, 1747, 75006, 36881, 65019, 33606, 26715, 99448, 119822, 121915, 447, 83487, 10839, 48468, 51789, 20629, 84741, 25769, 19368, 103157, 36004, 26332, 1185, 91047, 86907, 20978, 35871, 24478, 16478, 9866, 36182, 5274, 56078, 53686, 121911, 63275, 10101, 33587, 74375, 85942, 60411, 87086, 11798, 1955, 4848, 22203, 112090, 12814, 41537, 47990, 93337, 66817, 58125, 39610, 54933, 69151, 14832, 12082, 8126, 98756, 85596, 42330, 94103, 35880, 115783, 88818, 25097, 30345, 47020, 4960, 104743, 91330, 92653, 6410, 967, 65179, 2490, 86021, 29659, 32312, 60473, 60570, 1029, 96742, 94952, 110632, 114805, 27520, 45306, 13114, 98970, 7362, 127731, 67849, 100835, 2810, 16021, 98167, 14817, 99770, 2109, 22967, 85641, 45919, 14507, 100493, 30231, 84731, 78950, 67137, 62070, 76348, 64071, 77598, 15968, 100336, 3716, 25860, 86242, 10099, 118775, 5189, 27622, 1895, 29509, 20735, 84721, 25027, 50755, 119837, 124360, 18949, 119115, 98170, 96385, 11139, 105259, 57429, 988, 76845, 53114, 15904, 37382, 23111, 62063, 123416, 8478, 92463, 40237, 94951, 126192, 59005, 1244, 77552, 19004, 7975, 37490, 84200, 74576, 66917, 111916, 38931, 56277, 98854, 83246, 76122, 81546, 32120, 16813, 22705, 85931, 4986, 35825, 23048, 65941, 108496, 63278, 1589, 5993, 15035, 123400, 12824, 66365, 75237, 104479, 85921, 34081, 116129, 39235, 38517, 45878, 7311, 82697, 99978, 24838, 89126, 67485, 597, 2123, 21206, 1961, 4159, 98968, 66146, 96760, 96029, 119823, 17371, 29412, 85986, 99769, 74502, 15910, 123116, 51444, 86311, 68036, 60478, 63603, 38228, 25041, 5311, 73924, 33411, 5454, 20409, 24904, 122686, 102548, 24411, 100816, 18765, 124981, 22891, 18405, 26726, 91512, 41162, 114915, 83224, 87022, 8052, 107450, 19959, 36908, 14477, 59726, 101510, 19339, 77122, 30343, 106318, 126078, 84687, 62562, 127463, 24369, 10062, 47014, 41599, 92743, 4442, 15876, 57535, 23033, 69259, 112548, 9217, 111215, 74, 18767, 1385, 84229, 10366, 5887, 2860, 20231, 29639, 32372, 91567, 84726, 80151, 51667, 41303, 114793, 84181, 24665, 65339, 21219, 15771, 9024, 27642, 65289, 27619, 23323, 83482, 26317, 101368, 39249, 46715, 74960, 15755, 83078, 29424, 12577, 21069, 39603, 114690, 72654, 4582, 34238, 114313, 15897, 15474, 107188, 91564, 5268, 14709, 66512, 7135, 5862, 95295, 18375, 56258, 8960, 28118, 114254, 79302, 29444, 32124, 35283, 1517, 28392, 40125, 28363, 20441, 117648, 114419, 3225, 54913, 4924, 34717, 82129, 39732, 64457, 62986, 116969, 126221, 17893, 8012, 126167, 22918, 67670, 41557, 25698, 14079, 27458, 84730, 27719, 104997, 2125, 18215, 22938, 1588, 92160, 100331, 28075, 7695, 15218, 7724, 35636, 112318, 111458, 94, 103473, 99658, 81682, 28074, 54922, 66342, 101945, 30337, 10813, 129495, 18937, 67906, 21410, 27297, 6148, 10461, 129017, 114251, 792, 3316, 2854, 32863, 38326, 15399, 61338, 2147, 2351, 67899, 76121, 72661, 38946, 82417, 15710, 98252, 19701, 1920, 12168, 47549, 14009, 3206, 49411, 3731, 68546, 94108, 119040, 106776, 5417, 59590, 1970, 5387, 5997, 15049, 3507, 76246, 32118, 341, 84768, 42582, 42831, 1959, 23857, 84532, 94932, 128464, 112428, 84187, 99446, 118496, 14984, 23373, 31461, 42456, 96019, 64469, 2864, 49859, 31508, 20445, 62713, 124985, 47947, 12327, 21879, 82241, 107194, 13314, 54924, 3999, 12971, 6038, 9874, 68032, 109095, 72653, 25818, 96016, 54013, 19369, 45272, 112003, 8631, 34705, 6814, 10363, 55566, 85926, 51602, 6640, 91369, 80172, 28384, 49414, 63665, 115777, 49595, 30572, 112559, 22135, 51434, 46942, 120844, 73, 10524, 20425, 61179, 71381, 88975, 84864, 98977, 107829, 64062, 36314, 73898, 14395, 116108, 1332, 91703, 81548, 51624, 22249, 115397, 67159, 20298, 118121, 67900, 4132, 9521, 17381, 49904, 37638, 83223, 18151, 44049, 18529, 116487, 11252, 67586, 80177, 118776, 37561, 25839, 3802, 61314, 44737, 84666, 2219, 47925, 14022, 85446, 3463, 83241, 10496, 108938, 25286, 14190, 58864, 76057, 382, 28401, 28062, 1004, 2114, 44313, 80951, 21662, 1487, 112555, 38937, 31324, 115987, 42260, 3285, 12466, 128451, 124982, 34803, 84990, 125395, 28389, 66275, 67749, 84643, 80953, 98749, 74998, 42076, 19696, 12329, 67389, 31462, 798, 89136, 67897, 91366, 24163, 3197, 18393, 39611, 73933, 83256, 46078, 117406, 11420, 10376, 12347, 6240, 11227, 1662, 10349, 84997, 99421, 68813, 1009, 115380, 24523, 11215, 8374, 108932, 23759, 33455, 36127, 28796, 1249, 41559, 8806, 4443, 52144, 33088, 11648, 49704, 32182, 4244, 90408, 87675, 65888, 107581, 115539, 4117, 56257, 12283, 52356, 85922, 84524, 18786, 102097, 909, 107388, 55614, 40621, 1971, 101516, 22076, 49415, 1596, 96073, 26952, 12839, 49900, 2553, 15700, 75395, 28402, 120566, 30157, 80183, 29430, 97337, 52354, 92180, 76338, 1001, 39181, 90863, 126210, 10033, 63285, 41595, 7744, 75990, 116877, 6795, 84189, 119047, 37384, 23217, 33356, 12351, 4726, 124876, 74951, 16806, 86325, 11222, 76353, 5371, 84732, 5963, 27037, 10468, 66154, 98298, 5420, 51439, 3495, 10453, 40504, 127348, 437, 45323, 12637, 55671, 66448, 67891, 115554, 118767, 54421, 85652, 15062, 10495, 93515, 89245, 62621, 87848, 2575, 114120, 1901, 86600, 107376, 22385, 64926, 23701, 31457, 55552, 33886, 45673, 67059, 80289, 98855, 97513, 107192, 27724, 96897, 95285, 8485, 4758, 41596, 101361, 92470, 31439, 76850, 104893, 43720, 1892, 113180, 50749, 32141, 122181, 75882, 27533, 97958, 55598, 3243, 106780, 127750, 119311, 62068, 33160, 939, 108491, 98171, 111213, 38936, 3476, 97489, 73870, 9124, 22908, 4087, 63943, 59752, 55576, 62069, 98757, 83077, 87868, 6818, 15918, 24046, 17452, 44947, 46202, 44814, 74241, 76344, 21975, 69261, 103386, 32728, 23109, 3658, 84745, 81549, 34245, 56068, 53370, 3797, 995, 2148, 1857, 8248, 753, 66, 9106, 22970, 948, 2279, 2896, 6185, 873, 11261, 1272, 1317, 439, 6804, 1698, 442, 4113, 3107, 8255, 25267, 2919, 1922, 424, 1341, 5558, 7073, 9010, 13405, 41289, 10622, 13813, 11674, 5138, 31598, 10314, 158, 20299, 20851, 6638, 5125, 5550, 3145, 3899, 8051, 6085, 6165, 10838, 10336, 7461, 41772, 18131, 5471, 7420, 45646, 7790, 4217, 1822, 7439, 2201, 1316, 11007, 13732, 7235, 10183, 23347, 12117, 5972, 64761, 5635, 24911, 31345, 8015, 22301, 176, 25185, 1609, 11643, 7525, 5725, 9430, 2639, 17232, 14029, 27292, 5440, 26220, 6641, 48658, 1629, 3791, 655, 70, 8148, 675, 7676, 2294, 29721, 4350, 8633, 38927, 11783, 26993, 19610, 22, 27732, 47970, 72084, 14462, 606, 9872, 1176, 35233, 24909, 5357, 29038, 3584, 19200, 45401, 34318, 3903, 1926, 7944, 3413, 1775, 5442, 3472, 8713, 23248, 1371, 2020, 4474, 29110, 12364, 3102, 11152, 198, 4704, 5292, 1310, 4905, 16867, 6113, 26497, 26213, 16425, 500, 38913, 25797, 6344, 25742, 6788, 4609, 30439, 20996, 377, 3560, 4333, 41780, 33631, 4466, 6233, 6203, 53097, 5224, 30357, 23496, 1615, 13948, 51152, 5964, 36492, 27287, 2072, 8586, 5356, 807, 9748, 46195, 75858, 13060, 17616, 4710, 5343, 8587, 52214, 1298, 28407, 4916, 5369, 1499, 433, 7261, 12386, 791, 18919, 23035, 4922, 5614, 8092, 27536, 7798, 17647, 152, 2858, 13144, 4721, 3420, 7280, 9043, 200, 1750, 6412, 329, 3308, 8245, 37230, 6773, 57012, 7346, 17935, 47546, 5307, 2476, 7550, 20293, 9086, 53659, 2567, 43750, 1201, 54378, 39168, 919, 17412, 5449, 11112, 8938, 681, 14243, 2047, 17695, 64035, 43039, 65190, 5990, 42417, 16790, 15704, 11731, 25307, 38578, 13310, 21897, 44070, 40175, 3232, 18773, 2386, 716, 2217, 430, 10275, 58337, 21024, 23133, 1595, 589, 19099, 64779, 2608, 958, 83886, 24300, 38904, 17147, 2099, 6655, 2208, 4001, 19933, 669, 8040, 18278, 3905, 5994, 3722, 24347, 6183, 7265, 1528, 625, 7656, 2259, 28350, 4083, 1039, 38809, 3578, 50055, 16768, 4568, 1907, 7342, 20487, 3350, 33414, 17492, 4184, 5842, 10609, 17162, 4858, 17100, 1113, 8283, 5282, 2540, 6316, 4051, 3910, 2918, 205, 476, 30928, 2070, 58535, 31013, 17535, 32916, 6230, 22633, 82108, 18994, 11468, 37065, 29605, 78382, 8450, 16049, 10472, 110814, 33350, 445, 16825, 5175, 826, 9493, 1032, 66611, 13271, 5776, 11020, 3426, 66861, 846, 23004, 5942, 42616, 39180, 38538, 16717, 14175, 44619, 1608, 7260, 7016, 36374, 2481, 7541, 194, 34822, 38817, 29261, 4054, 23447, 12628, 49584, 5876, 64693, 13582, 12139, 5261, 3506, 12842, 10305, 16596, 12455, 40611, 22052, 36431, 51846, 59394, 53756, 44905, 7888, 29708, 22643, 34395, 19751, 15381, 23920, 4962, 6823, 44125, 1693, 21454, 2448, 38082, 28397, 81520, 35546, 6252, 29422, 63892, 2445, 2909, 23213, 32133, 23955, 84931, 11383, 112374, 4008, 29716, 53313, 31923, 12444, 74176, 8690, 3460, 22545, 13630, 55324, 14975, 49361, 72617, 56161, 8177, 1977, 20201, 77382, 6162, 2670, 56015, 55883, 57756, 60234, 61, 16247, 7894], [893, 13587, 8043, 4010, 940, 16664, 28395, 9274, 4026, 11680, 1727, 1013, 15746, 17479, 13628, 1443, 2837, 26811, 45870, 43753, 11072, 36976, 1783, 10801, 11467, 22263, 4282, 26273, 315, 9367, 4779, 15126, 15879, 381, 6290, 31385, 56633, 11826, 809, 34592, 29556, 47998, 7024, 3835, 555, 44795, 23355, 13425, 28128, 61299, 22651, 9346, 3955, 13378, 374, 18812, 697, 21942, 21802, 12569, 27047, 17042, 1893, 1253, 367, 1707, 485, 20583, 11899, 1958, 7138, 3044, 4230, 4417, 4894, 21702, 20355, 6767, 2107, 22527, 20586, 16392, 857, 6195, 1433, 8651, 5352, 1154, 11831, 15473, 8928, 29264, 4210, 18776, 4999, 19260, 13516, 8824, 645, 20410, 2731, 833, 1320, 22780, 5252, 8934, 20005, 33275, 4616, 25217, 10398, 29139, 14815, 956, 3237, 9190, 2169, 5837, 6063, 3452, 632, 2119, 6197, 11282, 2824, 2113, 24841, 36557, 14328, 1311, 4108, 8348, 2178, 6019, 12281, 9539, 6861, 8693, 23090, 273, 2577, 443, 4836, 977, 25409, 42943, 48186, 47554, 644, 33865, 29689, 58650, 15759, 11726, 22400, 7248, 12157, 10733, 69849, 47575, 42217, 4441, 37417, 19618, 386, 64237, 29503, 54422, 7561, 21408, 30051, 16438, 12032, 67981, 91707, 9079, 2502, 18225, 13268, 117336, 7848, 930, 70073, 28393, 65310, 23364, 73706, 5854, 49359, 57988, 47464, 30922, 75581, 45890, 4892, 38581, 33565, 30883, 54485, 122192, 18875, 72834, 70170, 22246, 82590, 14631, 3895, 44421, 45021, 1192, 2313, 21585, 1400, 5294, 8678, 53484, 65980, 28114, 64033, 766, 67489, 56234, 84488, 5151, 28364, 33821, 20608, 27882, 4782, 29127, 67723, 22445, 12129, 29700, 25456, 98883, 77493, 105497, 88415, 103373, 11734, 28177, 32346, 65907, 17517, 86967, 24612, 36909, 18319, 17326, 72815, 23644, 7253, 1194, 37547, 5618, 11689, 110817, 10652, 23684, 668, 4369, 6041, 22531, 13731, 7470, 51481, 3699, 6870, 14143, 12557, 51774, 5467, 2261, 47798, 2405, 796, 61294, 91027, 38117, 98237, 8605, 55079, 1404, 29642, 11219, 2275, 44159, 122, 29299, 8753, 27508, 20110, 11260, 48451, 9324, 33907, 6116, 40627, 29292, 14850, 24205, 4856, 10369, 62467, 27861, 69392, 31764, 98982, 18363, 99768, 20882, 26169, 23694, 3394, 4996, 8676, 17369, 52229, 2541, 10105, 58665, 87588, 18132, 33970, 93585, 6786, 21358, 5934, 16474, 51676, 90075, 64038, 22012, 1682, 78400, 270, 35082, 16754, 86948, 34450, 1093, 70179, 7980, 65798, 2037, 12661, 3702, 6775, 10059, 22794, 57232, 33830, 8646, 69399, 11647, 24506, 44291, 14771, 22878, 68334, 2850, 15288, 17514, 39377, 22657, 40827, 33828, 18873, 3985, 9741, 9772, 6783, 28939, 18864, 14457, 11874, 82020, 96746, 83437, 17896, 11066, 4025, 9121, 3290, 40785, 54934, 2683, 35334, 29697, 86407, 43338, 47277, 41050, 16822, 59097, 17431, 11151, 31435, 64336, 23798, 17509, 69579, 42619, 1299, 12784, 101225, 27547, 59838, 16419, 28974, 16968, 4029, 12613, 94632, 31906, 57532, 38828, 143, 83889, 7357, 22487, 39161, 4700, 17629, 2454, 1984, 4621, 11906, 3969, 10104, 4353, 16522, 8501, 8801, 8588, 4498, 2707, 8341, 261, 6642, 11649, 7555, 10778, 311, 5481, 20635, 309, 34574, 33595, 1879, 16164, 1523, 10471, 13135, 1325, 7070, 11145, 4737, 10546, 24473, 3983, 1625, 56333, 4961, 9935, 18386, 6416, 9118, 18653, 30346, 22291, 2637, 2999, 37604, 3827, 4389, 12242, 14844, 6241, 10233, 7542, 735, 67, 23512, 14208, 2056, 370, 29863, 2506, 2467, 466, 1781, 4387, 866, 2298, 1763, 33902, 2847, 55594, 12013, 5191, 25693, 26710, 33888, 15419, 5409, 2835, 38513, 6096, 24176, 13638, 36767, 3494, 932, 3122, 7551, 2104, 1112, 20742, 2368, 14360, 25226, 27994, 5744, 2397, 4578, 6001, 591, 50076, 4106, 62034, 13683, 30014, 17605, 22616, 10837, 1931, 2088, 12791, 6616, 27389, 3926, 2708, 11118, 44444, 10339, 7302, 49120, 19555, 26387, 7373, 1144, 9036, 1181, 1511, 6086, 3278, 10063, 40682, 3428, 6866, 6, 16209, 9761, 10638, 2478, 3774, 73550, 5272, 84190, 1346, 10463, 4085, 26802, 8371, 10908, 66829, 21397, 256, 28791, 17511, 11745, 54906, 6008, 26288, 49696, 72091, 18119, 4020, 5192, 3717, 85127, 51843, 7291, 21432, 25915, 68372, 60549, 1535, 10497, 1390, 41041, 24791, 20192, 19954, 11063, 53379, 8823, 20253, 7986, 1120, 20199, 25842, 32378, 17625, 17060, 23247, 27499, 59557, 13704, 48335, 17499, 56857, 25766, 91566, 14960, 53309, 85034, 60768, 23526, 18780, 17342, 52849, 5066, 23819, 14795, 40902, 8014, 78485, 66636, 22903, 39241, 18315, 39371, 3095, 42938, 3165, 6007, 36114, 21221, 5144, 46365, 19507, 55433, 94374, 31171, 16763, 49801, 11392, 19395, 89392, 2839, 25363, 6078, 15496, 22776, 17271, 30479, 53442, 61342, 2124, 4041, 35497, 29601, 3784, 96255, 8288, 14031, 7309, 1233, 265, 5594, 682, 11819, 10835, 3843, 4599, 280, 2120, 7161, 5832, 11967, 5314, 23503, 5486, 10532, 6009, 6368, 31720, 17372, 10836, 790, 1261, 3938, 1031, 4337, 12149, 24474, 8672, 23547, 12393, 20549, 10476, 14839, 7478, 2813, 8515, 1717, 24546, 2235, 4747, 5246, 11274, 13283, 8582, 3790, 6068, 20515, 11900, 149, 1305, 8056, 4017, 2116, 6762, 2300, 3324, 13313, 84427, 51664, 10362, 678, 19396, 9419, 23508, 11813, 11132, 28518, 17623, 12785, 40075, 8600, 2728, 2979, 23795, 35703, 3520, 4985, 15869, 35307, 11889, 2082, 34496, 3115, 3140, 11740, 5459, 73814, 11478, 19254, 12829, 32361, 27444, 14763, 1876, 13322, 14030, 17419, 52705, 4358, 19297, 45521, 11148, 17672, 849, 14255, 1953, 18741, 34595, 1300, 66688, 11469, 11157, 11376, 72828, 2193, 1951, 13026, 14052, 3878, 685, 609, 268, 3343, 1405, 8829, 12787, 35363, 18302, 3262, 8219, 59393, 11451, 5939, 610, 3836, 10093, 10450, 20438, 925, 436, 812, 18249, 9881, 19184, 27196, 72736, 10739, 14700, 9486, 50331, 31254, 15991, 12263, 13300, 53521, 2129, 14967, 47689, 27286, 52207, 6998, 14283, 13547, 15127, 34751, 13054, 37665, 3442, 656, 4991, 38633, 17363, 886, 7310, 972, 5367, 5605, 615, 3959, 12441, 5930, 2035, 12091, 634, 11394, 1067, 8920, 1324, 196, 7230, 455, 17124, 23733, 10345, 11665, 2734, 19614, 4238, 8211, 1394, 2374, 15009, 12733, 13482, 26363, 845, 9176, 11302, 31599, 8779, 4385, 10111, 20718, 3592, 6235, 95, 8194, 7453, 13983, 29311, 3277, 10603, 8136, 788, 2911, 15731, 6401, 6665, 14458, 10887, 26466, 11733, 18895, 28223, 22243, 4998, 12741, 18071, 4678, 26078, 2487, 4800, 20934, 318, 6192, 3752, 18915, 12167, 4343, 4906, 7303, 2962, 4610, 157, 5338, 5243, 16947, 7634, 6625, 10743, 21603, 6411, 4940, 5922, 5306, 12596, 3071, 1393, 13404, 20216, 2544, 10549, 3918, 1406, 2288, 6675, 4861, 14040, 20331, 5926, 12646, 4234, 5895, 18059, 10756, 29288, 1180, 24545, 23899, 503, 8271, 282, 213, 1206, 2712, 8593, 1157, 5850, 12304, 4373, 1173, 3574, 17953, 5579, 505, 125, 3272, 7254, 692, 6862, 514, 8345, 3331, 9686, 4377, 4375, 337, 12554, 1363, 10, 36691, 64381, 120058, 74954, 79445, 16696, 63694, 92935, 46244, 28043, 73403, 22528, 44437, 97529, 3582, 60108, 46760, 44693, 60709, 957, 110117, 62181, 89919, 109898, 73399, 74527, 89164, 81361, 46761, 18958, 22313, 26807, 38917, 85232, 105736, 46037, 59928, 18853, 15944, 119859, 9763, 12811, 14757, 70383, 2471, 86143, 884, 31591, 22094, 3867, 42479, 127828, 92337, 6947, 19596, 86638, 40612, 60647, 35620, 73771, 124915, 15891, 39139, 62297, 72606, 36117, 27703, 81830, 13449, 109990, 2532, 34824, 29507, 60345, 105733, 30473, 53698, 77627, 123971, 13272, 64443, 102102, 128033, 53817, 102691, 689, 89744, 63696, 18095, 23843, 46933, 21239, 19175, 2390, 45936, 26167, 38058, 44458, 27890, 13230, 6082, 26165, 53630, 60630, 6253, 14387, 23803, 23675, 81248, 16435, 73398, 99676, 31610, 33443, 4325, 21551, 85903, 1591, 19592, 71422, 574, 32439, 1327, 73468, 26168, 40173, 46372, 53888, 58987, 3295, 110131, 4318, 28358, 22349, 67052, 8595, 73605, 20638, 44483, 127711, 3083, 584, 90152, 69031, 120525, 26136, 62055, 786, 125579, 110390, 63677, 24751, 56743, 23690, 54447, 85082, 113460, 81654, 115729, 21642, 34442, 95707, 6758, 8671, 58960, 56282, 109987, 28930, 118172, 113211, 31648, 12588, 102108, 10324, 86639, 94840, 2561, 58981, 74152, 1445, 5315, 43073, 51338, 82641, 24061, 97472, 37846, 28270, 7115, 83052, 46337, 6953, 7006, 6066, 222, 42607, 39140, 6878, 9061, 14990, 50896, 1288, 577, 23212, 12549, 37246, 126, 20422, 60761, 39224, 102506, 9342, 3136, 31507, 69257, 90867, 86130, 53718, 92922, 10962, 7350, 34843, 114350, 114343, 31502, 4502, 6342, 32486, 85735, 112464, 19586, 13103, 56146, 128505, 1054, 58066, 16777, 31070, 56779, 79824, 73596, 32347, 118777, 7934, 163, 67351, 32117, 25778, 47041, 6794, 78547, 114785, 11163, 5376, 74228, 65036, 109977, 120336, 23377, 77804, 84773, 110071, 50753, 13528, 56417, 112461, 3978, 109901, 74219, 81890, 1871, 39435, 45640, 32437, 18514, 102889, 5381, 18258, 50524, 23723, 70951, 64044, 79322, 113446, 58662, 56117, 117326, 60346, 11160, 25393, 52093, 30037, 4809, 43976, 31658, 128926, 108739, 11126, 14712, 3860, 1365, 37866, 42875, 86395, 101048, 86069, 83114, 55807, 427, 123718, 110070, 113209, 67313, 59932, 59929, 4867, 103969, 6267, 2659, 3375, 80519, 33889, 21792, 33901, 63373, 99096, 64390, 39203, 49877, 39634, 23776, 111310, 83062, 10332, 59936, 113466, 29604, 33903, 79431, 18429, 39908, 1334, 39226, 74165, 43955, 31512, 5370, 60548, 23134, 18965, 18963, 5310, 37893, 100707, 53577, 105747, 50679, 27516, 5835, 19776, 48255, 9352, 58642, 30027, 113440, 49895, 95760, 40538, 104310, 70099, 122583, 2078, 5408, 86158, 38242, 57556, 95717, 128539, 57069, 70386, 87736, 117346, 9706, 5918, 17760, 3204, 6318, 17258, 32103, 31634, 47231, 2229, 34578, 120327, 698, 22603, 3011, 2885, 70429, 8539, 51572, 3701, 79176, 221, 7299, 34017, 32393, 1914, 42063, 93944, 43412, 22895, 78466, 28369, 10811, 35881, 120326, 519, 99948, 55896, 59774, 81454, 27974, 42839, 86023, 56149, 927, 13699, 60700, 15329, 101827, 66254, 101902, 74321, 72816, 4315, 83061, 60098, 57679, 99951, 100702, 3470, 66866, 5247, 112458, 122303, 23658, 4650, 101050, 15070, 100723, 113213, 55805, 105498, 75739, 123622, 127696, 1236, 54003, 17518, 31665, 27772, 53975, 15529, 12912, 113214, 122666, 64394, 76454, 76926, 17338, 29302, 86653, 38614, 125428, 72782, 23242, 1590, 53512, 66440, 98090, 15714, 8267, 14896, 23796, 22764, 15530, 103422, 17612, 2442, 23953, 52839, 71926, 43070, 4501, 6993, 58180, 88257, 47227, 85728, 34847, 14375, 73384, 85905, 124963, 36342, 59661, 3577, 108034, 72981, 108754, 14213, 629, 93184, 8778, 79227, 17520, 55008, 21656, 17311, 2915, 56878, 72813, 53127, 38269, 22774, 10347, 28944, 7300, 41924, 14713, 83111, 9989, 7273, 15455, 30496, 8746, 95736, 50071, 84777, 91979, 79313, 43213, 48352, 61450, 71419, 32024, 9751, 16762, 55985, 79810, 21922, 70845, 8610, 34577, 21236, 8578, 10054, 32445, 16449, 90868, 19767, 8638, 99690, 89771, 13393, 10488, 89978, 122317, 26616, 3576, 28344, 31976, 48114, 77446, 933, 31510, 122305, 10534, 33895, 43166, 9982, 90149, 32724, 61469, 39234, 125436, 4361, 7015, 66852, 70155, 26730, 23939, 335, 110129, 71719, 104852, 72985, 6777, 121574, 33655, 51438, 33181, 21794, 42612, 136, 8259, 123727, 26409, 68018, 56296, 16820, 1891, 19591, 3743, 53126, 11705, 31675, 41489, 32401, 6168, 6854, 24424, 117591, 46166, 85741, 105085, 6841, 23597], [2550, 9350, 3040, 47664, 113217, 76545, 41991, 79822, 12660, 262, 6035, 11161, 20232, 23024, 88656, 7234, 66238, 99826, 84801, 113472, 449, 109992, 93168, 104309, 6073, 17309, 20394, 102103, 127703, 78960, 53964, 81290, 83060, 10080, 67150, 96011, 16949, 13723, 66662, 31614, 10266, 7005, 11048, 71303, 9828, 32355, 99847, 123860, 55005, 99742, 20950, 11064, 83735, 78436, 115299, 74008, 73669, 12831, 127804, 60544, 34006, 90875, 79798, 28500, 16446, 110731, 34832, 27892, 118798, 120523, 2183, 16221, 103064, 71421, 3247, 72591, 31951, 90096, 16761, 4701, 60631, 6083, 2658, 3216, 42482, 1604, 89166, 8947, 38433, 38663, 68340, 4987, 7000, 14699, 19929, 8167, 72997, 13925, 81819, 68061, 18890, 6058, 3186, 128554, 102886, 79419, 27180, 954, 2420, 1477, 4965, 4062, 4413, 1508, 372, 17480, 3989, 1841, 24651, 20935, 5697, 448, 5296, 2248, 9817, 482, 2566, 11316, 141, 7146, 78, 7971, 16670, 2611, 653, 2598, 16104, 23156, 2682, 3940, 6077, 5199, 18189, 8816, 2589, 11812, 827, 2360, 16060, 313, 1121, 5341, 10481, 5102, 11415, 3156, 21223, 4725, 6776, 15703, 39400, 18891, 9059, 2992, 16414, 710, 1979, 896, 3310, 595, 13372, 225, 16444, 15028, 36697, 32345, 7363, 10094, 2501, 2050, 661, 17717, 3194, 129, 4857, 15293, 7999, 2878, 13058, 10048, 6333, 42948, 5924, 2175, 5736, 5778, 5772, 7571, 696, 13175, 6664, 4820, 22394, 12820, 7529, 35180, 4299, 1073, 2507, 3491, 17423, 2074, 13659, 30821, 12783, 9272, 953, 4251, 19697, 799, 12651, 64, 18835, 808, 5078, 14511, 6123, 5202, 1044, 16194, 1356, 2417, 5300, 2951, 13877, 11804, 9970, 14251, 1312, 15117, 464, 3031, 4421, 91, 585, 1622, 23344, 30210, 16362, 58039, 2590, 8125, 10646, 11526, 12622, 4787, 3089, 27501, 913, 10602, 18865, 31208, 36689, 23148, 14752, 21440, 25869, 31438, 27436, 19546, 3291, 23816, 53671, 15898, 12278, 26781, 9031, 16377, 73154, 9658, 1910, 28146, 16066, 4925, 3052, 613, 107, 3820, 5226, 35619, 16, 3444, 7340, 6789, 5453, 918, 11045, 1315, 9735, 1115, 3757, 5753, 2741, 2865, 3498, 8918, 1153, 15425, 8822, 4077, 10995, 3565, 64025, 20717, 23968, 16766, 15988, 21996, 9996, 8071, 11001, 1889, 22674, 3876, 8774, 10077, 2914, 288, 20003, 6021, 36617, 16737, 14946, 14443, 5297, 10814, 33433, 10166, 6868, 8921, 2466, 16821, 651, 39778, 43670, 45215, 5248, 14019, 17446, 5883, 30611, 71082, 19160, 18452, 76, 8453, 52857, 894, 2067, 4423, 461, 781, 5641, 14915, 1985, 34627, 10975, 1399, 2073, 6597, 11735, 3839, 378, 33770, 4424, 27034, 18399, 23802, 13548, 1875, 654, 8019, 17557, 60802, 5498, 185, 1839, 9312, 699, 2176, 14825, 8811, 2525, 8266, 8335, 16853, 35927, 676, 22413, 10533, 29250, 1234, 31672, 9069, 2603, 8795, 3380, 5648, 15041, 5250, 23149, 19851, 15634, 4035, 10291, 12145, 24400, 21293, 10854, 5597, 35389, 70938, 2529, 88595, 899, 52488, 10346, 6825, 11069, 11270, 38496, 444, 83195, 12806, 18155, 31786, 61466, 113521, 5340, 41976, 12284, 22541, 18271, 32313, 83319, 22416, 7359, 48333, 85827, 26156, 20218, 2319, 1247, 18957, 27767, 64423, 5368, 4430, 6831, 54553, 3654, 51552, 10883, 6829, 8519, 64132, 57839, 7685, 39429, 11503, 37279, 5424, 9066, 63363, 26146, 53958, 31767, 43601, 4993, 22617, 890, 52973, 28464, 9889, 47686, 19615, 34387, 19277, 91356, 20235, 2557, 5999, 22640, 6835, 892, 13550, 5929, 6842, 10772, 12693, 75244, 18673, 24495, 23284, 11871, 2177, 27506, 7691, 11003, 688, 5658, 1720, 3384, 3697, 145, 607, 1376, 1338, 2317, 693, 544, 2089, 868, 67775, 2635, 7142, 2439, 29206, 5546, 452, 16069, 11411, 31780, 2661, 13524, 10133, 20615, 4684, 9759, 239, 7939, 31383, 17665, 5487, 9194, 4414, 6094, 543, 67242, 23251, 1291, 33601, 38843, 13706, 1415, 9451, 49894, 78396, 4075, 2059, 9042, 8339, 56150, 9765, 10984, 16609, 65656, 13651, 36086, 45426, 4364, 21619, 20431, 23501, 13288, 8639, 1303, 3561, 5213, 31145, 10113, 888, 18715, 1413, 1162, 8737, 1784, 281, 7924, 10994, 9037, 18427, 3386, 762, 9132, 35188, 48343, 48716, 11499, 9285, 17483, 6833, 18555, 30951, 2528, 795, 14027, 1193, 4043, 3735, 19714, 1981, 7406, 19079, 13694, 834, 8770, 19730, 726, 1972, 50617, 15032, 3055, 5562, 823, 2325, 15244, 29471, 20161, 4422, 3434, 2416, 5353, 12864, 422, 898, 2126, 2081, 2613, 366, 4360, 26378, 23957, 21899, 35723, 292, 51583, 8597, 33436, 2836, 3583, 12044, 9068, 53368, 1738, 16959, 2996, 89741, 35339, 69144, 54981, 47581, 2055, 18177, 71814, 22275, 23216, 40916, 45446, 6257, 27666, 21423, 581, 21483, 21929, 8768, 27201, 23146, 22621, 8386, 2464, 28760, 31830, 84800, 12875, 5824, 43422, 26125, 22271, 65739, 217, 9733, 7083, 1963, 18190, 5230, 12150, 30973, 1289, 872, 89619, 35924, 44296, 460, 17334, 33935, 8298, 77491, 43050, 13437, 63155, 34804, 3374, 18412, 25444, 8018, 10030, 4479, 22790, 19943, 63564, 15468, 70569, 5937, 12728, 100700, 12162, 20335, 17255, 6772, 11909, 77150, 8173, 10271, 84760, 32435, 69393, 14561, 26296, 56789, 4316, 10893, 13703, 79408, 841, 9129, 57542, 20563, 1600, 9120, 10921, 27922, 26709, 52438, 85145, 4370, 47458, 32934, 99160, 4499, 6076, 40095, 10877, 800, 2605, 48733, 14007, 15071, 10846, 67942, 97729, 21379, 26610, 5931, 41600, 19011, 6819, 80642, 52863, 14374, 22254, 26225, 35559, 5249, 52090, 101139, 64445, 5014, 18884, 43714, 5065, 4992, 4129, 9559, 2174, 5234, 27174, 114843, 10516, 4005, 10740, 30510, 22302, 12794, 72359, 43137, 23942, 57500, 2564, 15428, 115254, 8075, 24154, 73401, 690, 117335, 115535, 15166, 4780, 7272, 7358, 15536, 62113, 26066, 45733, 6137, 21813, 7256, 3226, 104381, 27175, 5547, 6681, 14741, 76952, 55801, 13063, 6876, 20414, 85125, 29243, 70911, 1007, 71766, 20856, 79451, 23075, 657, 25994, 35443, 49517, 21378, 1251, 1199, 8804, 38916, 32405, 12867, 14058, 73394, 44762, 77797, 21200, 1527, 46211, 150, 26, 438, 1301, 9026, 17256, 3850, 9714, 1607, 20919, 4045, 1598, 16404, 2973, 7961, 11144, 10608, 8581, 10279, 5465, 895, 9239, 283, 3322, 1742, 2663, 14898, 12880, 26180, 1427, 6408, 4213, 1630, 2479, 2461, 14655, 14439, 20692, 1733, 1594, 11189, 3708, 29946, 9729, 11046, 11701, 19798, 4772, 18897, 24083, 7007, 1514, 6332, 706, 535, 4688, 182, 10781, 10729, 4368, 9060, 521, 21994, 8852, 216, 1801, 17989, 21217, 6036, 11070, 9100, 13257, 3478, 12045, 17346, 8711, 11785, 3542, 18009, 14791, 37572, 12898, 11459, 700, 159, 2172, 2892, 3798, 11102, 3084, 14934, 10765, 4717, 87, 30018, 31061, 26839, 15728, 203, 27871, 3608, 4078, 16318, 13473, 1472, 771, 2318, 84, 15011, 31194, 11412, 2121, 3213, 6941, 3230, 28473, 21938, 5846, 7021, 32026, 72140, 9510, 71857, 4881, 31673, 56626, 665, 25358, 12890, 31824, 34428, 18868, 18208, 10978, 5363, 24479, 11715, 10293, 36954, 36007, 18138, 65113, 13806, 30438, 10849, 50902, 12877, 51902, 32556, 14971, 63369, 714, 56900, 57020, 2702, 16760, 29282, 20045, 33097, 5816, 6756, 40805, 775, 12148, 2265, 46640, 2212, 16951, 73899, 11056, 633, 7004, 11106, 31609, 47011, 33972, 14951, 9013, 3098, 23131, 25340, 7025, 3015, 4911, 14625, 9343, 8443, 10732, 4356, 35487, 32829, 34478, 72996, 12724, 5882, 40533, 27294, 6851, 18502, 48565, 80902, 56844, 27442, 23844, 30502, 63418, 74746, 56207, 3838, 45548, 2167, 19972, 6585, 12, 9241, 855, 2894, 3138, 1467, 3389, 22004, 1796, 5671, 6834, 214, 42405, 9675, 271, 9464, 8129, 27131, 2457, 28514, 2641, 6112, 7659, 20557, 2701, 12006, 2084, 2237, 7859, 759, 9643, 1126, 14615, 10937, 1227, 12899, 2585, 21524, 2367, 16891, 3200, 23223, 27252, 2636, 33260, 40033, 56055, 4165, 6460, 17674, 941, 36575, 18718, 67348, 23790, 39446, 21419, 882, 1042, 14515, 38353, 8563, 56099, 23200, 23139, 5799, 11788, 25429, 8207, 37424, 138, 10491, 22422, 12141, 37303, 5107, 52360, 47461, 15479, 22347, 2493, 7341, 858, 1259, 30920, 15315, 30176, 13324, 3687, 21443, 36394, 459, 4366, 17015, 223, 10151, 5218, 12853, 7945, 2690, 3834, 4118, 10810, 73047, 62516, 12002, 674, 60754, 34079, 49609, 21382, 40407, 35793, 31624, 550, 32438, 26863, 8134, 7214, 13435, 47186, 16062, 22350, 3537, 17792, 34244, 2662, 16211, 13633, 4897, 45945, 36773, 5717, 20340, 4665, 22000, 8621, 1122, 4740, 1877, 7514, 1066, 16541, 4614, 4689, 8489, 4330, 3690, 1091, 4671, 23569, 10031, 18644, 22207, 4164, 13692, 7927, 6236, 20906, 35808, 9884, 593, 691, 3466, 539, 5185, 3051, 3700, 2440, 9551, 1190, 5969, 2316, 34010, 5724, 7029, 4699, 3666, 601, 13529, 25167, 3822, 5431, 2515, 2463, 8699, 14122, 23506, 1037, 29388, 37825, 3297, 12049, 11721, 8700, 646, 8500, 4175, 23653, 32492, 25773, 6120, 2146, 4111, 481, 794, 1368, 18113, 8320, 2086, 11336, 5737, 586, 11724, 19737, 4308, 4907, 2548, 2863, 13897, 22552, 31551, 14317, 6144, 22921, 10434, 1697, 10581, 7785, 1214, 18645, 1465, 45419, 36957, 7520, 7269, 15472, 18083, 5348, 1735, 5001, 13315, 8118, 557, 999, 9849, 24569, 1685, 21529, 6134, 212, 3001, 4137, 13111, 1867, 2207, 4681, 2677, 5564, 33462, 124401, 210, 73113, 78536, 101524, 73041, 70905, 14459, 106716, 6231, 17035, 8398, 28176, 62371, 28126, 32456, 4365, 54780, 22011, 84065, 21944, 65635, 65903, 35693, 31817, 113030, 52141, 106142, 121961, 8702, 15038, 854, 12771, 53355, 7220, 9885, 85830, 876, 67405, 7763, 86965, 86637, 26253, 18299, 54915, 113512, 57929, 677, 58021, 85495, 105726, 27355, 32005, 32460, 105469, 53354, 106738, 3028, 114931, 58225, 5750, 54531, 90853, 48848, 10343, 13450, 1024, 50001, 103492, 70697, 4340, 852, 50642, 64743, 94375, 31311, 24080, 118568, 75861, 77110, 85204, 42335, 115722, 5553, 106693, 11444, 6466, 38838, 59347, 53954, 71788, 40601, 89004, 31826, 45507, 51756, 24964, 74528, 15589, 35647, 120236, 72607, 35791, 56764, 24490, 110270, 71211, 10973, 13940, 56082, 24084, 22319, 3538, 27435, 51559, 19242, 53633, 59471, 32805, 129591, 64103, 3190, 53504, 35472, 63423, 887, 81409, 57930, 118573, 11439, 28060, 15138, 129742, 8939, 48556, 40605, 18513, 1683, 71769, 94381, 16190, 15064, 31596, 105729, 8747, 14774, 120407, 22303, 69762, 47030, 19787, 79938, 1111, 74861, 14188, 75846, 45737, 52614, 18851, 40753, 48824, 59454, 13458, 25647, 51581, 7729, 12232, 2468, 72629, 37339, 28341, 58012, 84057, 2301, 6121, 984, 19795, 53760, 85673, 6674, 80836, 24831, 71773, 51591, 879, 54168, 16964, 73115, 74126, 54742, 2802, 12027, 100953, 110913, 80494, 11384, 26628, 8324, 27193, 114383, 33786, 26862, 70913, 79219, 128681, 52098, 51984, 38279, 15294, 100663, 27449, 19794, 57804, 31562, 100029, 50765, 76106, 1105, 126728, 67226, 38292, 81858, 39431, 11500, 35591, 46787, 454, 4355, 34876, 12654, 56598, 68272, 30476, 111583, 103281, 23395, 209, 865, 13436, 93117, 66463, 106739, 4967, 124, 20397, 124643, 24527, 12158, 72131, 6588, 37428, 83779, 86074, 42088, 124287, 6355, 1575, 42354, 22667, 122547, 1743, 113540, 19797, 32305, 115714, 57039, 22668, 7782, 15694, 110541, 67366, 101357, 57230, 90463, 67715, 110612, 3845, 12770, 7912, 64865, 14371, 120958, 38320, 19142, 74287, 60551, 71765, 95962, 58330, 54341, 106017, 28151, 20930], [75862, 66398, 39482, 78198, 5944, 88585, 24497, 128676, 2071, 38052, 28164, 66268, 88576, 96822, 6327, 114460, 14189, 97247, 5416, 92486, 1904, 3854, 33611, 16743, 6245, 89300, 110363, 115265, 10469, 33540, 64437, 65786, 34873, 60842, 44429, 118567, 10841, 94372, 37236, 8980, 12789, 5626, 34640, 119451, 34583, 40561, 1333, 15261, 64432, 96360, 119449, 5135, 43064, 20224, 24824, 52140, 62158, 105409, 2632, 19028, 3705, 121966, 54979, 123694, 24521, 37861, 18274, 67055, 41972, 49578, 54788, 90444, 25414, 71866, 33617, 13133, 51580, 17027, 60927, 12031, 49615, 113794, 22334, 4426, 10323, 1788, 85302, 47219, 18245, 77149, 122249, 51433, 19155, 107663, 120975, 78187, 110816, 80911, 86411, 7566, 107637, 85211, 85692, 69756, 99189, 120685, 75411, 65116, 1321, 70998, 14956, 30490, 74612, 63279, 58008, 14748, 26350, 5644, 22341, 56641, 44322, 37870, 40804, 49349, 45938, 88929, 41269, 68359, 56656, 88130, 13431, 22772, 8968, 119543, 18847, 108204, 41143, 46358, 2562, 4120, 124495, 24581, 41590, 53353, 58215, 2077, 91541, 45935, 22783, 127649, 9934, 28948, 119788, 56107, 122056, 80651, 9307, 3208, 65872, 21940, 30511, 5647, 27641, 46731, 34400, 21999, 103401, 45229, 57390, 18063, 44061, 16954, 7825, 118407, 70990, 9956, 103410, 40582, 57019, 76447, 84935, 28189, 53443, 21575, 81775, 101515, 129647, 93779, 93927, 67043, 16017, 62663, 5661, 13895, 14664, 30522, 35690, 8113, 110687, 129737, 64020, 128323, 27768, 72619, 87747, 7203, 31928, 57057, 28064, 82329, 2446, 51748, 94365, 1468, 72147, 18934, 51564, 129857, 38841, 110668, 24669, 53616, 57993, 3741, 475, 94488, 30509, 56203, 38863, 60753, 53230, 38358, 25648, 83629, 66465, 52199, 13954, 10489, 124157, 7690, 60939, 30919, 108457, 99188, 11472, 63830, 1020, 122747, 120231, 51191, 18469, 77207, 72151, 10539, 36556, 18852, 38209, 28166, 39594, 23356, 89702, 15702, 10122, 12667, 2851, 8308, 8575, 57229, 81239, 70246, 74210, 57682, 39361, 23949, 21992, 22146, 7732, 14629, 11881, 21421, 226, 51952, 92572, 86961, 129281, 38232, 45128, 22813, 11717, 38337, 75109, 129302, 27453, 113683, 15950, 40088, 54175, 38244, 32719, 53647, 1521, 5668, 109159, 450, 67134, 123254, 14582, 63479, 57801, 68357, 117093, 20524, 80643, 121060, 120809, 93113, 92847, 10333, 84050, 51747, 27776, 40925, 23494, 35788, 3162, 13377, 35997, 68451, 23938, 53401, 9078, 5172, 104568, 22335, 54635, 22634, 1340, 52474, 51255, 81765, 7562, 58464, 28067, 87031, 16852, 803, 2876, 57788, 22680, 34593, 40588, 75736, 1744, 32984, 8905, 50072, 29327, 122529, 22010, 2675, 100662, 68617, 113532, 124419, 37459, 6157, 45126, 110359, 92077, 52845, 45138, 103271, 4345, 126816, 69001, 54327, 5619, 56094, 2068, 6704, 41252, 9277, 4447, 7556, 7751, 25434, 37291, 41678, 77631, 107639, 66395, 113693, 18739, 71816, 40787, 94369, 9123, 22013, 89307, 27257, 783, 1287, 5494, 2957, 4341, 2395, 1420, 296, 2090, 483, 1217, 35633, 4653, 8791, 2315, 118, 5177, 712, 8601, 7185, 21479, 3894, 6951, 1379, 8831, 4082, 3371, 10606, 19494, 20225, 1109, 6467, 10554, 8987, 3728, 2522, 13716, 5752, 26855, 9093, 3327, 31143, 4478, 45718, 24565, 1240, 14191, 5554, 10242, 3874, 8458, 4243, 5051, 3967, 6071, 5663, 1502, 38060, 8439, 1699, 10882, 12166, 13274, 5236, 5665, 9554, 4070, 10719, 9080, 2377, 8680, 12550, 22164, 46822, 46382, 11493, 1221, 2600, 4666, 1119, 6707, 19894, 34856, 5560, 12658, 11147, 2588, 8310, 8227, 15057, 5624, 123, 900, 16573, 1080, 1571, 1103, 875, 32498, 45956, 20821, 32030, 38355, 54997, 16698, 4036, 27788, 201, 6881, 2225, 255, 7753, 46336, 8460, 711, 31192, 6356, 34055, 32391, 6830, 2868, 4037, 6659, 4428, 718, 562, 33257, 29500, 18030, 1793, 2308, 45964, 4840, 4852, 154, 1684, 717, 10267, 3517, 29377, 17117, 12138, 15847, 3203, 2982, 9400, 5706, 1526, 612, 5372, 58932, 68236, 37344, 33614, 12442, 28348, 13057, 1297, 1593, 8072, 26360, 24433, 32784, 2656, 26152, 8989, 43053, 44402, 31055, 44119, 68151, 28949, 220, 10318, 987, 6710, 18320, 7377, 16770, 6717, 35743, 36073, 56237, 1448, 19545, 2302, 35449, 21196, 1230, 1507, 463, 3009, 1602, 304, 1078, 5886, 1086, 6827, 13306, 22537, 5130, 16035, 811, 754, 4058, 2877, 12039, 8222, 12200, 10019, 7313, 672, 1455, 8139, 10961, 19561, 14773, 3480, 6326, 3062, 4763, 6459, 8472, 2460, 12153, 21065, 10338, 32342, 8232, 559, 6454, 63674, 10783, 815, 11009, 107848, 9090, 23952, 148, 30338, 114270, 25811, 187, 35784, 78527, 49239, 29778, 31188, 18017, 3973, 10588, 56658, 27194, 9941, 8789, 8703, 32264, 65756, 748, 82355, 43214, 18323, 2995, 15424, 18005, 49601, 35609, 76418, 11453, 5720, 13786, 31588, 4064, 45447, 4651, 7391, 18721, 64225, 93109, 37482, 54517, 101677, 134, 12863, 24480, 285, 14877, 7099, 5557, 37639, 16039, 96731, 1915, 70678, 56757, 24520, 72884, 6375, 596, 523, 6832, 13256, 785, 21904, 4507, 10043, 2568, 15323, 6005, 3048, 3826, 5241, 1883, 558, 5628, 3447, 7594, 4404, 32918, 6479, 20932, 6743, 2555, 9705, 9454, 39383, 23951, 715, 2209, 4649, 22874, 10894, 13807, 33466, 9463, 4126, 2531, 636, 458, 5492, 880, 15000, 5775, 35902, 20242, 5943, 11954, 990, 15431, 1342, 4619, 15260, 37299, 7287, 5137, 15928, 3229, 870, 3720, 7756, 23970, 16726, 11238, 4149, 2565, 3550, 2665, 3300, 2061, 121, 2509, 9041, 389, 7124, 8652, 294, 631, 640, 11138, 14796, 12241, 4500, 85190, 14671, 35479, 64021, 49702, 308, 10312, 797, 26174, 37040, 12021, 16099, 23166, 74249, 61184, 36160, 22353, 10889, 70128, 51675, 9251, 12851, 1513, 51435, 19158, 4494, 31988, 36491, 12233, 26796, 35853, 26126, 1102, 14465, 2041, 30917, 80290, 20391, 4976, 12795, 1681, 117445, 765, 11795, 5238, 34386, 12832, 10050, 47728, 8249, 27667, 19782, 16748, 10627, 33624, 3449, 15282, 47732, 25823, 63111, 102808, 4362, 19598, 29081, 11910, 25989, 20354, 24216, 66586, 5711, 3214, 63413, 23036, 38895, 12140, 10230, 6575, 24224, 52609, 3819, 44697, 13336, 91643, 85260, 39454, 4970, 9786, 60289, 51749, 22423, 24561, 26655, 7526, 7805, 25632, 20815, 1749, 8205, 10716, 13698, 2213, 2221, 76028, 35845, 86462, 29959, 63478, 9465, 10482, 32827, 8642, 80158, 19156, 64866, 17123, 31356, 12904, 48220, 133, 40413, 41688, 74749, 15143, 110740, 4326, 5198, 35312, 48723, 695, 52862, 18969, 103556, 354, 3462, 10258, 3873, 3019, 36693, 6457, 4612, 1197, 9527, 2579, 9049, 2202, 670, 6650, 11140, 2998, 4489, 11882, 996, 8319, 989, 16640, 1782, 1603, 565, 2664, 7765, 8388, 5672, 8856, 1522, 12152, 5511, 14677, 19594, 13743, 10782, 26163, 34663, 32894, 7503, 28772, 2231, 23824, 4147, 31177, 1714, 660, 2214, 23947, 1786, 3528, 5552, 1095, 27373, 4675, 14819, 14909, 12120, 12900, 778, 11456, 701, 10614, 8914, 5415, 9807, 863, 6784, 1450, 40481, 14846, 1676, 6363, 15520, 683, 2669, 8399, 21532, 3363, 570, 27992, 6367, 3477, 12020, 16506, 441, 9240, 431, 19018, 5005, 5503, 673, 2986, 29782, 8666, 5251, 8562, 6251, 6684, 9712, 4335, 3660, 829, 517, 11497, 9226, 3235, 4654, 571, 9574, 8653, 32896, 6345, 14148, 10328, 67422, 14131, 21870, 22913, 35854, 16388, 68576, 13999, 56816, 180, 23620, 48951, 86740, 21394, 17777, 46098, 24429, 7180, 1410, 7077, 31676, 35383, 73867, 40292, 23087, 31947, 5231, 43365, 43773, 68076, 34598, 26372, 34943, 293, 23813, 3824, 37760, 4194, 56368, 65028, 365, 15908, 49509, 2062, 70129, 106355, 4767, 891, 168, 1785, 23841, 4660, 22935, 12454, 51734, 24138, 31517, 3541, 1732, 6142, 3555, 8403, 4633, 5190, 25395, 9447, 1694, 4177, 5347, 38797, 5232, 2537, 51273, 28207, 11445, 7862, 61906, 8347, 20185, 6130, 2891, 5569, 4921, 5031, 3987, 146, 3361, 889, 3002, 29841, 44530, 8088, 13066, 73001, 7849, 35594, 36484, 8121, 16753, 18655, 35595, 12861, 40483, 57083, 9084, 27538, 524, 46582, 43663, 548, 4152, 17023, 9349, 3191, 41915, 53715, 17021, 15142, 76080, 4893, 3021, 5504, 199, 5770, 2895, 27956, 2220, 3696, 9581, 4322, 7921, 478, 8408, 25997, 423, 1584, 18034, 13052, 4492, 8999, 6879, 45560, 7523, 28105, 19271, 11489, 1677, 704, 11907, 9077, 40623, 26653, 6010, 5350, 10320, 84432, 8393, 11534, 11428, 5143, 5451, 7508, 7322, 21381, 286, 15564, 160, 15721, 11156, 8565, 174, 19189, 36574, 4628, 13387, 512, 2064, 16217, 26206, 2045, 35345, 2504, 9001, 2886, 1798, 1974, 3100, 27039, 3963, 4496, 3070, 16755, 5223, 536, 9183, 49603, 590, 13653, 12026, 18061, 26434, 35732, 26338, 21469, 7344, 32031, 6366, 1964, 8800, 224, 6612, 74948, 2574, 4429, 2049, 9800, 8300, 663, 2311, 357, 5499, 26978, 567, 4158, 373, 4168, 779, 1745, 10023, 3788, 773, 6705, 3017, 3693, 33256, 3551, 5321, 4074, 284, 8557, 782, 6703, 12017, 8391, 7675, 1729, 2808, 4086, 2993, 1213, 6729, 3663, 287, 2516, 16173, 2181, 16129, 6108, 3514, 1015, 2234, 9314, 3789, 4639, 13096, 8338, 1865, 43916, 98014, 100318, 92645, 26111, 10852, 105576, 31294, 3003, 32326, 21376, 26769, 15008, 19540, 11442, 32461, 48732, 23484, 34019, 15919, 114450, 23061, 4880, 53496, 95631, 78021, 47035, 35597, 76966, 71179, 85310, 12301, 41705, 37923, 110321, 7392, 65400, 6826, 56568, 117719, 161, 73159, 65504, 362, 115206, 12921, 56348, 17261, 3771, 12004, 127569, 18308, 20047, 1457, 6938, 40248, 25569, 76930, 36660, 71840, 73881, 92707, 45045, 44531, 707, 66472, 10245, 90543, 8296, 85855, 101897, 114264, 8940, 11573, 11570, 110125, 86810, 13365, 1294, 4878, 47112, 10401, 10243, 115181, 74554, 8201, 60930, 56440, 73050, 538, 34860, 19153, 97736, 20219, 8838, 97437, 18108, 14784, 90837, 75917, 9783, 35925, 9803, 33702, 73887, 39641, 33613, 2138, 11581, 81298, 17105, 23589, 73723, 36635, 8123, 114446, 12874, 65746, 45716, 115209, 14860, 58854, 75352, 10864, 244, 6573, 18431, 44689, 90836, 98469, 3013, 120147, 86826, 10890, 289, 5514, 101783, 18510, 43980, 78018, 34641, 58055, 291, 52439, 91480, 64622, 118245, 39381, 4581, 17920, 60849, 32705, 18012, 18940, 124338, 35123, 30029, 78351, 6336, 5030, 97731, 49818, 54873, 122124, 5677, 59011, 21885, 25805, 64842, 41700, 38553, 103726, 71191, 86661, 11431, 62753, 22798, 52987, 69012, 24445, 7389, 18421, 79554, 2238, 95340, 126566, 36598, 11002, 81941, 111128, 36301, 1453, 446, 19720, 23943, 115092, 14137, 23291, 14947, 90856, 115081, 41259, 51482, 85762, 91347, 9294, 70862, 21420, 22925, 53428, 4378, 53929, 25438, 110317, 40141, 68583, 1295, 20697, 73040, 25447, 28767, 92119, 3862, 97448, 8318, 126550, 31152, 73886, 2978, 37732, 63431, 2510, 51726, 51063, 93757, 5512, 19330, 129891, 71190, 32321, 36947, 4349, 64275, 120859, 65735, 79510, 48157, 24583, 12144, 35726, 22615, 27654, 11401, 44425, 102937, 20215, 24186, 3726, 126372, 15467, 12840, 29328, 125734, 40160, 81339, 56137, 75843, 14138, 24010, 39279, 3379, 50322, 14132, 106537, 59382, 43967, 86440, 39376, 115646, 18948, 49328, 40374, 18321, 25436, 51723, 76619, 35613, 81984, 35392, 7775, 26779, 41275, 137, 32088, 21588, 73045, 117733, 8192, 26856, 63415, 43977, 81296, 115856, 114285, 3917, 33621, 10012, 4482, 2242, 7192, 122117, 80679, 77186, 41971, 49796, 49563, 12231, 79657, 69604, 55193, 68587, 15567, 91489, 11742, 61126, 24522, 50515, 4912, 36643, 31276, 43972, 563, 97738], [91653, 4920, 38295, 6480, 86227, 18314, 25439, 300, 61142, 28769, 30521, 12161, 91456, 50280, 62296, 24578, 20046, 26118, 1189, 18025, 50543, 10386, 28759, 3018, 26689, 114155, 55268, 123840, 52971, 13655, 40250, 34883, 10863, 76974, 24457, 119942, 7281, 18098, 52665, 74055, 68521, 123897, 8984, 106351, 17046, 51989, 22137, 527, 126531, 21388, 82344, 12657, 104545, 6465, 25762, 43215, 126379, 64102, 109796, 19612, 15256, 38689, 13631, 21414, 38545, 12298, 105572, 30002, 43690, 103727, 35127, 16751, 89854, 16160, 15802, 93160, 103818, 101771, 60765, 70972, 56974, 89792, 8983, 14309, 694, 850, 4044, 6572, 2596, 6054, 6845, 5124, 950, 6088, 6273, 6118, 1447, 4472, 8116, 10630, 1739, 19104, 530, 7179, 11324, 11720, 3008, 6335, 29082, 16179, 1202, 11927, 9260, 5220, 5716, 7633, 6133, 7812, 2667, 14329, 15542, 5020, 19218, 4314, 24502, 19718, 525, 13691, 10395, 8290, 4240, 569, 1344, 4680, 3547, 3006, 2443, 16749, 13686, 32487, 21238, 2480, 13294, 39390, 26728, 30353, 6617, 16513, 649, 804, 34750, 23571, 1386, 39622, 2513, 24829, 20058, 22810, 22140, 3141, 5183, 16038, 24713, 8237, 2465, 6255, 45500, 11566, 6799, 9666, 44781, 3012, 36661, 4522, 15458, 1304, 5206, 1746, 14348, 17026, 5709, 2633, 3533, 862, 2187, 7795, 2244, 359, 20512, 7851, 12234, 8441, 49479, 60044, 19889, 4155, 9256, 27402, 4918, 38040, 49810, 5260, 13095, 21170, 26128, 36300, 17991, 84220, 39387, 56251, 42731, 8097, 40156, 34619, 13031, 7723, 8, 1209, 27873, 6691, 16448, 6440, 1882, 4410, 5761, 50548, 3024, 3799, 4685, 24714, 465, 8106, 44786, 183, 1038, 877, 432, 1430, 6339, 1506, 549, 55504, 19253, 37043, 76030, 26001, 79661, 31355, 57627, 93758, 8923, 20939, 6000, 64128, 4236, 11515, 10304, 45241, 32268, 40492, 27897, 21383, 13098, 84644, 6922, 53736, 1748, 10927, 19580, 20393, 218, 2533, 13798, 57721, 7757, 14111, 469, 53939, 16162, 18000, 8833, 60034, 21181, 33700, 18470, 3857, 22713, 19162, 18250, 6170, 21168, 353, 10097, 1884, 520, 22717, 17978, 4926, 9663, 1331, 2240, 7516, 7866, 1426, 2132, 8908, 4528, 4136, 3984, 169, 27853, 35126, 112, 14017, 6371, 13915, 1509, 47955, 5515, 67177, 57549, 4376, 6852, 6018, 3090, 2307, 6579, 6331, 39848, 5664, 4662, 11903, 1454, 6350, 5008, 2952, 1512, 2043, 4638, 15796, 5040, 3005, 9529, 1473, 8236, 617, 48103, 7758, 13390, 37021, 15157, 49731, 68827, 44236, 19065, 658, 2250, 5255, 648, 4382, 24584, 24427, 41227, 1343, 8890, 67412, 24844, 74478, 1880, 21759, 59690, 1329, 102807, 12934, 4156, 860, 18727, 50541, 8857, 35635, 4785, 67428, 91115, 7371, 6935, 52127, 83770, 8079, 116690, 11419, 14660, 38420, 8893, 6915, 18056, 77726, 6721, 3966, 10628, 47662, 70260, 81631, 16750, 17017, 95324, 44328, 6956, 37019, 67108, 10306, 43518, 110, 6450, 75513, 9179, 78291, 29374, 71177, 6125, 13790, 4518, 4775, 4898, 1323, 6365, 5764, 6322, 553, 20840, 3532, 3847, 8834, 5129, 29153, 13991, 9245, 8993, 10916, 3376, 15523, 41712, 25203, 7140, 11506, 6461, 16334, 3362, 27519, 12123, 352, 10918, 51724, 12913, 35203, 48207, 21578, 52773, 24764, 4663, 24413, 34929, 1734, 18499, 1576, 106, 9659, 641, 6056, 3688, 429, 5157, 1446, 1072, 14676, 10728, 80448, 83206, 17783, 22721, 21887, 48280, 1085, 79792, 9779, 11584, 16756, 21628, 9770, 5235, 2517, 71377, 48710, 75518, 64889, 20349, 6677, 31468, 20536, 67415, 40821, 23381, 50258, 44701, 34850, 24957, 1025, 6692, 3202, 21187, 1425, 11898, 170, 10024, 11556, 8919, 3853, 17772, 12252, 780, 12288, 23785, 14123, 9667, 13538, 19736, 8873, 44766, 24469, 12909, 19038, 20013, 13656, 9299, 1460, 6576, 32108, 44684, 35844, 11960, 26644, 26674, 10752, 9549, 2997, 51388, 12969, 2058, 10760, 1794, 36504, 630, 2888, 5046, 5738, 26095, 34355, 3276, 40534, 1872, 2526, 3958, 6357, 73713, 8570, 4084, 14003, 6946, 29056, 31954, 6676, 9458, 6124, 2681, 7286, 7779, 57875, 5715, 457, 848, 2239, 12008, 4419, 3837, 23371, 18717, 11559, 15069, 26772, 643, 4677, 4380, 3655, 6778, 23637, 9557, 6329, 5054, 16730, 5149, 23222, 31407, 10725, 6629, 541, 167, 9047, 9394, 2226, 4524, 453, 50812, 11440, 4593, 30379, 7766, 6916, 1118, 4148, 547, 10022, 3846, 8477, 659, 3177, 53658, 18045, 48521, 3388, 119626, 11323, 14998, 26793, 3370, 83809, 46419, 9778, 125793, 42733, 38445, 35454, 86748, 20687, 104365, 10321, 89056, 48503, 2521, 29389, 14116, 48524, 21226, 109778, 8238, 18002, 51988, 17727, 74067, 97656, 46435, 115800, 27282, 126051, 16370, 28063, 122100, 114553, 100439, 125852, 1458, 53651, 106983, 7505, 18661, 59987, 70860, 17009, 112354, 36993, 1434, 56003, 2542, 7780, 52681, 11578, 49737, 90545, 86737, 92889, 12910, 28191, 116610, 21165, 90351, 11567, 45290, 119639, 94756, 105879, 43639, 73192, 13523, 76295, 78802, 5229, 5133, 5227, 90724, 39623, 40327, 76979, 71316, 44172, 109974, 58484, 96288, 8096, 59695, 3855, 57613, 96316, 13540, 53661, 15632, 59403, 30162, 111658, 2054, 11329, 67007, 44166, 25452, 116784, 61717, 67199, 65819, 27339, 108237, 13525, 97655, 101098, 126304, 64086, 37888, 47141, 44251, 126800, 78797, 40355, 4771, 35201, 70439, 8231, 57607, 44249, 41389, 78740, 123793, 85469, 126787, 8315, 80052, 78038, 18656, 40425, 69477, 73683, 14659, 111665, 18806, 12291, 8091, 5217, 38539, 113077, 37887, 21874, 65827, 14661, 15689, 3659, 104000, 73681, 777, 37005, 3385, 51731, 51644, 77247, 9546, 70880, 115134, 25545, 26786, 110017, 74059, 7531, 64323, 59680, 35373, 14117, 3831, 121178, 3365, 546, 71305, 20703, 14113, 127547, 124600, 76303, 56016, 51030, 108618, 50357, 37037, 27866, 125789, 20374, 31364, 42727, 90546, 93252, 3856, 8407, 37890, 37671, 76093, 108624, 96152, 42399, 8555, 11098, 121175, 9175, 12907, 105165, 59818, 5122, 111666, 14110, 39632, 18020, 14675, 100999, 128006, 58596, 16861, 24139, 99302, 1438, 41956, 49474, 40430, 44580, 5116, 15515, 21615, 66875, 30790, 30800, 54687, 52858, 35019, 67197, 23799, 96157, 24415, 20926, 3661, 73679, 51032, 104826, 51999, 52943, 9516, 35004, 21116, 18240, 105150, 69475, 40290, 105198, 49287, 34862, 91628, 62178, 62418, 38693, 80862, 11552, 97758, 121819, 51935, 66882, 48198, 83323, 9519, 34698, 84271, 70143, 47663, 38697, 49551, 57748, 24137, 3646, 37977, 36232, 81924, 1881, 119691, 11545, 35859, 1077, 91634, 11501, 7028, 1585, 7375, 58507, 4643, 62246, 104087, 57610, 34993, 109973, 48728, 16331, 35401, 55498, 8917, 126311, 17102, 43220, 36239, 7802, 29151, 43211, 6128, 12229, 2875, 2044, 1503, 16332, 15795, 15205, 16848, 34637, 6374, 350, 5735, 8645, 10755, 363, 20051, 9076, 1229, 27920, 573, 28056, 6028, 3823, 6265, 27552, 5455, 2524, 1808, 1461, 556, 6476, 4645, 11402, 9178, 11195, 24430, 9440, 12911, 34357, 1731, 13477, 4652, 41845, 9555, 156, 34665, 4673, 2879, 8214, 4576, 13051, 7032, 6923, 9532, 3975, 26898, 8394, 22157, 561, 4768, 18443, 39450, 48291, 3387, 99173, 30216, 16373, 17781, 1873, 15487, 14355, 1452, 5221, 25825, 3865, 34265, 19723, 1416, 8549, 2991, 2511, 467, 93399, 48398, 12240, 9804, 8756, 13522, 90285, 3832, 9161, 12022, 9911, 34102, 107989, 45292, 21178, 27797, 55634, 76569, 27909, 16524, 1466, 9159, 1070, 851, 2691, 468, 18517, 7035, 18014, 44750, 37981, 27907, 25803, 3228, 713, 667, 12143, 16120, 6843, 4167, 4509, 12906, 4504, 13695, 2230, 144, 16124, 65604, 70318, 1220, 26545, 33810, 21764, 85722, 59813, 45801, 16376, 9541, 33255, 3974, 57753, 6824, 5212, 50263, 31200, 70450, 43216, 15150, 1886, 7937, 52878, 6352, 71175, 18003, 23810, 56418, 87328, 93397, 43801, 32734, 7750, 51385, 85159, 178, 552, 7036, 9383, 6658, 1462, 348, 1094, 1863, 9393, 16648, 5023, 1069, 8566, 24452, 5508, 4664, 1076, 4128, 6062, 1235, 5034, 16161, 35836, 9438, 8977, 74043, 25966, 15206, 4629, 4908, 104660, 22504, 3074, 135, 37878, 31485, 3745, 6735, 26767, 80600, 68655, 7797, 26383, 77254, 23376, 72326, 35676, 13917, 9611, 7794, 1074, 6438, 153, 19629, 15257, 37371, 22728, 46906, 4246, 1564, 15854, 1088, 28238, 62313, 83950, 9917, 51185, 31292, 5009, 3236, 1800, 10295, 1561, 49288, 20830, 15451, 23193, 2185, 1728, 35352, 64153, 20334, 6447, 17016, 3648, 10721, 26514, 15660, 1679, 21175, 1092, 44169, 41682, 1862, 1444, 4471, 15849, 3647, 8553, 85349, 7872, 13465, 90052, 112240, 39461, 72766, 38435, 54876, 99286, 54961, 59823, 48209, 19143, 122793, 21180, 10930, 1215, 37883, 38446, 17278, 14310, 32199, 108605, 121682, 25018, 32013, 6589, 57618, 59808, 9022, 114724, 78219, 89874, 73674, 50262, 42410, 80866, 48219, 87301, 8577, 88171, 23185, 94144, 29375, 75525, 67029, 38451, 25958, 112257, 7861, 45349, 37024, 71661, 21627, 48100, 65610, 80072, 41844, 59796, 9398, 59809, 86471, 124902, 2512, 4485, 9795, 39467, 45353, 20809, 71731, 77257, 120428, 106197, 63224, 48564, 43199, 68669, 121153, 15144, 44231, 20819, 78838, 119627, 50720, 105180, 2686, 8846, 19039, 57590, 123079, 28241, 90967, 4491, 9809, 43918, 89948, 15249, 99295, 26435, 31204, 14002, 5013, 45041, 24811, 6701, 55642, 37879, 11091, 8877, 85717, 12941, 127008, 3692, 41776, 1878, 63636, 86172, 27553, 115943, 75668, 1563, 21125, 22845, 13985, 77410, 21169, 90529, 6571, 1431, 35324, 84825, 80535, 106187, 3645, 20210, 59707, 554, 75669, 60960, 129549, 51645, 31218, 8569, 346, 1581, 687, 11543, 7760, 11194, 23039, 25964, 17731, 17283, 7876, 31414, 32511, 27077, 650, 7034, 14601, 17995, 290, 27578, 23587, 12965, 16530, 6442, 3861, 39692, 27357, 8871, 4516, 35393, 6448, 2462, 2587, 5214, 23969, 15552, 29305, 54132, 19031, 15461, 9455, 67097, 7880, 88690, 84268, 20193, 37736, 40471, 13472, 4141, 4620, 45995, 8874, 21512, 45814, 27854, 1423, 75943, 6702, 16308, 15838, 15856, 12961, 34352, 51351, 16651, 91629, 1569, 19321, 76882, 35951, 94118, 14126, 3549, 17282, 16324, 16501, 51903, 22852, 5721, 67027, 19318, 89860, 12950, 7521, 6429, 6261, 6689, 15190, 1579, 2394, 65216, 1583, 34283, 1559, 4520, 31426, 15186, 6736, 1228, 21189, 12954, 18090, 60005, 9606, 15185, 5506, 7409, 15848, 37360, 12905, 14599, 3644, 1567, 16116, 351, 30540, 3864, 32505, 8859, 1089, 112807, 17732, 76217, 54273, 25391, 4514, 22515, 6728, 67443, 65972, 8884, 17746, 117857, 116850, 53722, 84581, 47356, 57810, 41860, 34649, 28109, 15782, 1424, 36941, 103123, 10915, 104218, 53726, 112806, 128440, 29909, 27060, 17730, 6269, 27793, 52639, 104978, 59308, 6428, 1223, 103993, 17729, 87431, 59800, 73317, 93776, 2232, 53727, 33409, 66934, 22362, 6439, 59300, 97609, 9910, 1068, 100015, 46162, 48393, 19138, 69523, 23186, 98448, 85335, 1573, 33397, 84590, 17779, 98449, 11340, 15557, 124672, 73874, 27800, 103133, 1864, 75618, 22500, 6445, 18026, 6441, 28579, 1574, 37828, 1570, 21716, 1557, 28153, 2508, 84601, 59978, 8885, 16111, 5022, 122514, 48898, 44002, 358, 4523, 109471, 1560, 2245, 88870, 34509, 6430, 1572, 34503, 9904, 6680, 19126, 126246, 1083, 11188, 93768, 3643, 60003, 58377, 80069, 70032, 117205, 26908, 63763, 90912, 109417, 91846, 73314, 14597, 31999, 113566, 5913, 120282, 50801, 84826, 1577, 117207, 4503, 44643, 25383, 11090, 82824, 108647, 7793, 25331, 8872, 8206, 39378, 109232, 70031, 121895, 48896, 16532, 76010, 82819, 82822, 116148, 15841, 1565, 18793, 100003, 128268, 121888, 41838]]
region_id_group = [[125825, 56186, 7323, 39501, 28457, 26929, 40767, 20945, 50590, 109529, 6518, 118681, 16543, 69816, 57190, 102675, 54113, 12170, 30454, 28590, 101286, 102177, 47204, 95244, 26927, 36725, 6566, 13219, 7834, 34801, 1478, 28557, 23923, 12996, 2932, 54608, 46618, 47089, 110746, 74698, 120499, 55313, 12671, 17150, 2621, 99937, 89516, 34158, 10566, 5962, 21710, 110248, 12172, 110247, 5838, 48545, 34817, 33323, 111197, 17411, 121720, 48541, 16271, 76508, 21708, 47421, 86282, 94915, 19531, 39511, 14481, 29068, 23413, 19124, 5076, 11166, 2937, 23912, 2954, 50592, 4540, 21688, 13873, 9141, 79001, 12670, 10559, 53846, 54594, 38583, 63508, 30736, 8433, 31902, 68867, 14400, 92312, 246, 14080, 1705, 1372, 2830, 1058, 20988, 12608, 24682, 2325, 27369, 51797, 15236, 78333, 25510, 1623, 27235, 2006, 39497, 7234, 2923, 5379, 11594, 49241, 5390, 841, 768, 4824, 47110, 29985, 7590, 88161, 50842, 41092, 1266, 22203, 46393, 110238, 96408, 11995, 75591, 52272, 6528, 109574, 6624, 66990, 43604, 117169, 13403, 1851, 12368, 24060, 64505, 7664, 13747, 88801, 116389, 12369, 91998, 93189, 87372, 6548], [24119, 102203, 111877, 13348, 94341, 2343, 18227, 58406, 67284, 11943, 17493, 379, 4557, 1049, 725, 810, 1260, 6969, 1995, 742, 6988, 20036, 62082, 4, 12381, 2927, 42906, 26590, 43, 5843, 19990, 19696, 14272, 13625, 13766, 32519, 1060, 6397, 1848, 406, 747, 96148, 18613, 20, 1267, 4617, 3162, 5816, 2759, 1281, 2910, 53015, 5967, 10687, 13968, 13409, 1354, 401, 32355, 10356, 11978, 2742, 10818, 319, 903, 59654, 47095, 44174, 6, 83, 15350, 69, 5301, 1632, 13563, 25588, 1136, 22879, 60740, 21707, 6813, 2755, 733, 2320, 65079, 3990, 21054, 579, 33923, 4796, 325, 7256, 22045, 5826, 22693, 6202, 6175, 1760, 384, 3756, 15090, 81, 2335, 45, 720, 12579, 262, 489, 3249, 3736, 82895, 126102, 28682, 30765, 12475, 51800, 94413, 10713, 1719, 16231, 82965, 126988, 72107, 58295, 88295, 11527, 56863, 109247, 106426, 84852, 969, 49183, 72344, 78340, 68547, 620, 13665, 99153, 14278, 15238, 8795, 129332, 14645, 45879, 84851, 12071, 35041, 78488, 4464, 32528, 32526, 104021, 76049, 24321, 49864, 3979, 91380, 106444], [50924, 19427, 32574, 25286, 8145, 110597, 18557, 16691, 82970, 87979, 9276, 338, 2749, 911, 1551, 668, 10946, 3391, 3347, 3334, 9134, 17938, 328, 2417, 28334, 24213, 3996, 7015, 3848, 7630, 409, 1773, 426, 3753, 24336, 7251, 269, 241, 5286, 13612, 190, 18128, 17136, 10704, 24351, 7200, 17, 2264, 1397, 4171, 49, 229, 2718, 22399, 9692, 9212, 68113, 13556, 22971, 35157, 24070, 627, 102691, 1624, 213, 4949, 8424, 1652, 23924, 736, 574, 2896, 12218, 3274, 921, 39, 5265, 6970, 12449, 953, 1407, 16558, 11833, 184, 5527, 10154, 173, 18, 1332, 614, 2293, 109019, 84985, 9682, 26483, 21368, 47533, 337, 91823, 60392, 5402, 8625, 23815, 46604, 8732, 12181, 7623, 3230, 65239, 12576, 31616, 40958, 2821, 1139, 14535, 3107, 829, 2313, 1628, 235, 5268, 7672, 1124, 3876, 623, 4598, 1348, 54478, 16982, 6786, 32159, 30, 104, 5200, 1837, 2334, 4502, 380, 437, 14206, 17643, 33731, 39264, 22265, 52690, 573, 63319, 2722, 52270, 316, 317, 31, 3179, 1759, 1639, 3997, 308, 29639], [25178, 4793, 11394, 2449, 7137, 373, 198, 6006, 4105, 10951, 3159, 12263, 9102, 3323, 6871, 12748, 69774, 23451, 4205, 1546, 340, 13117, 1554, 1156, 2258, 14847, 25713, 511, 955, 59, 58067, 25467, 24645, 11007, 17209, 15, 35061, 63318, 1243, 14757, 2713, 307, 80, 1718, 2541, 2110, 295, 9279, 607, 3886, 618, 30664, 2445, 9748, 1199, 105941, 62063, 14900, 7150, 90003, 21488, 11029, 28321, 29692, 22688, 48040, 58336, 7809, 40689, 129761, 79762, 9496, 110298, 47295, 88412, 194, 112133, 9225, 86840, 64684, 25473, 34451, 121342, 10510, 84121, 10199, 83970, 5086, 98760, 1610, 58553, 6959, 14141, 24259, 71953, 100612, 6377, 63406, 111651, 47943, 80411, 110556, 50470, 11269, 16826, 98648, 55574, 111419, 53379, 62938, 5580, 9480, 3283, 3237, 597, 275, 1752, 942, 3473, 1119, 2167, 4175, 10819, 2493, 3901, 9021, 98, 9868, 1921, 18884, 1263, 10080, 5227, 738, 42874, 8735, 1141, 1779, 2697, 5960, 13157, 25574, 5853, 6109, 37, 2365, 378, 2346, 1484, 2789, 10864, 53574, 13882, 3350, 10146, 67388, 3227, 37313], [13300, 24736, 3344, 1500, 38510, 150, 35531, 2273, 30312, 24137, 245, 4331, 17955, 3616, 3812, 3083, 3238, 3511, 50, 8149, 16453, 33456, 97, 809, 1062, 933, 9856, 4594, 5266, 14173, 22798, 30733, 12772, 13063, 1030, 12400, 16885, 26309, 24689, 29, 17265, 30621, 798, 91, 4008, 10456, 210, 985, 498, 4077, 209, 3346, 3739, 89, 23, 719, 1121, 14044, 4701, 11087, 17178, 288, 1487, 7938, 1029, 82, 78, 53, 1915, 6990, 3418, 32777, 743, 10593, 87, 2599, 815, 8709, 428, 191, 274, 80875, 74179, 22028, 8171, 87769, 16225, 44312, 16625, 3863, 52417, 22694, 30836, 66616, 9864, 48053, 79627, 86008, 68908, 13570, 29577, 1064, 2816, 1004, 27, 1387, 4042, 1330, 9095, 3878, 3449, 5997, 587, 16901, 6000, 16409, 8267, 4789, 539, 4963, 857, 8276, 39629, 14199, 43407, 57, 572, 339, 1492, 1614, 26228, 5991, 5778, 2103, 3764, 153, 4986, 5355, 22916, 3250, 42854, 21323, 17726, 200, 27303, 55392, 6649, 9504, 31133, 2322, 3835, 1723, 7698, 834, 427, 3463, 3622, 2374], [1907, 14713, 41086, 486, 2206, 9084, 3452, 2134, 966, 2911, 19852, 1045, 4020, 2385, 15909, 48, 6766, 6036, 1893, 5291, 271, 5342, 1929, 29622, 1941, 4049, 34, 11425, 4764, 887, 99, 2728, 22, 20089, 9491, 615, 1241, 420, 697, 29285, 26196, 500, 141, 6783, 10318, 62, 1771, 926, 2362, 44, 40687, 152, 18596, 17718, 20844, 1105, 35, 1972, 520, 106783, 138, 44271, 20634, 28684, 42670, 96760, 17371, 10302, 32662, 90427, 107451, 9698, 44828, 123511, 4323, 28061, 62505, 5264, 30231, 16767, 93341, 7986, 7449, 31710, 61085, 63034, 19625, 12062, 10099, 6640, 111213, 23375, 6241, 66354, 27297, 10495, 67897, 27201, 20515, 48069, 19597, 67, 10405, 50385, 18767, 4185, 21800, 369, 54363, 96780, 11257, 114108, 7725, 3465, 9365, 62562, 83231, 37265, 247, 309, 1961, 1828, 109, 193, 1180, 4368, 29439, 240, 18300, 16670, 480, 1932, 314, 306, 1123, 659, 4207, 225, 370, 1875, 374, 16829, 7775, 2389, 182, 3714, 581, 4998, 1673, 5578, 165, 4885, 256, 71765, 1161, 12347, 253, 6971], [1891, 5886, 6161, 1803, 3256, 1461, 16820, 2620, 7653, 3716, 48633, 10288, 19658, 45979, 3343, 2215, 10533, 5369, 8696, 72, 5189, 634, 3887, 439, 1016, 1711, 1824, 3864, 559, 30530, 678, 16792, 26019, 19189, 4435, 443, 9014, 503, 553, 1475, 716, 507, 6188, 993, 525, 3978, 1299, 10541, 1061, 236, 88, 37344, 51747, 692, 2187, 16845, 10123, 799, 7326, 6155, 19, 1535, 12386, 2905, 957, 4068, 5724, 7820, 53324, 32310, 12543, 36054, 15743, 13973, 1404, 2502, 11243, 85051, 14143, 62663, 18452, 24549, 26986, 28359, 1905, 216, 4416, 35937, 176, 1120, 14821, 3457, 914, 448, 2842, 7235, 9339, 12904, 496, 965, 2354, 808, 11213, 3397, 300, 430, 159, 10336, 51822, 63927, 5372, 16935, 21239, 10638, 12879, 7563, 37788, 3002, 3478, 2116, 1320, 1686, 656, 797, 3853, 204, 7257, 6009, 39224, 9227, 60, 12118, 65, 124, 509, 2680, 16762, 4437, 3591, 6998, 52635, 17606, 456, 15142, 126, 10863, 7, 1591, 9, 2461, 1298, 3577, 41787, 3032, 4200, 1701, 3695, 1192], [2397, 5635, 7135, 7561, 7995, 1741, 20747, 12420, 305, 5590, 1215, 13441, 11107, 20913, 2286, 14763, 952, 442, 849, 112, 603, 33443, 73398, 64400, 39122, 108037, 45271, 46183, 6356, 24871, 4984, 12281, 41736, 109990, 4916, 9223, 7003, 1122, 71933, 129412, 100730, 6340, 86660, 43427, 23480, 118121, 7944, 76454, 95717, 37710, 101050, 18971, 100797, 19614, 66410, 1376, 101875, 56162, 36070, 21792, 26099, 95707, 11893, 96, 6160, 11, 8779, 812, 203, 1603, 2617, 293, 18363, 13305, 26973, 12318, 514, 3971, 2, 1390, 3, 1460, 1201, 268, 767, 19697, 711, 75, 5939, 84, 6717, 9737, 21402, 90, 5356, 450, 1009, 2267, 715, 5368, 995, 3475, 7939, 17756, 196, 4778, 1513, 5624, 5486, 123, 904, 3103, 212, 7777, 28093, 10291, 103535, 18949, 18945, 36, 543, 16769, 1708, 519, 3867, 7180, 8727, 73, 1729, 366, 20706, 2563, 1885, 312, 9400, 8, 48091, 665, 26378, 4421, 53368, 104840, 36599, 1085, 37380, 4878, 482, 170, 220, 424, 2682, 10061, 133, 1871, 10250, 1154, 4061, 8588], [47030, 27429, 47457, 5875, 877, 3717, 4897, 1008, 163, 33829, 595, 12533, 22130, 94, 2214, 905, 2882, 466, 4447, 7389, 19795, 5130, 16754, 156, 4240, 74383, 19494, 21660, 3016, 22422, 5001, 950, 2181, 37184, 86635, 4126, 588, 9668, 8639, 1, 161, 433, 2263, 782, 2845, 2307, 1987, 987, 481, 5921, 2470, 6748, 1166, 555, 5256, 26, 2448, 16, 18172, 57020, 6010, 62669, 8515, 34006, 56082, 75858, 14148, 34876, 292, 2675, 70423, 14188, 84232, 2595, 19613, 45980, 123701, 21789, 7525, 81312, 34867, 7692, 51766, 2670, 62247, 123727, 2588, 129281, 86045, 61884, 218, 653, 101515, 16016, 10776, 7302, 6674, 524, 143, 3399, 255, 248, 79, 145, 3011, 661, 436, 9031, 455, 164, 6834, 386, 9550, 5177, 1455, 294, 3565, 8866, 13974, 56303, 25743, 3262, 3828, 25448, 423, 1415, 64, 285, 287, 350, 4915, 16796, 41034, 30023, 74, 6736, 1321, 187, 1464, 865, 1450, 160, 1879, 357, 34701, 21922, 52000, 7374, 67644, 96637, 80158, 8831, 453, 7514, 282, 284, 9666, 137], [683, 34626, 86740, 9809, 66, 291, 4372, 16763, 16812, 445, 6460, 3955, 14773, 74902, 34743, 352, 3527, 483, 6742, 1575, 21679, 1217, 121, 472, 281, 10401, 25436, 34866, 68523, 73726, 42166, 35703, 126568, 64330, 90840, 10381, 70545, 60765, 38537, 86244, 80356, 103614, 348, 1521, 1190, 1567, 8390, 478, 270, 850, 549, 18735, 2877, 9783, 461, 4335, 1294, 13630, 4507, 800, 432, 3505, 4485, 781, 6367, 40657, 14114, 1512, 1685, 167, 851, 1643, 4970, 585, 15541, 878, 8185, 9770, 9440, 4922, 23146, 3826, 363, 1342, 5157, 88454, 4074, 3194, 346, 3964, 548, 40367, 1038, 19308, 1728, 26653, 15206, 23173, 33542, 2245, 46142, 3824, 5132, 3833, 2997, 1676, 51353, 1574, 2890, 5231, 4314, 3005, 4503, 77254, 1570, 102264, 24445, 32199, 358, 6731, 11202, 47112, 4520, 73323, 11191, 11194, 48905, 7409, 6745, 6572, 9768, 23968, 15653, 87429, 17731, 85528, 8909, 11195, 21723, 15500, 7336, 3064, 21724, 75952, 364, 11190, 10009, 98381, 5911, 5912, 21717, 48898, 48896, 354, 58367, 27596, 56962, 1564]]


In [ ]:
wine_train_list = wine_train_df.values.tolist()

In [ ]:
# 역순 리스트로 
wine_train_col_list = []
for i in range(len(wine_train_list[0])):
    tmp_list=[]
    for j in range(len(wine_train_list)):
        tmp_list.append(wine_train_list[j][i])
    wine_train_col_list.append(tmp_list)

In [ ]:
# grouping id list to object list 

# winery
for i in range(10):
    for j in range(len(winery_id_group[i])):
        index = wine_train_col_list[0].index(winery_id_group[i][j])
        winery_id_group[i][j] = wine_train_col_list[6][index]

# region
for i in range(10):
    for j in range(len(region_id_group[i])):
        index = wine_train_col_list[0].index(region_id_group[i][j])
        region_id_group[i][j] = wine_train_col_list[9][index]

In [ ]:
# grouping
def switch_to_str(num):
    if num == 1:
        num = 'one'
    elif num == 2:
        num = 'two'
    elif num == 3:
        num = 'three'
    elif num == 4:
        num = 'four'
    elif num == 5:
        num = 'five'
    elif num == 6:
        num = 'six'
    elif num == 7:
        num = 'seven'
    elif num == 8:
        num = 'eight'
    elif num == 9:
        num = 'nine'
    elif num == 10:
        num = 'ten'
    return num
# region(9)
count=0
for i in range (11):
    for j in range(len(wine_train_col_list[9])):
        if i==10:
            if not isinstance(wine_train_col_list[9][j], int):
                count+=1
        else:
            if wine_train_col_list[9][j] in region_id_group[i] and wine_train_col_list[9][j] is not np.nan and wine_train_col_list[9][j] is not None:
                wine_train_col_list[9][j] = switch_to_str(i+1)
print("region",count)

# variety(5)
count=0
for i in range (11):
    for j in range(len(wine_train_col_list[5])):
        if i ==10:
            if not isinstance(wine_train_col_list[5][j], int):
                count+=1
        else:
            if wine_train_col_list[5][j] in variety_group[i] and wine_train_col_list[5][j] is not np.nan and wine_train_col_list[5][j] is not None:
                wine_train_col_list[5][j] = switch_to_str(i+1)
print("variety",count)

# winery(6)
count=0
for i in range (11):
    for j in range(len(wine_train_col_list[6])):
        if i==10:
            if not isinstance(wine_train_col_list[6][j], int):
                count+=1
        else:
            if wine_train_col_list[6][j] in winery_id_group[i] and wine_train_col_list[6][j] is not np.nan and wine_train_col_list[6][j] is not None:
                wine_train_col_list[6][j] = switch_to_str(i+1)
print("winery", count)

# Description 에서 word_high_score, word_low_score 추출
## score = 단어의 score * description에서 나타난 횟수

In [ ]:
# 단어 점수 추출
word_high_score = []
word_low_score = []
for i in range(len(wine_train_col_list[2])):
    description = wine_train_col_list[2][i]
    count = 0;
    count2 = 0;
    for j in range(len(word_high_list)):
        occurence = description.count(word_high_list[j][0])
        count+=word_high_list[j][1]*occurence
    for j in range(len(word_low_list)):
        occurence2 = description.count(word_low_list[j][0])
        count2+=word_low_list[j][1]*occurence2
    word_high_score.append(count)
    word_low_score.append(count2)

print(len(word_high_score))
print(len(word_low_score))

wine_train_col_list.append(word_high_score)
wine_train_col_list.append(word_low_score)


In [ ]:
wine_train_fixed_list = []
for i in range(len(wine_train_col_list[0])):
    tmp_list=[]
    for j in range(len(wine_train_col_list)):
        tmp_list.append(wine_train_col_list[j][i])
    wine_train_fixed_list.append(tmp_list)

print(wine_train_fixed_list[0])

In [ ]:
from pandas import DataFrame

all_features = ['id', 'country', 'description', 'price', 'taster_name', 'variety', 'winery', 'points', 'length', 'region', 'word_high_score', 'word_low_score']
wine_train_df = DataFrame(wine_train_fixed_list,columns=all_features)
wine_train_df.head(3)

In [ ]:
plt.title('Original Wine high_score Histogram')
sns.distplot(wine_train_df['word_high_score'])

In [ ]:
plt.title('Original Wine low_score Histogram')
sns.distplot(wine_train_df['word_low_score'])

In [ ]:
plt.title('Log Transformed Wine high_score Histogram')
log_high = np.log1p(wine_train_df['word_high_score'])
sns.distplot(log_high)

In [ ]:
plt.title('Log Transformed Wine low_score Histogram')
log_low = np.log1p(wine_train_df['word_low_score'])
sns.distplot(log_low)

# Word_high_score, word_low_score log 변환

In [ ]:
wine_train_df['word_high_score'] = np.log1p(wine_train_df['word_high_score'])
wine_train_df['word_low_score'] = np.log1p(wine_train_df['word_low_score'])

wine_train_df.head(3)

In [ ]:
print('데이터 전체의 Shape:', wine_train_df.shape)
print('\n전체 피처의 type\n', wine_train_df.dtypes.value_counts())
isnull_series = wine_train_df.isnull().sum()/116974*100
print('\nNull 칼럼과 그 건수:\n', isnull_series[isnull_series>0].sort_values(ascending=False))


In [ ]:
# # 드롭하지 않는 숫자형 Null 칼럼은 평균값으로 대체(price)
# wine_train_df.fillna(wine_train_df.mean(), inplace=True)

# # Null 값이 있는 피처명과 타입을 추출
# null_column_count = wine_train_df.isnull().sum()[wine_train_df.isnull().sum()>0]
# print('Null 피처의 Type :\n', wine_train_df.dtypes[null_column_count.index])

# Classification based on Sentiment Analysis on description
## Word2Vec for word embedding, LSTM as classifier
## 목적칼럼: predict_class

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize

# functions
def points_to_class(points):
    if points in range(80,83):  # Acceptable: 80-82
        return 0
    elif points in range(83,87):  # Good: 83-86
        return 1
    elif points in range(87,90):  # Very Good: 87-89
        return 2
    elif points in range(90,94):  # Excellent: 90-93
        return 3
    else:  # Superb: 94-100
        return 4
    
def class_onehot(arr, num_class):
    return np.eye(num_class)[np.array(arr.astype(int)).reshape(-1)]

def description_cleaned(desc):
    desc = word_tokenize(desc.lower())
    desc = [token for token in desc if token not in stopwords and token.isalpha()]
    return detokenizer.detokenize(desc)

# parameters
max_length = 200
embedding_dimension = 300
batch_size = 256
number_of_classes = 5
epoch = 30

class_weights = {0: 7,
                1: 1,
                2: 1, 
                3: 1,
                4: 7}

stopwords = set(stopwords.words('english'))
detokenizer = TreebankWordDetokenizer()

# wine_train_df 에 'point_class', 'predict_class', 'description_cleaned' 생성
wine_train_df = wine_train_df.assign(predict_class=np.nan)
wine_train_df = wine_train_df.assign(description_cleaned=np.nan)
wine_train_df = wine_train_df.assign(point_class=np.nan)

wine_train_df["point_class"] = wine_train_df["points"].apply(points_to_class)
wine_train_df["description_cleaned"] = wine_train_df["description"].apply(description_cleaned)

y = class_onehot(wine_train_df["point_class"], number_of_classes)

print("Ready for Classification")

In [ ]:
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input, Activation, LSTM, Bidirectional, Dropout, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(wine_train_df["description_cleaned"], y, test_size = 0.05, random_state=0)

# Tokenizer Setting for Training
tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(X_train)

sequences_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences_train, maxlen=max_length)

sequences_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences_test, maxlen=max_length)
word_index = tokenizer.word_index


# Embedding using word2vec
embeddings_index = {}

f = open("../input/word2vec-sample/word2vec_sample/pruned.word2vec.txt", encoding = "utf8")

for line in f:
    values = line.split()
    word = ''.join(values[:-embedding_dimension])
    coefs = np.asarray(values[-embedding_dimension:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
                
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimension))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(len(word_index) + 1, embedding_dimension, weights = [embedding_matrix], input_length = max_length, trainable = False))
model.add(Bidirectional(LSTM(50, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dense(50, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(number_of_classes, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
    

checkpoint = ModelCheckpoint("model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor='val_loss', mode='min', patience=3)
callback = [checkpoint, early]
        
model.fit(X_train, y_train, batch_size=batch_size, epochs=epoch, validation_data=(X_test, y_test), callbacks=callback, class_weight=class_weights)

# predict_class
sequences_train = tokenizer.texts_to_sequences(wine_train_df['description_cleaned'])
X_train = pad_sequences(sequences_train, maxlen=max_length)

predict_class = model.predict(X_train)
predict_class = [np.argmax(x) for x in predict_class]
count = 0
for i in wine_train_df.index:
    wine_train_df.loc[i, 'predict_class'] = predict_class[count]
    count += 1
    
print('Classification Complete')

# TextBlob Sentiment Analysis on description
## 목적칼럼: polarity(정도), compound(neg, pos, neu 총체적점수)
### polarity, compound log 변환

In [ ]:
!pip install --upgrade pip
!pip install vaderSentiment
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
desc_blob = [TextBlob(desc) for desc in wine_train_df['description_cleaned']]
wine_train_df['polarity'] = [b.sentiment.polarity for b in desc_blob]
analyzer = SentimentIntensityAnalyzer()
wine_train_df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in wine_train_df['description_cleaned']]
wine_train_df['neg'] = [analyzer.polarity_scores(v)['neg'] for v in wine_train_df['description_cleaned']]
wine_train_df['neu'] = [analyzer.polarity_scores(v)['neu'] for v in wine_train_df['description_cleaned']]
wine_train_df['pos'] = [analyzer.polarity_scores(v)['pos'] for v in wine_train_df['description_cleaned']]


In [ ]:
wine_train_df['polarity'] =  np.log1p(wine_train_df['polarity'])
wine_train_df['compound'] = np.log1p(wine_train_df['compound'])

In [ ]:
wine_train_df.drop(['description', 'description_cleaned', 'point_class', 'neg', 'neu', 'pos'], axis = 1, inplace = True)

# OneHotEncoder, Imputer
## create new df wine_train_ohe

In [ ]:
# 미리 y_target 빼놓기 
y_target = wine_train_df['points']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

all_features = ['id', 'country', 'price', 'taster_name', 'length', 'region', 'winery', 'variety', 'word_high_score', 'word_low_score', 'predict_class', 'polarity', 'compound']
num_features = ['id', 'price', 'length', 'word_high_score', 'word_low_score', 'predict_class', 'polarity', 'compound']
cat_features = list(set(all_features)-set(num_features))

onehot = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_encoder = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', onehot)
])
wine_train_df_ohe = pd.DataFrame(OH_encoder.fit_transform(wine_train_df[cat_features]))
wine_train_df_ohe.index = wine_train_df.index
wine_train_df_ohe.columns = onehot.get_feature_names(cat_features)

imputer = SimpleImputer(strategy='mean')
wine_train_df_num = pd.DataFrame(imputer.fit_transform(wine_train_df[num_features]))
wine_train_df_num.index = wine_train_df.index
wine_train_df_num.columns = wine_train_df[num_features].copy().columns

In [ ]:
wine_train_df_ohe = pd.concat([wine_train_df_num, wine_train_df_ohe], axis=1)

# TEST MODELS
## 후보: LinearRegression, Ridge, Lasso, XGB, LGBM, 혼합, 스태킹

In [ ]:
def get_rmse(model):
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test,pred)
    rmse = np.sqrt(mse)
    print(model.__class__.__name__, ' 로그 변환된 RMSE:', np.round(rmse,3))
    return rmse

def get_rmses(models):
    rmses = []
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X_features = wine_train_df_ohe.drop(['id'], axis =1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state = 156)

# LinearRegression, Ridge, Lasso 학습, 예측, 평가
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)
lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

In [ ]:
def get_top_bottom_coef(model, n=10):
    # coef_ 속성을 기반으로 Series 객체를 생성. index 는 칼럼명.
    coef = pd.Series(model.coef_, index=X_features.columns)
    
    # + 상위 10개, - 하위 10개의 회귀 계수를 추출해 반환
    coef_high = coef.sort_values(ascending=False).head(n)
    coef_low = coef.sort_values(ascending=False).tail(n)
    return coef_high, coef_low

In [ ]:
def visualize_coefficient(models):
    # 3개 회귀 모델의 시각화를 위해 3개의 칼럼을 가지는 subplot 생성
    fig, axs = plt.subplots(figsize = (24,10), nrows=1, ncols=3)
    fig.tight_layout()
    # 입력인자로 받은 list 객체인 models에서 차례로 model을 추출해 회귀 계수 시각화.
    for i_num, model in enumerate(models):
        # 상위 10개, 하위 10개 회귀계수를 구하고, 이를 판다스 concat으로 결합
        coef_high, coef_low = get_top_bottom_coef(model)
        coef_concat = pd.concat([coef_high, coef_low])
        # ax_subplot에 barchar로 표현. 한 화면에 표현하기 위해 tick label 위치와 font 크기 조정
        axs[i_num].set_title(model.__class__.__name__+' Coefficients', size=25)
        axs[i_num].tick_params(axis="y", direction="in", pad = -120)
        for label in (axs[i_num].get_xticklabels() + axs[i_num].get_yticklabels()):
            label.set_fontsize(22)
        sns.barplot(x=coef_concat.values, y=coef_concat.index, ax = axs[i_num])

# 앞 예제에서 학습한 lr_reg, ridge_reg, lasso_reg 모델의 회귀 계수 시각화
models = [lr_reg, ridge_reg, lasso_reg]
visualize_coefficient(models)
    

In [ ]:
from sklearn.model_selection import cross_val_score

def get_avg_rmse_cv(models):
    
    for model in models:
        # 분할하지 않고 전체 데이터로 cross_val_score() 수행. 모델별 CV RMSE 값과 평균 RMSE 출력
        rmse_list = np.sqrt(-cross_val_score(model, X_features, y_target, scoring = "neg_mean_squared_error", cv = 5))
        rmse_avg = np.mean(rmse_list)
        print('\n{0} CV RMSE 값 리스트: {1}'.format(model.__class__.__name__, np.round(rmse_list, 3)))
        print('\n{0} CV 평균 RMSE 값: {1}'.format(model.__class__.__name__, np.round(rmse_avg, 3)))


# 앞 예제에서 학습한 lr_reg, ridge_reg, lasso_reg 모델의 CV RMSE 값 출력
models = [lr_reg, ridge_reg, lasso_reg]
get_avg_rmse_cv(models)


In [ ]:
from sklearn.model_selection import GridSearchCV

def print_best_params(model, params):
    grid_model = GridSearchCV(model, param_grid=params, scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X_features, y_target)
    rmse = np.sqrt(-1*grid_model.best_score_)
    print('{0} 5 CV 시 최적 평균 RMSE 값:{1}, 최적 alpha:{2}'.format(model.__class__.__name__, np.round(rmse,4), grid_model.best_params_))
    
ridge_params = {'alpha': [0.05,0.1,1,5,8,10,12,15,20]}
lasso_params = {'alpha': [0.001, 0.005, 0.008, 0.05, 0.03, 0.1, 0.5, 1, 5, 10]}
print_best_params(ridge_reg, ridge_params)
print_best_params(lasso_reg, lasso_params)

In [ ]:
# 앞의 최적화 alpha 값으로 학습 데이터로 학습, 테스트 데이터로 예측 및 평가 수행.

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
ridge_reg = Ridge(alpha = 5)
ridge_reg.fit(X_train, y_train)
lasso_reg = Lasso(alpha=0.0001)
lasso_reg.fit(X_train, y_train)

# 모든 모델의 RMSE 출력
models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

# 모든 모델의 회귀 계수 시각화
models = [lr_reg, ridge_reg, lasso_reg]
visualize_coefficient(models)

In [ ]:
from xgboost import XGBRegressor

xgb_params = {'n_estimators':[1000]}
xgb_reg = XGBRegressor(n_estimators=1000, learning_rate=0.05, colsample_bytree=0.5, subsample=0.8)
print_best_params(xgb_reg, xgb_params)

In [ ]:
from lightgbm import LGBMRegressor

lgbm_params = {'n_estimators':[1000]}
lgbm_reg = LGBMRegressor(n_estimators = 1000, learning_rate=0.05, num_leaves=4, subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)
print_best_params(lgbm_reg, lgbm_params)

In [ ]:
def get_rmse_pred(preds):
    for key in preds.keys():
        pred_value = preds[key]
        mse = mean_squared_error(y_test, pred_value)
        rmse = np.sqrt(mse)
        print('{0} 모델의 RMSE: {1}'.format(key,rmse))

In [ ]:
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
ridge_reg = Ridge(alpha=5)
ridge_reg.fit(X_train, y_train)
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

lr_pred= lr_reg.predict(X_test)
ridge_pred = ridge_reg.predict(X_test)
lasso_pred = lasso_reg.predict(X_test)

pred = 0.9*ridge_pred + 0.1*lr_pred

preds = {'최종혼합': pred, 'Linear': lr_pred, 'Ridge': ridge_pred, 'Lasso': lasso_pred}

get_rmse_pred(preds)

In [ ]:
xgb_reg.fit(X_train, y_train)
lgbm_reg.fit(X_train, y_train)
xgb_pred = xgb_reg.predict(X_test)
lgbm_pred = lgbm_reg.predict(X_test)

pred = 0.6 * xgb_pred + 0.4* lgbm_pred
preds = {'최종혼합': pred, 'XGB': xgb_pred, 'LGBM': lgbm_pred}

get_rmse_pred(preds)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds 값으로 KFold 생성
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=None)
    # 추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, 'model 시작 ')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print('\t 폴드 세트:', folder_counter, ' 시작 ')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr, y_tr)
        # 폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        # 입력된 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
        test_pred[:, folder_counter] = model.predict(X_test_n)
        
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    # train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred, test_pred_mean



In [ ]:
# get_stacking_base_datasets()는 넘파이 ndarray를 인자로 사용하므로 df 를 numpy로 변환

X_train_n = X_train.values
X_test_n = X_test.values
y_train_n = y_train.values

# 각 개별 기반 모델이 생성한 학습용, 테스트용 데이터 반환
ridge_train, ridge_test = get_stacking_base_datasets(ridge_reg, X_train_n, y_train_n, X_test_n, 5)
lasso_train, lasso_test = get_stacking_base_datasets(lasso_reg, X_train_n, y_train_n, X_test_n, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb_reg, X_train_n, y_train_n, X_test_n, 5)
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_reg, X_train_n, y_train_n, X_test_n, 5)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_lasso = Lasso(alpha=0.0005)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_lasso.fit(Stack_final_X_train, y_train)
final = meta_model_lasso.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_lasso = Lasso(alpha=0.0001)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_lasso.fit(Stack_final_X_train, y_train)
final = meta_model_lasso.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_ridge = Ridge(alpha=5)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_ridge.fit(Stack_final_X_train, y_train)
final = meta_model_ridge.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_xgb = XGBRegressor(n_estimators=1000, learning_rate=0.05, colsample_bytree=0.5, subsample=0.8)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_xgb.fit(Stack_final_X_train, y_train)
final = meta_model_xgb.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_lgbm = LGBMRegressor(n_estimators = 1000, learning_rate=0.05, num_leaves=4, subsample=0.8, colsample_bytree=0.5, reg_lambda=10, n_jobs=-1)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_lgbm.fit(Stack_final_X_train, y_train)
final = meta_model_lgbm.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
# 각 개별 기반 모델이 생성한 학습용, 테스트용 데이터 반환
lr_train, lr_test = get_stacking_base_datasets(lr_reg, X_train_n, y_train_n, X_test_n, 5)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((lr_train, ridge_train, lasso_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((lr_test, ridge_test, lasso_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_ridge = Ridge(alpha=5)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_ridge.fit(Stack_final_X_train, y_train)
final = meta_model_ridge.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((lr_train, ridge_train, lasso_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((lr_test, ridge_test, lasso_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_ridge = Lasso(alpha=0.0001)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_ridge.fit(Stack_final_X_train, y_train)
final = meta_model_ridge.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((lr_train, ridge_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((lr_test, ridge_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_ridge = Ridge(alpha=5)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_ridge.fit(Stack_final_X_train, y_train)
final = meta_model_ridge.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((ridge_train, xgb_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, xgb_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_ridge = Ridge(alpha=5)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_ridge.fit(Stack_final_X_train, y_train)
final = meta_model_ridge.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((ridge_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_lr = LinearRegression()

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_lr.fit(Stack_final_X_train, y_train)
final = meta_model_lr.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

# FEATURE ENGINEERING WINE_TEST_DF (train_df와 같음)

In [ ]:
# description length
length_list = []
for description in wine_test_df['description']:
    length_list.append(len(description))
wine_test_df['length'] = length_list
wine_test_df.head(3)

In [ ]:
# region = region_1 fill with province
wine_test_df['region'] = np.where(pd.notnull(wine_test_df['region_1']) == True, wine_test_df['region_1'], wine_test_df['province'])
wine_test_df.head(3)

In [ ]:
# Price 로그 변환
original_price = wine_test_df['price']
wine_test_df['price'] = np.log1p(wine_test_df['price'])

In [ ]:
# Null이 너무 많은 칼럼과 불필요한 칼럼 삭제
wine_test_df.drop(["designation","province","region_1","region_2","taster_twitter_handle","title"], axis = 1, inplace = True)

In [ ]:
wine_test_df

In [ ]:
wine_test_list = wine_test_df.values.tolist()
# 역순 리스트로 
wine_test_col_list = []
for i in range(len(wine_test_list[0])):
    tmp_list=[]
    for j in range(len(wine_test_list)):
        tmp_list.append(wine_test_list[j][i])
    wine_test_col_list.append(tmp_list)

In [ ]:
# grouping
def switch_to_str(num):
    if num == 1:
        num = 'one'
    elif num == 2:
        num = 'two'
    elif num == 3:
        num = 'three'
    elif num == 4:
        num = 'four'
    elif num == 5:
        num = 'five'
    elif num == 6:
        num = 'six'
    elif num == 7:
        num = 'seven'
    elif num == 8:
        num = 'eight'
    elif num == 9:
        num = 'nine'
    elif num == 10:
        num = 'ten'
    return num
# region(8)
count=0
for i in range (11):
    for j in range(len(wine_test_col_list[8])):
        if i==10:
            if wine_test_col_list[8][j]!='one' and wine_test_col_list[8][j]!='two' and wine_test_col_list[8][j]!='three' and wine_test_col_list[8][j]!='four' and wine_test_col_list[8][j]!='five' and wine_test_col_list[8][j]!='six' and wine_test_col_list[8][j]!='seven' and wine_test_col_list[8][j]!='eight' and wine_test_col_list[8][j]!='nine' and wine_test_col_list[8][j]!='ten':
                wine_test_col_list[8][j] = np.nan
                count+=1
        else:
            if wine_test_col_list[8][j] in region_id_group[i] and wine_test_col_list[8][j] is not np.nan and wine_test_col_list[8][j] is not None:
                wine_test_col_list[8][j] = switch_to_str(i+1)
print("region",count)

# variety(5)
count=0
for i in range (11):
    for j in range(len(wine_test_col_list[5])):
        if i ==10:
            if wine_test_col_list[5][j]!='one' and wine_test_col_list[5][j]!='two' and wine_test_col_list[5][j]!='three' and wine_test_col_list[5][j]!='four' and wine_test_col_list[5][j]!='five' and wine_test_col_list[5][j]!='six' and wine_test_col_list[5][j]!='seven' and wine_test_col_list[5][j]!='eight' and wine_test_col_list[5][j]!='nine' and wine_test_col_list[5][j]!='ten':
                wine_test_col_list[5][j] = np.nan
                count+=1
        else:
            if wine_test_col_list[5][j] in variety_group[i] and wine_test_col_list[5][j] is not np.nan and wine_test_col_list[5][j] is not None:
                wine_test_col_list[5][j] = switch_to_str(i+1)
print("variety",count)

# winery(6)
count=0
for i in range (11):
    for j in range(len(wine_test_col_list[6])):
        if i==10:
            if wine_test_col_list[6][j]!='one' and wine_test_col_list[6][j]!='two' and wine_test_col_list[6][j]!='three' and wine_test_col_list[6][j]!='four' and wine_test_col_list[6][j]!='five' and wine_test_col_list[6][j]!='six' and wine_test_col_list[6][j]!='seven' and wine_test_col_list[6][j]!='eight' and wine_test_col_list[6][j]!='nine' and wine_test_col_list[6][j]!='ten':
                wine_test_col_list[6][j] = np.nan
                count+=1
        else:
            if wine_test_col_list[6][j] in winery_id_group[i] and wine_test_col_list[6][j] is not np.nan and wine_test_col_list[6][j] is not None:
                wine_test_col_list[6][j] = switch_to_str(i+1)
print("winery", count)

In [ ]:
# 단어 점수 추출
word_high_score = []
word_low_score = []
for i in range(len(wine_test_col_list[2])):
    description = wine_test_col_list[2][i]
    count = 0;
    count2 = 0;
    for j in range(len(word_high_list)):
        occurence = description.count(word_high_list[j][0])
        count+=word_high_list[j][1]*occurence
    for j in range(len(word_low_list)):
        occurence2 = description.count(word_low_list[j][0])
        count2+=word_low_list[j][1]*occurence2
    word_high_score.append(count)
    word_low_score.append(count2)

print(len(word_high_score))
print(len(word_low_score))

wine_test_col_list.append(word_high_score)
wine_test_col_list.append(word_low_score)



In [ ]:
wine_test_fixed_list = []
for i in range(len(wine_test_col_list[0])):
    tmp_list=[]
    for j in range(len(wine_test_col_list)):
        tmp_list.append(wine_test_col_list[j][i])
    wine_test_fixed_list.append(tmp_list)

print(wine_test_fixed_list[0])

In [ ]:
from pandas import DataFrame

wine_test_df = DataFrame (wine_test_fixed_list, columns=['id', 'country', 'description', 'price', 'taster_name', 'variety', 'winery', 'length', 'region', 'word_high_score', 'word_low_score'])
wine_test_df.head(3)

In [ ]:
wine_test_df = wine_test_df.assign(predict_class=np.nan)
wine_test_df = wine_test_df.assign(description_cleaned=np.nan)

wine_test_df["description_cleaned"] = wine_test_df["description"].apply(description_cleaned)

print('Ready for Classification')

sequences_test = tokenizer.texts_to_sequences(wine_test_df['description_cleaned'])
X_test = pad_sequences(sequences_test, maxlen=max_length)

predict_class_test = model.predict(X_test)
predict_class_test = [np.argmax(x) for x in predict_class_test]
count = 0
for i in wine_test_df.index:
    wine_test_df.loc[i, 'predict_class'] = predict_class_test[count]
    count += 1
    
print('Classification_Complete')

In [ ]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
desc_blob = [TextBlob(desc) for desc in wine_test_df['description_cleaned']]
wine_test_df['polarity'] = [b.sentiment.polarity for b in desc_blob]
analyzer = SentimentIntensityAnalyzer()
wine_test_df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in wine_test_df['description_cleaned']]
wine_test_df['neg'] = [analyzer.polarity_scores(v)['neg'] for v in wine_test_df['description_cleaned']]
wine_test_df['neu'] = [analyzer.polarity_scores(v)['neu'] for v in wine_test_df['description_cleaned']]
wine_test_df['pos'] = [analyzer.polarity_scores(v)['pos'] for v in wine_test_df['description_cleaned']]


In [ ]:
wine_test_df.drop(['description', 'description_cleaned', 'neg', 'neu', 'pos'], axis = 1, inplace = True)

In [ ]:
wine_test_df['word_high_score'] = np.log1p(wine_test_df['word_high_score'])
wine_test_df['word_low_score'] = np.log1p(wine_test_df['word_low_score'])
wine_test_df['polarity'] = np.log1p(wine_test_df['polarity'])
wine_test_df['compound'] = np.log1p(wine_test_df['compound'])

In [ ]:
# 드롭하지 않는 숫자형 Null 칼럼은 평균값으로 대체(price)
wine_test_df.fillna(wine_test_df.mean(), inplace=True)

In [ ]:
wine_test_df_ohe = pd.DataFrame(OH_encoder.transform(wine_test_df[cat_features]))
wine_test_df_ohe.index = wine_test_df.index
wine_test_df_ohe.columns = onehot.get_feature_names(cat_features)
print(wine_test_df_ohe.shape)

wine_test_df_num = pd.DataFrame(imputer.transform(wine_test_df[num_features]))
wine_test_df_num.index = wine_test_df.index
wine_test_df_num.columns = wine_test_df[num_features].copy().columns

In [ ]:
wine_test_df_ohe = pd.concat([wine_test_df_num, wine_test_df_ohe], axis=1)

# Final Model: Stacking(Ridge, Lasso, XGB, LGBM), MetaModel = Ridge(alpha=5)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 ?? 모델을 적용
meta_model_ridge = Ridge(alpha=5)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습/테스트 데이터로 메타 모델 예측 및 RSME 측정
meta_model_ridge.fit(Stack_final_X_train, y_train)
final = meta_model_ridge.predict(Stack_final_X_test)
mse = mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RSME 값은: ', rmse)

In [ ]:
X_test_data = wine_test_df_ohe.drop('id', axis =1, inplace=False)
lasso_predict = lasso_reg.predict(X_test_data.values).reshape(-1,1)
ridge_predict = ridge_reg.predict(X_test_data.values).reshape(-1,1)
xgb_predict = xgb_reg.predict(X_test_data.values).reshape(-1,1)
lgbm_predict = lgbm_reg.predict(X_test_data.values).reshape(-1,1)
final_X_test = np.concatenate((ridge_predict, lasso_predict, xgb_predict, lgbm_predict), axis=1)
predict = meta_model_ridge.predict(final_X_test)

# Make submission
my_submission = pd.DataFrame({'id': wine_test_df.id, 'points': predict})
my_submission.to_csv('wine_submission_stacking_feature_select1', index=False)

print("Submission Complete")

In [ ]:
final_X_test